# 703 Homework Assignment: Disease-Causing Mutations

Author: Matthew Crowson

## 0. Setup: 


### Library Loading

- VariantAnnotation
- org.Hs.eg.db 
- TxDb.Hsapiens.UCSC.hg19.knownGene


In [66]:
source("/home/ec2-user/SageMaker/studies/BMI-703-2021-READ-only/BMI703/VCFs/loadLibraries.R")

### 1. Read the 3 VCF files from a family

Use the Jupyter infrastructure to scan each of the following genes of this family who has a child with seizures: SCN1A, SCN2A, SCN3A, SCN4A, SCN5A, SCN7A, SCN8A, SCN9A, SCN10A, SCN11A, NGLY1, AAAS, GMPPA.

#### 1.0 Obtain Entrez IDs and genomic ranges (DNA coordinates) for genes of interest

In [67]:
# Identify the genes of interest

genesym <- c("SCN1A", "SCN2A", "SCN3A", "SCN4A", "SCN5A", "SCN7A", "SCN8A", "SCN9A", "SCN10A", "SCN11A", "NGLY1", "AAAS", "GMPPA")
geneid <- select(x       = org.Hs.eg.db,
                 keys    = genesym,
                 keytype = "SYMBOL",
                 columns = "ENTREZID")
geneid

'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns



SYMBOL,ENTREZID
<chr>,<chr>
SCN1A,6323
SCN2A,6326
SCN3A,6328
SCN4A,6329
SCN5A,6331
SCN7A,6332
SCN8A,6334
SCN9A,6335
SCN10A,6336


SYMBOL,ENTREZID
<chr>,<chr>
SCN1A,6323
SCN2A,6326
SCN3A,6328
SCN4A,6329
SCN5A,6331
SCN7A,6332
SCN8A,6334
SCN9A,6335
SCN10A,6336


We then need to figure out from which chromosomes these genes come from in order to subset our search in later steps.

Looking at the _NCBI database_, we see that the genes are on the chromosomes: **2, 3, 12, 17**

In [68]:
# Load the txdb annotation set

txdb <- TxDb.Hsapiens.UCSC.hg19.knownGene

In [69]:
# Check the sequence levels of the txdb annotation set and the format
head(seqlevels(txdb))

[1] "2"  "3"  "12" "17"

[1] "2"  "3"  "12" "17"

We see that the chromosomes have "chr" prefixes. This needs to be changed in addition to subsetting the chromosomes 2, 3, 12, 17.

In [72]:
txdb <- restoreSeqlevels(txdb)
txdb <- renameSeqlevels(txdb, 
                        gsub("chr", "", seqlevels(txdb)))
# subset to chromosomes 2, 3, 12, 17
txdbgene <- keepSeqlevels(txdb, c("2","3","12","17"))

In [74]:
# Verify the format 
head(seqlevels(txdbgene))

[1] "2"  "3"  "12" "17"

[1] "2"  "3"  "12" "17"

In [75]:
txbygene <- transcriptsBy(txdbgene, "gene")
txbygene

GRangesList object of length 5260:
$`100009676`
GRanges object with 1 range and 2 metadata columns:
      seqnames              ranges strand |     tx_id     tx_name
         <Rle>           <IRanges>  <Rle> | <integer> <character>
  [1]        3 101395274-101398057      + |     14200  uc003dvg.3
  -------
  seqinfo: 4 sequences from hg19 genome

$`10010`
GRanges object with 3 ranges and 2 metadata columns:
      seqnames              ranges strand |     tx_id     tx_name
         <Rle>           <IRanges>  <Rle> | <integer> <character>
  [1]        2 161993466-162061490      + |      9524  uc002ubq.1
  [2]        2 161993466-162092683      + |      9525  uc002ubr.2
  [3]        2 162016939-162092683      + |      9526  uc002ubs.3
  -------
  seqinfo: 4 sequences from hg19 genome

$`100113381`
GRanges object with 1 range and 2 metadata columns:
      seqnames            ranges strand |     tx_id     tx_name
         <Rle>         <IRanges>  <Rle> | <integer> <character>
  [1]        3 

GRangesList object of length 5260:
$`100009676`
GRanges object with 1 range and 2 metadata columns:
      seqnames              ranges strand |     tx_id     tx_name
         <Rle>           <IRanges>  <Rle> | <integer> <character>
  [1]        3 101395274-101398057      + |     14200  uc003dvg.3
  -------
  seqinfo: 4 sequences from hg19 genome

$`10010`
GRanges object with 3 ranges and 2 metadata columns:
      seqnames              ranges strand |     tx_id     tx_name
         <Rle>           <IRanges>  <Rle> | <integer> <character>
  [1]        2 161993466-162061490      + |      9524  uc002ubq.1
  [2]        2 161993466-162092683      + |      9525  uc002ubr.2
  [3]        2 162016939-162092683      + |      9526  uc002ubs.3
  -------
  seqinfo: 4 sequences from hg19 genome

$`100113381`
GRanges object with 1 range and 2 metadata columns:
      seqnames            ranges strand |     tx_id     tx_name
         <Rle>         <IRanges>  <Rle> | <integer> <character>
  [1]        3 

In [76]:
# Create a vector of genomic ranges for our genes of interest
gnrng <- unlist(range(txbygene[geneid$ENTREZID]), use.names=FALSE)
gnrng

GRanges object with 13 ranges and 0 metadata columns:
       seqnames              ranges strand
          <Rle>           <IRanges>  <Rle>
   [1]        2 166845670-167005642      -
   [2]        2 166095912-166248820      +
   [3]        2 165944030-166060577      -
   [4]       17   62015914-62050278      -
   [5]        3   38589553-38691164      -
   ...      ...                 ...    ...
   [9]        3   38738837-38835501      -
  [10]        3   38887260-38992052      -
  [11]        3   25760435-25831530      -
  [12]       12   53701240-53715412      -
  [13]        2 220363587-220371718      +
  -------
  seqinfo: 4 sequences from hg19 genome

GRanges object with 13 ranges and 0 metadata columns:
       seqnames              ranges strand
          <Rle>           <IRanges>  <Rle>
   [1]        2 166845670-167005642      -
   [2]        2 166095912-166248820      +
   [3]        2 165944030-166060577      -
   [4]       17   62015914-62050278      -
   [5]        3   38589553-38691164      -
   ...      ...                 ...    ...
   [9]        3   38738837-38835501      -
  [10]        3   38887260-38992052      -
  [11]        3   25760435-25831530      -
  [12]       12   53701240-53715412      -
  [13]        2 220363587-220371718      +
  -------
  seqinfo: 4 sequences from hg19 genome

In [77]:
# Incorporate the gene names
names(gnrng) <- geneid$SYMBOL
gnrng

GRanges object with 13 ranges and 0 metadata columns:
         seqnames              ranges strand
            <Rle>           <IRanges>  <Rle>
   SCN1A        2 166845670-167005642      -
   SCN2A        2 166095912-166248820      +
   SCN3A        2 165944030-166060577      -
   SCN4A       17   62015914-62050278      -
   SCN5A        3   38589553-38691164      -
     ...      ...                 ...    ...
  SCN10A        3   38738837-38835501      -
  SCN11A        3   38887260-38992052      -
   NGLY1        3   25760435-25831530      -
    AAAS       12   53701240-53715412      -
   GMPPA        2 220363587-220371718      +
  -------
  seqinfo: 4 sequences from hg19 genome

GRanges object with 13 ranges and 0 metadata columns:
         seqnames              ranges strand
            <Rle>           <IRanges>  <Rle>
   SCN1A        2 166845670-167005642      -
   SCN2A        2 166095912-166248820      +
   SCN3A        2 165944030-166060577      -
   SCN4A       17   62015914-62050278      -
   SCN5A        3   38589553-38691164      -
     ...      ...                 ...    ...
  SCN10A        3   38738837-38835501      -
  SCN11A        3   38887260-38992052      -
   NGLY1        3   25760435-25831530      -
    AAAS       12   53701240-53715412      -
   GMPPA        2 220363587-220371718      +
  -------
  seqinfo: 4 sequences from hg19 genome

Next, we read in the VCF files for the mother, father, and child.

#### 1.1 Read in van2555 (MOTHER)

In [78]:
van2555<-readVcf(file = "van2555.analysisReady.annotated.vcf.gz",genome = "hg19")

In [79]:
hdr <- scanVcfHeader(file = "van2555.analysisReady.annotated.vcf.gz")
# create the index file
indexTabix(file = "van2555.analysisReady.annotated.vcf.gz", format="vcf")

[1] "van2555.analysisReady.annotated.vcf.gz.tbi"

[1] "van2555.analysisReady.annotated.vcf.gz.tbi"

In [80]:
# Read only the "GT" field of the VCF file
param.gt_m = ScanVcfParam(geno = c("GT"), info = NA)
gt.vcf_m = readVcf(file = "van2555.analysisReady.annotated.vcf.gz", genome = "hg19", param = param.gt_m)

In [81]:
# Extract genetic variants belonging to the genes of interest
param_m <- ScanVcfParam(which = gnrng, 
                      geno  = c("GT"))

In [82]:
vcf_m <- readVcf( file   = "van2555.analysisReady.annotated.vcf.gz", 
                genome = "hg19", 
                param  = param_m)
vcf_m

class: CollapsedVCF 
dim: 285 1 
rowRanges(vcf):
  GRanges with 5 metadata columns: paramRangeID, REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 21 columns: AC, AF, AN, BaseQRankSum, DB, DP, DS, Dels, FS,...
info(header(vcf)):
                   Number Type    Description                                  
   AC              A      Integer Allele count in genotypes, for each ALT al...
   AF              A      Float   Allele Frequency, for each ALT allele, in ...
   AN              1      Integer Total number of alleles in called genotypes  
   BaseQRankSum    1      Float   Z-score from Wilcoxon rank sum test of Alt...
   DB              0      Flag    dbSNP Membership                             
   DP              1      Integer Approximate read depth; some reads may hav...
   DS              0      Flag    Were any of the samples downsampled?         
   Dels            1      Float   Fraction of Reads Containing Spanning Dele...
   FS              1      Float   Phred-scaled p-

class: CollapsedVCF 
dim: 285 1 
rowRanges(vcf):
  GRanges with 5 metadata columns: paramRangeID, REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 21 columns: AC, AF, AN, BaseQRankSum, DB, DP, DS, Dels, FS,...
info(header(vcf)):
                   Number Type    Description                                  
   AC              A      Integer Allele count in genotypes, for each ALT al...
   AF              A      Float   Allele Frequency, for each ALT allele, in ...
   AN              1      Integer Total number of alleles in called genotypes  
   BaseQRankSum    1      Float   Z-score from Wilcoxon rank sum test of Alt...
   DB              0      Flag    dbSNP Membership                             
   DP              1      Integer Approximate read depth; some reads may hav...
   DS              0      Flag    Were any of the samples downsampled?         
   Dels            1      Float   Fraction of Reads Containing Spanning Dele...
   FS              1      Float   Phred-scaled p-

In [83]:
# Examine the head of the mother's VCF file
head(rowRanges(vcf_m), 5)

GRanges object with 5 ranges and 5 metadata columns:
              seqnames              ranges strand | paramRangeID            REF
                 <Rle>           <IRanges>  <Rle> |     <factor> <DNAStringSet>
   rs34871628        2           166849154      * |        SCN1A              T
     rs568376        2           166870600      * |        SCN1A              A
     rs577306        2           166871905      * |        SCN1A              C
  rs116647099        2           166894209      * |        SCN1A              T
   rs72132828        2 166895913-166895914      * |        SCN1A             CT
                             ALT      QUAL      FILTER
              <DNAStringSetList> <numeric> <character>
   rs34871628                 TA     86.23        PASS
     rs568376                  G   1452.88        PASS
     rs577306                  A   1449.08        PASS
  rs116647099                  C    923.44        PASS
   rs72132828                  C     35.03    QDFilter
  

GRanges object with 5 ranges and 5 metadata columns:
              seqnames              ranges strand | paramRangeID            REF
                 <Rle>           <IRanges>  <Rle> |     <factor> <DNAStringSet>
   rs34871628        2           166849154      * |        SCN1A              T
     rs568376        2           166870600      * |        SCN1A              A
     rs577306        2           166871905      * |        SCN1A              C
  rs116647099        2           166894209      * |        SCN1A              T
   rs72132828        2 166895913-166895914      * |        SCN1A             CT
                             ALT      QUAL      FILTER
              <DNAStringSetList> <numeric> <character>
   rs34871628                 TA     86.23        PASS
     rs568376                  G   1452.88        PASS
     rs577306                  A   1449.08        PASS
  rs116647099                  C    923.44        PASS
   rs72132828                  C     35.03    QDFilter
  

Repeat for the father.

#### 1.2 Read in van2556 (FATHER)

In [84]:
van2556<-readVcf(file = "van2556.analysisReady.annotated.vcf.gz",genome = "hg19")

In [85]:
hdr <- scanVcfHeader(file = "van2556.analysisReady.annotated.vcf.gz")
# creating the index file
indexTabix(file = "van2556.analysisReady.annotated.vcf.gz", format="vcf")

[1] "van2556.analysisReady.annotated.vcf.gz.tbi"

[1] "van2556.analysisReady.annotated.vcf.gz.tbi"

In [86]:
# Read only the "GT" field of the VCF file
param.gt_f = ScanVcfParam(geno = c("GT"), info = NA)
gt.vcf_f = readVcf(file = "van2556.analysisReady.annotated.vcf.gz", genome = "hg19", param = param.gt_f)

In [87]:
# Extract genetic variants belonging to the genes of interest
param_f <- ScanVcfParam(which = gnrng, 
                      geno  = c("GT"))

In [88]:
vcf_f <- readVcf( file   = "van2556.analysisReady.annotated.vcf.gz", 
                genome = "hg19", 
                param  = param_f)
vcf_f

class: CollapsedVCF 
dim: 331 1 
rowRanges(vcf):
  GRanges with 5 metadata columns: paramRangeID, REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 21 columns: AC, AF, AN, BaseQRankSum, DB, DP, DS, Dels, FS,...
info(header(vcf)):
                   Number Type    Description                                  
   AC              A      Integer Allele count in genotypes, for each ALT al...
   AF              A      Float   Allele Frequency, for each ALT allele, in ...
   AN              1      Integer Total number of alleles in called genotypes  
   BaseQRankSum    1      Float   Z-score from Wilcoxon rank sum test of Alt...
   DB              0      Flag    dbSNP Membership                             
   DP              1      Integer Approximate read depth; some reads may hav...
   DS              0      Flag    Were any of the samples downsampled?         
   Dels            1      Float   Fraction of Reads Containing Spanning Dele...
   FS              1      Float   Phred-scaled p-

class: CollapsedVCF 
dim: 331 1 
rowRanges(vcf):
  GRanges with 5 metadata columns: paramRangeID, REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 21 columns: AC, AF, AN, BaseQRankSum, DB, DP, DS, Dels, FS,...
info(header(vcf)):
                   Number Type    Description                                  
   AC              A      Integer Allele count in genotypes, for each ALT al...
   AF              A      Float   Allele Frequency, for each ALT allele, in ...
   AN              1      Integer Total number of alleles in called genotypes  
   BaseQRankSum    1      Float   Z-score from Wilcoxon rank sum test of Alt...
   DB              0      Flag    dbSNP Membership                             
   DP              1      Integer Approximate read depth; some reads may hav...
   DS              0      Flag    Were any of the samples downsampled?         
   Dels            1      Float   Fraction of Reads Containing Spanning Dele...
   FS              1      Float   Phred-scaled p-

In [89]:
head(rowRanges(vcf_f), 5)

GRanges object with 5 ranges and 5 metadata columns:
             seqnames              ranges strand | paramRangeID            REF
                <Rle>           <IRanges>  <Rle> |     <factor> <DNAStringSet>
  rs34871628        2           166849154      * |        SCN1A              T
  rs67967509        2 166851160-166851168      * |        SCN1A      AAAACAAAC
    rs568376        2           166870600      * |        SCN1A              A
    rs577306        2           166871905      * |        SCN1A              C
   rs1834839        2           166885888      * |        SCN1A              A
                            ALT      QUAL      FILTER
             <DNAStringSetList> <numeric> <character>
  rs34871628                 TA     76.25        PASS
  rs67967509                  A    486.75        PASS
    rs568376                  G   1173.08        PASS
    rs577306                  A    769.97        PASS
   rs1834839                  G     49.99        PASS
  -------
  seqi

GRanges object with 5 ranges and 5 metadata columns:
             seqnames              ranges strand | paramRangeID            REF
                <Rle>           <IRanges>  <Rle> |     <factor> <DNAStringSet>
  rs34871628        2           166849154      * |        SCN1A              T
  rs67967509        2 166851160-166851168      * |        SCN1A      AAAACAAAC
    rs568376        2           166870600      * |        SCN1A              A
    rs577306        2           166871905      * |        SCN1A              C
   rs1834839        2           166885888      * |        SCN1A              A
                            ALT      QUAL      FILTER
             <DNAStringSetList> <numeric> <character>
  rs34871628                 TA     76.25        PASS
  rs67967509                  A    486.75        PASS
    rs568376                  G   1173.08        PASS
    rs577306                  A    769.97        PASS
   rs1834839                  G     49.99        PASS
  -------
  seqi

Repeat for the child.

#### 1.3 Read in van2554 (CHILD)

In [90]:
van2554<-readVcf(file = "van2554.analysisReady.annotated.vcf.gz",genome = "hg19")

In [91]:
hdr <- scanVcfHeader(file = "van2554.analysisReady.annotated.vcf.gz")
# creating the index file
indexTabix(file = "van2554.analysisReady.annotated.vcf.gz", format="vcf")

[1] "van2554.analysisReady.annotated.vcf.gz.tbi"

[1] "van2554.analysisReady.annotated.vcf.gz.tbi"

In [92]:
# Read only the "GT" field of the VCF file
param.gt_c = ScanVcfParam(geno = c("GT"), info = NA)
gt.vcf_c = readVcf(file = "van2554.analysisReady.annotated.vcf.gz", genome = "hg19", param = param.gt_c)

In [93]:
# Extract genetic variants belonging to the genes of interest
param_c <- ScanVcfParam(which = gnrng, 
                      geno  = c("GT"))

In [94]:
vcf_c <- readVcf( file   = "van2554.analysisReady.annotated.vcf.gz", 
                genome = "hg19", 
                param  = param_c)
vcf_c

class: CollapsedVCF 
dim: 336 1 
rowRanges(vcf):
  GRanges with 5 metadata columns: paramRangeID, REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 21 columns: AC, AF, AN, BaseQRankSum, DB, DP, DS, Dels, FS,...
info(header(vcf)):
                   Number Type    Description                                  
   AC              A      Integer Allele count in genotypes, for each ALT al...
   AF              A      Float   Allele Frequency, for each ALT allele, in ...
   AN              1      Integer Total number of alleles in called genotypes  
   BaseQRankSum    1      Float   Z-score from Wilcoxon rank sum test of Alt...
   DB              0      Flag    dbSNP Membership                             
   DP              1      Integer Approximate read depth; some reads may hav...
   DS              0      Flag    Were any of the samples downsampled?         
   Dels            1      Float   Fraction of Reads Containing Spanning Dele...
   FS              1      Float   Phred-scaled p-

class: CollapsedVCF 
dim: 336 1 
rowRanges(vcf):
  GRanges with 5 metadata columns: paramRangeID, REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 21 columns: AC, AF, AN, BaseQRankSum, DB, DP, DS, Dels, FS,...
info(header(vcf)):
                   Number Type    Description                                  
   AC              A      Integer Allele count in genotypes, for each ALT al...
   AF              A      Float   Allele Frequency, for each ALT allele, in ...
   AN              1      Integer Total number of alleles in called genotypes  
   BaseQRankSum    1      Float   Z-score from Wilcoxon rank sum test of Alt...
   DB              0      Flag    dbSNP Membership                             
   DP              1      Integer Approximate read depth; some reads may hav...
   DS              0      Flag    Were any of the samples downsampled?         
   Dels            1      Float   Fraction of Reads Containing Spanning Dele...
   FS              1      Float   Phred-scaled p-

In [95]:
head(rowRanges(vcf_c), 5)

GRanges object with 5 ranges and 5 metadata columns:
              seqnames              ranges strand | paramRangeID            REF
                 <Rle>           <IRanges>  <Rle> |     <factor> <DNAStringSet>
   rs34871628        2           166849154      * |        SCN1A              T
   rs67967509        2 166851160-166851164      * |        SCN1A          AAAAC
     rs568376        2           166870600      * |        SCN1A              A
     rs577306        2           166871905      * |        SCN1A              C
  rs140274879        2 166892444-166892445      * |        SCN1A             TA
                             ALT      QUAL      FILTER
              <DNAStringSetList> <numeric> <character>
   rs34871628                 TA    255.10        PASS
   rs67967509                  A    277.00        PASS
     rs568376                  G   1505.50        PASS
     rs577306                  A   1643.66        PASS
  rs140274879                  T    136.42    QDFilter
  

GRanges object with 5 ranges and 5 metadata columns:
              seqnames              ranges strand | paramRangeID            REF
                 <Rle>           <IRanges>  <Rle> |     <factor> <DNAStringSet>
   rs34871628        2           166849154      * |        SCN1A              T
   rs67967509        2 166851160-166851164      * |        SCN1A          AAAAC
     rs568376        2           166870600      * |        SCN1A              A
     rs577306        2           166871905      * |        SCN1A              C
  rs140274879        2 166892444-166892445      * |        SCN1A             TA
                             ALT      QUAL      FILTER
              <DNAStringSetList> <numeric> <character>
   rs34871628                 TA    255.10        PASS
   rs67967509                  A    277.00        PASS
     rs568376                  G   1505.50        PASS
     rs577306                  A   1643.66        PASS
  rs140274879                  T    136.42    QDFilter
  

### 2. Detect variants

Prompt:

_Rows (variants) in these VCF files are either labeled by Reference SNP (rs) accession numbers if the variant has been seen before or a label that exactly describes the location and characteristics of the variant if it has not.
Given these lists of identifiers, determine which of the variants are:_
* _new in the child (i.e., “de novo”)_
* _inherited from the parents_
* _Which variants are inherited from the mother and which from the father?_

_You can safely assume that if the variant is not present in the VCF files of the mother and father but is present in the VCF of the child, it is a de novo mutation_

#### 2.0 Child's Variants

In [96]:
# First, we examine all of the variants in the child's VCF file
row.names(vcf_c)
length(row.names(vcf_c))

[1] "rs34871628"                       "rs67967509"                      
  [3] "rs568376"                         "rs577306"                        
  [5] "rs140274879"                      "rs72132828"                      
  [7] "rs490317"                         "2:166897989_G/A"                 
  [9] "2:166900130_T/G"                  "rs115015575"                     
 [11] "rs538921"                         "rs78391141"                      
 [13] "rs34747425"                       "2:166929347_C/T"                 
 [15] "rs566839"                         "rs353133"                        
 [17] "rs353134"                         "rs353135"                        
 [19] "rs353126"                         "rs2075704"                       
 [21] "rs7593568"                        "rs112475855"                     
 [23] "rs1965757"                        "rs2116658"                       
 [25] "rs13025009"                       "rs2121371"                       
 [27] "rs1838846"                        "rs1867864"                       
 [29] "rs12993173"                       "rs10191771"                      
 [31] "rs67198220"                       "rs3769951"                       
 [33] "rs36121220"                       "rs3835935"                       
 [35] "rs12692768"                       "rs150568699"                     
 [37] "rs6432821"                        "rs1368238"                       
 [39] "rs7578237"                        "rs10178662"                      
 [41] "rs2060198"                        "rs11885001"                      
 [43] "rs73019878"                       "rs11393050"                      
 [45] "rs1946892"                        "rs2289400"                       
 [47] "rs79777698"                       "rs10048748"                      
 [49] "rs2289401"                        "rs16850131"                      
 [51] "rs58539679"                       "rs16850134"                      
 [53] "rs7574983"                        "rs7561140"                       
 [55] "rs3731761"                        "rs3731760"                       
 [57] "2:166012356_G/T"                  "rs13011371"                      
 [59] "rs13010846"                       "rs6710092"                       
 [61] "rs56124029"                       "rs59172781"                      
 [63] "rs6756990"                        "rs72264475"                      
 [65] "rs11887309"                       "rs11887343"                      
 [67] "rs9333576"                        "rs2304710"                       
 [69] "rs7556825"                        "rs7579000"                       
 [71] "rs116682517"                      "rs11903711"                      
 [73] "rs11903799"                       "rs4667794"                       
 [75] "rs4667795"                        "rs4667796"                       
 [77] "rs11891394"                       "rs3754962"                       
 [79] "rs3754961"                        "rs3754960"                       
 [81] "rs16822785"                       "rs34971284"                      
 [83] "rs17819028"                       "rs10930221"                      
 [85] "2:167273403_T/C"                  "rs7561953"                       
 [87] "rs72623153"                       "rs72623155"                      
 [89] "rs6738031"                        "rs4550683"                       
 [91] "rs3791253"                        "rs113442768"                     
 [93] "rs138641247"                      "rs71428927"                      
 [95] "rs56951383"                       "rs7593207"                       
 [97] "rs13417885"                       "rs2293567"                       
 [99] "rs150350335"                      "rs7565062"                       
[101] "rs56164822"                       "rs10169523"                      
[103] "rs10180721"                       "2:167060900_C/T"                 
[105] "rs77944059"                       "rs341413

[1] 336

[1] "rs34871628"                       "rs67967509"                      
  [3] "rs568376"                         "rs577306"                        
  [5] "rs140274879"                      "rs72132828"                      
  [7] "rs490317"                         "2:166897989_G/A"                 
  [9] "2:166900130_T/G"                  "rs115015575"                     
 [11] "rs538921"                         "rs78391141"                      
 [13] "rs34747425"                       "2:166929347_C/T"                 
 [15] "rs566839"                         "rs353133"                        
 [17] "rs353134"                         "rs353135"                        
 [19] "rs353126"                         "rs2075704"                       
 [21] "rs7593568"                        "rs112475855"                     
 [23] "rs1965757"                        "rs2116658"                       
 [25] "rs13025009"                       "rs2121371"                       
 [27] "rs1838846"                        "rs1867864"                       
 [29] "rs12993173"                       "rs10191771"                      
 [31] "rs67198220"                       "rs3769951"                       
 [33] "rs36121220"                       "rs3835935"                       
 [35] "rs12692768"                       "rs150568699"                     
 [37] "rs6432821"                        "rs1368238"                       
 [39] "rs7578237"                        "rs10178662"                      
 [41] "rs2060198"                        "rs11885001"                      
 [43] "rs73019878"                       "rs11393050"                      
 [45] "rs1946892"                        "rs2289400"                       
 [47] "rs79777698"                       "rs10048748"                      
 [49] "rs2289401"                        "rs16850131"                      
 [51] "rs58539679"                       "rs16850134"                      
 [53] "rs7574983"                        "rs7561140"                       
 [55] "rs3731761"                        "rs3731760"                       
 [57] "2:166012356_G/T"                  "rs13011371"                      
 [59] "rs13010846"                       "rs6710092"                       
 [61] "rs56124029"                       "rs59172781"                      
 [63] "rs6756990"                        "rs72264475"                      
 [65] "rs11887309"                       "rs11887343"                      
 [67] "rs9333576"                        "rs2304710"                       
 [69] "rs7556825"                        "rs7579000"                       
 [71] "rs116682517"                      "rs11903711"                      
 [73] "rs11903799"                       "rs4667794"                       
 [75] "rs4667795"                        "rs4667796"                       
 [77] "rs11891394"                       "rs3754962"                       
 [79] "rs3754961"                        "rs3754960"                       
 [81] "rs16822785"                       "rs34971284"                      
 [83] "rs17819028"                       "rs10930221"                      
 [85] "2:167273403_T/C"                  "rs7561953"                       
 [87] "rs72623153"                       "rs72623155"                      
 [89] "rs6738031"                        "rs4550683"                       
 [91] "rs3791253"                        "rs113442768"                     
 [93] "rs138641247"                      "rs71428927"                      
 [95] "rs56951383"                       "rs7593207"                       
 [97] "rs13417885"                       "rs2293567"                       
 [99] "rs150350335"                      "rs7565062"                       
[101] "rs56164822"                       "rs10169523"                      
[103] "rs10180721"                       "2:167060900_C/T"                 
[105] "rs77944059"                       "rs341413

[1] 336

In [97]:
# Extract the variant IDs into a list for further processing.
vcf_c_l = lapply(vcf_c, rownames)
vcf_c_l

$rs34871628
[1] "rs34871628"

$rs67967509
[1] "rs67967509"

$rs568376
[1] "rs568376"

$rs577306
[1] "rs577306"

$rs140274879
[1] "rs140274879"

$rs72132828
[1] "rs72132828"

$rs490317
[1] "rs490317"

$`2:166897989_G/A`
[1] "2:166897989_G/A"

$`2:166900130_T/G`
[1] "2:166900130_T/G"

$rs115015575
[1] "rs115015575"

$rs538921
[1] "rs538921"

$rs78391141
[1] "rs78391141"

$rs34747425
[1] "rs34747425"

$`2:166929347_C/T`
[1] "2:166929347_C/T"

$rs566839
[1] "rs566839"

$rs353133
[1] "rs353133"

$rs353134
[1] "rs353134"

$rs353135
[1] "rs353135"

$rs353126
[1] "rs353126"

$rs2075704
[1] "rs2075704"

$rs7593568
[1] "rs7593568"

$rs112475855
[1] "rs112475855"

$rs1965757
[1] "rs1965757"

$rs2116658
[1] "rs2116658"

$rs13025009
[1] "rs13025009"

$rs2121371
[1] "rs2121371"

$rs1838846
[1] "rs1838846"

$rs1867864
[1] "rs1867864"

$rs12993173
[1] "rs12993173"

$rs10191771
[1] "rs10191771"

$rs67198220
[1] "rs67198220"

$rs3769951
[1] "rs3769951"

$rs36121220
[1] "rs36121220"

$rs3835935
[1] "rs3835935"

$rs12692768
[1] "rs12692768"

$rs150568699
[1] "rs150568699"

$rs6432821
[1] "rs6432821"

$rs1368238
[1] "rs1368238"

$rs7578237
[1] "rs7578237"

$rs10178662
[1] "rs10178662"

$rs2060198
[1] "rs2060198"

$rs11885001
[1] "rs11885001"

$rs73019878
[1] "rs73019878"

$rs11393050
[1] "rs11393050"

$rs1946892
[1] "rs1946892"

$rs2289400
[1] "rs2289400"

$rs79777698
[1] "rs79777698"

$rs10048748
[1] "rs10048748"

$rs2289401
[1] "rs2289401"

$rs16850131
[1] "rs16850131"

$rs58539679
[1] "rs58539679"

$rs16850134
[1] "rs16850134"

$rs7574983
[1] "rs7574983"

$rs7561140
[1] "rs7561140"

$rs3731761
[1] "rs3731761"

$rs3731760
[1] "rs3731760"

$`2:166012356_G/T`
[1] "2:166012356_G/T"

$rs13011371
[1] "rs13011371"

$rs13010846
[1] "rs13010846"

$rs6710092
[1] "rs6710092"

$rs56124029
[1] "rs56124029"

$rs59172781
[1] "rs59172781"

$rs6756990
[1] "rs6756990"

$rs72264475
[1] "rs72264475"

$rs11887309
[1] "rs11887309"

$rs11887343
[1] "rs11887343"

$rs9333576
[1] "rs9333576"

$rs2304710
[1] "rs2304710"

$rs7556825
[1] "rs7556825"

$rs7579000
[1] "rs7579000"

$rs116682517
[1] "rs116682517"

$rs11903711
[1] "rs11903711"

$rs11903799
[1] "rs11903799"

$rs4667794
[1] "rs4667794"

$rs4667795
[1] "rs4667795"

$rs4667796
[1] "rs4667796"

$rs11891394
[1] "rs11891394"

$rs3754962
[1] "rs3754962"

$rs3754961
[1] "rs3754961"

$rs3754960
[1] "rs3754960"

$rs16822785
[1] "rs16822785"

$rs34971284
[1] "rs34971284"

$rs17819028
[1] "rs17819028"

$rs10930221
[1] "rs10930221"

$`2:167273403_T/C`
[1] "2:167273403_T/C"

$rs7561953
[1] "rs7561953"

$rs72623153
[1] "rs72623153"

$rs72623155
[1] "rs72623155"

$rs6738031
[1] "rs6738031"

$rs4550683
[1] "rs4550683"

$rs3791253
[1] "rs3791253"

$rs113442768
[1] "rs113442768"

$rs138641247
[1] "rs138641247"

$rs71428927
[1] "rs71428927"

$rs56951383
[1] "rs56951383"

$rs7593207
[1] "rs7593207"

$rs13417885
[1] "rs13417885"

$rs2293567
[1] "rs2293567"

$rs150350335
[1] "rs150350335"

$rs7565062
[1] "rs7565062"

$rs56164822
[1] "rs56164822"

$rs10169523
[1] "rs10169523"

$rs10180721
[1] "rs10180721"

$`2:167060900_C/T`
[1] "2:167060900_C/T"

$rs77944059
[1] "rs77944059"

$rs34141377
[1] "rs34141377"

$rs7595255
[1] "rs7595255"

$rs3793936
[1] "rs3793936"

$rs77684130
[1] "rs77684130"

$`2:167085081_T/G`
[1] "2:167085081_T/G"

$rs10199799
[1] "rs10199799"

$rs58519838
[1] "rs58519838"

$`2:167085142_T/A`
[1] "2:167085142_T/A"

$rs61701909
[1] "rs61701909"

$rs722204
[1] "rs722204"

$rs6746030
[1] "rs6746030"

$rs71428907
[1] "rs71428907"

$rs35664204
[1] "rs35664204"

$rs10930215
[1] "rs10930215"

$rs6432893
[1] "rs6432893"

$rs13007020
[1] "rs13007020"

$rs71395215
[1] "rs71395215"

$rs7607896
[1] "rs7607896"

$rs7608201
[1] "rs7608201"

$rs9917231
[1] "rs9917231"

$rs4278943
[1] "rs4278943"

$rs9917396
[1] "rs9917396"

$rs113914877
[1] "rs113914877"

$rs9917401
[1] "rs9917401"

$rs9917250
[1] "rs9917250"

$rs10584581
[1] "rs10584581"

$rs7596100
[1] "rs7596100"

$rs6747673
[1] "rs6747673"

$rs7588632
[1] "rs7588632"

$rs12994338


$rs34871628
[1] "rs34871628"

$rs67967509
[1] "rs67967509"

$rs568376
[1] "rs568376"

$rs577306
[1] "rs577306"

$rs140274879
[1] "rs140274879"

$rs72132828
[1] "rs72132828"

$rs490317
[1] "rs490317"

$`2:166897989_G/A`
[1] "2:166897989_G/A"

$`2:166900130_T/G`
[1] "2:166900130_T/G"

$rs115015575
[1] "rs115015575"

$rs538921
[1] "rs538921"

$rs78391141
[1] "rs78391141"

$rs34747425
[1] "rs34747425"

$`2:166929347_C/T`
[1] "2:166929347_C/T"

$rs566839
[1] "rs566839"

$rs353133
[1] "rs353133"

$rs353134
[1] "rs353134"

$rs353135
[1] "rs353135"

$rs353126
[1] "rs353126"

$rs2075704
[1] "rs2075704"

$rs7593568
[1] "rs7593568"

$rs112475855
[1] "rs112475855"

$rs1965757
[1] "rs1965757"

$rs2116658
[1] "rs2116658"

$rs13025009
[1] "rs13025009"

$rs2121371
[1] "rs2121371"

$rs1838846
[1] "rs1838846"

$rs1867864
[1] "rs1867864"

$rs12993173
[1] "rs12993173"

$rs10191771
[1] "rs10191771"

$rs67198220
[1] "rs67198220"

$rs3769951
[1] "rs3769951"

$rs36121220
[1] "rs36121220"

$rs3835935
[1] "rs3835935"

$rs12692768
[1] "rs12692768"

$rs150568699
[1] "rs150568699"

$rs6432821
[1] "rs6432821"

$rs1368238
[1] "rs1368238"

$rs7578237
[1] "rs7578237"

$rs10178662
[1] "rs10178662"

$rs2060198
[1] "rs2060198"

$rs11885001
[1] "rs11885001"

$rs73019878
[1] "rs73019878"

$rs11393050
[1] "rs11393050"

$rs1946892
[1] "rs1946892"

$rs2289400
[1] "rs2289400"

$rs79777698
[1] "rs79777698"

$rs10048748
[1] "rs10048748"

$rs2289401
[1] "rs2289401"

$rs16850131
[1] "rs16850131"

$rs58539679
[1] "rs58539679"

$rs16850134
[1] "rs16850134"

$rs7574983
[1] "rs7574983"

$rs7561140
[1] "rs7561140"

$rs3731761
[1] "rs3731761"

$rs3731760
[1] "rs3731760"

$`2:166012356_G/T`
[1] "2:166012356_G/T"

$rs13011371
[1] "rs13011371"

$rs13010846
[1] "rs13010846"

$rs6710092
[1] "rs6710092"

$rs56124029
[1] "rs56124029"

$rs59172781
[1] "rs59172781"

$rs6756990
[1] "rs6756990"

$rs72264475
[1] "rs72264475"

$rs11887309
[1] "rs11887309"

$rs11887343
[1] "rs11887343"

$rs9333576
[1] "rs9333576"

$rs2304710
[1] "rs2304710"

$rs7556825
[1] "rs7556825"

$rs7579000
[1] "rs7579000"

$rs116682517
[1] "rs116682517"

$rs11903711
[1] "rs11903711"

$rs11903799
[1] "rs11903799"

$rs4667794
[1] "rs4667794"

$rs4667795
[1] "rs4667795"

$rs4667796
[1] "rs4667796"

$rs11891394
[1] "rs11891394"

$rs3754962
[1] "rs3754962"

$rs3754961
[1] "rs3754961"

$rs3754960
[1] "rs3754960"

$rs16822785
[1] "rs16822785"

$rs34971284
[1] "rs34971284"

$rs17819028
[1] "rs17819028"

$rs10930221
[1] "rs10930221"

$`2:167273403_T/C`
[1] "2:167273403_T/C"

$rs7561953
[1] "rs7561953"

$rs72623153
[1] "rs72623153"

$rs72623155
[1] "rs72623155"

$rs6738031
[1] "rs6738031"

$rs4550683
[1] "rs4550683"

$rs3791253
[1] "rs3791253"

$rs113442768
[1] "rs113442768"

$rs138641247
[1] "rs138641247"

$rs71428927
[1] "rs71428927"

$rs56951383
[1] "rs56951383"

$rs7593207
[1] "rs7593207"

$rs13417885
[1] "rs13417885"

$rs2293567
[1] "rs2293567"

$rs150350335
[1] "rs150350335"

$rs7565062
[1] "rs7565062"

$rs56164822
[1] "rs56164822"

$rs10169523
[1] "rs10169523"

$rs10180721
[1] "rs10180721"

$`2:167060900_C/T`
[1] "2:167060900_C/T"

$rs77944059
[1] "rs77944059"

$rs34141377
[1] "rs34141377"

$rs7595255
[1] "rs7595255"

$rs3793936
[1] "rs3793936"

$rs77684130
[1] "rs77684130"

$`2:167085081_T/G`
[1] "2:167085081_T/G"

$rs10199799
[1] "rs10199799"

$rs58519838
[1] "rs58519838"

$`2:167085142_T/A`
[1] "2:167085142_T/A"

$rs61701909
[1] "rs61701909"

$rs722204
[1] "rs722204"

$rs6746030
[1] "rs6746030"

$rs71428907
[1] "rs71428907"

$rs35664204
[1] "rs35664204"

$rs10930215
[1] "rs10930215"

$rs6432893
[1] "rs6432893"

$rs13007020
[1] "rs13007020"

$rs71395215
[1] "rs71395215"

$rs7607896
[1] "rs7607896"

$rs7608201
[1] "rs7608201"

$rs9917231
[1] "rs9917231"

$rs4278943
[1] "rs4278943"

$rs9917396
[1] "rs9917396"

$rs113914877
[1] "rs113914877"

$rs9917401
[1] "rs9917401"

$rs9917250
[1] "rs9917250"

$rs10584581
[1] "rs10584581"

$rs7596100
[1] "rs7596100"

$rs6747673
[1] "rs6747673"

$rs7588632
[1] "rs7588632"

$rs12994338


There are **336** total variants in the child's VCF file.

##### locateVariants method

`locateVariants()` captures variants with genes' intron, cds, splice site, intergenic, promoter, and utr regions.

In [98]:
all_variants_c <- locateVariants( query   = vcf_c, 
                       subject = txdbgene, 
                       region  = AllVariants() )

Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 330 out-of-bound ranges located on sequences
  9542, 12208, 12209, 12210, 12211, 12225, 12226, 12227, 10254, 10255,
  15589, 15590, 15591, 15592, 15593, 15594, 15595, 15596, 15597, 15598,
  15599, 15600, 15601, 15602, 15603, 15606, 15607, 15507, 46258, 46259,
  46260, 46256, 46257, and 64075. Note that ranges located on a sequence
  whose length is unknown (NA) or on a circular sequence are not
  considered out-of-bound (use seqlengths() and isCircular() to get the
  lengths and circularity flags of the underlying sequences). You can use
  trim() to trim these ranges. See ?`trim,GenomicRanges-method` for more
  information.”
Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 330 out-of-bound ranges located on sequences
  9542, 12208, 12209, 12210, 12211, 12225, 12226, 12227, 10254, 10255,
  15589, 15590, 15591, 15592, 15593, 15594, 15595, 15596,

In [99]:
class(all_variants_c)
head(all_variants_c)

[1] "GRanges"
attr(,"package")
[1] "GenomicRanges"

[1] "GRanges"
attr(,"package")
[1] "GenomicRanges"

GRanges object with 6 ranges and 9 metadata columns:
                  seqnames              ranges strand | LOCATION  LOCSTART
                     <Rle>           <IRanges>  <Rle> | <factor> <integer>
         rs568376        2           166870600      - |   intron     56066
         rs577306        2           166871905      - |   intron     54761
      rs140274879        2 166892444-166892445      - |   intron     34342
       rs72132828        2 166895913-166895914      - |   intron     31713
         rs490317        2           166896178      - |   intron     31623
  2:166897989_G/A        2           166897989      - |   intron     30051
                     LOCEND   QUERYID        TXID         CDSID      GENEID
                  <integer> <integer> <character> <IntegerList> <character>
         rs568376     56066         3       11728                    653489
         rs577306     54761         4       11728                    653489
      rs140274879     34343         5      

GRanges object with 6 ranges and 9 metadata columns:
                  seqnames              ranges strand | LOCATION  LOCSTART
                     <Rle>           <IRanges>  <Rle> | <factor> <integer>
         rs568376        2           166870600      - |   intron     56066
         rs577306        2           166871905      - |   intron     54761
      rs140274879        2 166892444-166892445      - |   intron     34342
       rs72132828        2 166895913-166895914      - |   intron     31713
         rs490317        2           166896178      - |   intron     31623
  2:166897989_G/A        2           166897989      - |   intron     30051
                     LOCEND   QUERYID        TXID         CDSID      GENEID
                  <integer> <integer> <character> <IntegerList> <character>
         rs568376     56066         3       11728                    653489
         rs577306     54761         4       11728                    653489
      rs140274879     34343         5      

In [100]:
unique(mcols(all_variants_c)$GENEID)

[1] "653489" "6323"   "84620"  "2688"   "55041"  "974"    "6326"   "445582"
 [9] "6328"   "2274"   "6332"   "10913"  "6335"   "6329"   "3384"   "729121"
[17] NA       "659"    "150864" "29926"  "4311"   "6331"   "8833"   "7881"  
[25] "25976"  "6336"   "11280"  "389170" "55768"  "23350"  "9435"   "6334"  
[33] "1181"   "7066"   "4598"   "8086"   "79058"

[1] "653489" "6323"   "84620"  "2688"   "55041"  "974"    "6326"   "445582"
 [9] "6328"   "2274"   "6332"   "10913"  "6335"   "6329"   "3384"   "729121"
[17] NA       "659"    "150864" "29926"  "4311"   "6331"   "8833"   "7881"  
[25] "25976"  "6336"   "11280"  "389170" "55768"  "23350"  "9435"   "6334"  
[33] "1181"   "7066"   "4598"   "8086"   "79058"

In [101]:
table(mcols(all_variants_c)$LOCATION)


spliceSite     intron    fiveUTR   threeUTR     coding intergenic   promoter 
         0        601          2          4        138          0         53 


spliceSite     intron    fiveUTR   threeUTR     coding intergenic   promoter 
         0        601          2          4        138          0         53 

In [102]:
names(all_variants_c)
length(names(all_variants_c))

[1] "rs568376"                         "rs577306"                        
  [3] "rs140274879"                      "rs72132828"                      
  [5] "rs490317"                         "2:166897989_G/A"                 
  [7] "2:166900130_T/G"                  "rs115015575"                     
  [9] "rs538921"                         "rs78391141"                      
 [11] "rs78391141"                       "rs78391141"                      
 [13] "rs34747425"                       "rs34747425"                      
 [15] "2:166929347_C/T"                  "2:166929347_C/T"                 
 [17] "2:166929347_C/T"                  "rs566839"                        
 [19] "rs566839"                         "rs566839"                        
 [21] "rs353133"                         "rs353133"                        
 [23] "rs353133"                         "rs353134"                        
 [25] "rs353134"                         "rs353134"                        
 [27] "rs353135"                         "rs353135"                        
 [29] "rs353135"                         "rs353126"                        
 [31] "rs353126"                         "rs353126"                        
 [33] "rs353126"                         "rs353126"                        
 [35] "rs353126"                         "rs2075704"                       
 [37] "rs2075704"                        "rs2075704"                       
 [39] "rs2075704"                        "rs2075704"                       
 [41] "rs2075704"                        "rs7593568"                       
 [43] "rs7593568"                        "rs7593568"                       
 [45] "rs7593568"                        "rs7593568"                       
 [47] "rs7593568"                        "rs7593568"                       
 [49] "rs112475855"                      "rs112475855"                     
 [51] "rs112475855"                      "rs112475855"                     
 [53] "rs112475855"                      "rs112475855"                     
 [55] "rs112475855"                      "rs1965757"                       
 [57] "rs1965757"                        "rs1965757"                       
 [59] "rs1965757"                        "rs1965757"                       
 [61] "rs1965757"                        "rs2116658"                       
 [63] "rs2116658"                        "rs2116658"                       
 [65] "rs2116658"                        "rs7578237"                       
 [67] "rs7578237"                        "rs10178662"                      
 [69] "rs10178662"                       "rs2060198"                       
 [71] "rs2060198"                        "rs2060198"                       
 [73] "rs2060198"                        "rs11885001"                      
 [75] "rs11885001"                       "rs11885001"                      
 [77] "rs11885001"                       "rs1946892"                       
 [79] "rs1946892"                        "rs1946892"                       
 [81] "rs1946892"                        "rs1946892"                       
 [83] "rs16850131"                       "rs16850131"                      
 [85] "rs16850131"                       "rs16850131"                      
 [87] "rs16850131"                       "2:166012356_G/T"                 
 [89] "2:166012356_G/T"                  "2:166012356_G/T"                 
 [91] "2:166012356_G/T"                  "2:166012356_G/T"                 
 [93] "rs13011371"                       "rs13010846"                      
 [95] "rs6710092"                        "rs56124029"                      
 [97] "rs59172781"                       "rs6756990"                       
 [99] "rs72264475"                       "rs11887309"                      
[101] "rs11887343"                       "rs9333576"                       
[103] "rs2304710"                        "rs7556825"                       
[105] "rs7556825"                        "rs757900

[1] 798

[1] "rs568376"                         "rs577306"                        
  [3] "rs140274879"                      "rs72132828"                      
  [5] "rs490317"                         "2:166897989_G/A"                 
  [7] "2:166900130_T/G"                  "rs115015575"                     
  [9] "rs538921"                         "rs78391141"                      
 [11] "rs78391141"                       "rs78391141"                      
 [13] "rs34747425"                       "rs34747425"                      
 [15] "2:166929347_C/T"                  "2:166929347_C/T"                 
 [17] "2:166929347_C/T"                  "rs566839"                        
 [19] "rs566839"                         "rs566839"                        
 [21] "rs353133"                         "rs353133"                        
 [23] "rs353133"                         "rs353134"                        
 [25] "rs353134"                         "rs353134"                        
 [27] "rs353135"                         "rs353135"                        
 [29] "rs353135"                         "rs353126"                        
 [31] "rs353126"                         "rs353126"                        
 [33] "rs353126"                         "rs353126"                        
 [35] "rs353126"                         "rs2075704"                       
 [37] "rs2075704"                        "rs2075704"                       
 [39] "rs2075704"                        "rs2075704"                       
 [41] "rs2075704"                        "rs7593568"                       
 [43] "rs7593568"                        "rs7593568"                       
 [45] "rs7593568"                        "rs7593568"                       
 [47] "rs7593568"                        "rs7593568"                       
 [49] "rs112475855"                      "rs112475855"                     
 [51] "rs112475855"                      "rs112475855"                     
 [53] "rs112475855"                      "rs112475855"                     
 [55] "rs112475855"                      "rs1965757"                       
 [57] "rs1965757"                        "rs1965757"                       
 [59] "rs1965757"                        "rs1965757"                       
 [61] "rs1965757"                        "rs2116658"                       
 [63] "rs2116658"                        "rs2116658"                       
 [65] "rs2116658"                        "rs7578237"                       
 [67] "rs7578237"                        "rs10178662"                      
 [69] "rs10178662"                       "rs2060198"                       
 [71] "rs2060198"                        "rs2060198"                       
 [73] "rs2060198"                        "rs11885001"                      
 [75] "rs11885001"                       "rs11885001"                      
 [77] "rs11885001"                       "rs1946892"                       
 [79] "rs1946892"                        "rs1946892"                       
 [81] "rs1946892"                        "rs1946892"                       
 [83] "rs16850131"                       "rs16850131"                      
 [85] "rs16850131"                       "rs16850131"                      
 [87] "rs16850131"                       "2:166012356_G/T"                 
 [89] "2:166012356_G/T"                  "2:166012356_G/T"                 
 [91] "2:166012356_G/T"                  "2:166012356_G/T"                 
 [93] "rs13011371"                       "rs13010846"                      
 [95] "rs6710092"                        "rs56124029"                      
 [97] "rs59172781"                       "rs6756990"                       
 [99] "rs72264475"                       "rs11887309"                      
[101] "rs11887343"                       "rs9333576"                       
[103] "rs2304710"                        "rs7556825"                       
[105] "rs7556825"                        "rs757900

[1] 798

In [103]:
# There are repeats in this dataset. Count how many unique variants there are.
length(unique(names(all_variants_c)))

[1] 233

[1] 233

In [104]:
# Convert the list of unique variants
all_var_c = as.list(unique(names(all_variants_c)))
all_var_c

[[1]]
[1] "rs568376"

[[2]]
[1] "rs577306"

[[3]]
[1] "rs140274879"

[[4]]
[1] "rs72132828"

[[5]]
[1] "rs490317"

[[6]]
[1] "2:166897989_G/A"

[[7]]
[1] "2:166900130_T/G"

[[8]]
[1] "rs115015575"

[[9]]
[1] "rs538921"

[[10]]
[1] "rs78391141"

[[11]]
[1] "rs34747425"

[[12]]
[1] "2:166929347_C/T"

[[13]]
[1] "rs566839"

[[14]]
[1] "rs353133"

[[15]]
[1] "rs353134"

[[16]]
[1] "rs353135"

[[17]]
[1] "rs353126"

[[18]]
[1] "rs2075704"

[[19]]
[1] "rs7593568"

[[20]]
[1] "rs112475855"

[[21]]
[1] "rs1965757"

[[22]]
[1] "rs2116658"

[[23]]
[1] "rs7578237"

[[24]]
[1] "rs10178662"

[[25]]
[1] "rs2060198"

[[26]]
[1] "rs11885001"

[[27]]
[1] "rs1946892"

[[28]]
[1] "rs16850131"

[[29]]
[1] "2:166012356_G/T"

[[30]]
[1] "rs13011371"

[[31]]
[1] "rs13010846"

[[32]]
[1] "rs6710092"

[[33]]
[1] "rs56124029"

[[34]]
[1] "rs59172781"

[[35]]
[1] "rs6756990"

[[36]]
[1] "rs72264475"

[[37]]
[1] "rs11887309"

[[38]]
[1] "rs11887343"

[[39]]
[1] "rs9333576"

[[40]]
[1] "rs2304710"

[[41]]
[1] "rs7556825"

[[42]]
[1] "rs7579000"

[[43]]
[1] "rs116682517"

[[44]]
[1] "rs11903711"

[[45]]
[1] "rs11903799"

[[46]]
[1] "rs4667794"

[[47]]
[1] "rs4667795"

[[48]]
[1] "rs4667796"

[[49]]
[1] "rs11891394"

[[50]]
[1] "rs3754962"

[[51]]
[1] "rs3754961"

[[52]]
[1] "rs3754960"

[[53]]
[1] "rs16822785"

[[54]]
[1] "rs34971284"

[[55]]
[1] "2:167273403_T/C"

[[56]]
[1] "rs6738031"

[[57]]
[1] "rs113442768"

[[58]]
[1] "rs138641247"

[[59]]
[1] "rs71428927"

[[60]]
[1] "rs56951383"

[[61]]
[1] "rs7593207"

[[62]]
[1] "rs13417885"

[[63]]
[1] "rs2293567"

[[64]]
[1] "rs150350335"

[[65]]
[1] "rs7565062"

[[66]]
[1] "rs56164822"

[[67]]
[1] "rs10169523"

[[68]]
[1] "rs10180721"

[[69]]
[1] "2:167060900_C/T"

[[70]]
[1] "rs77944059"

[[71]]
[1] "rs34141377"

[[72]]
[1] "rs7595255"

[[73]]
[1] "rs3793936"

[[74]]
[1] "rs77684130"

[[75]]
[1] "2:167085081_T/G"

[[76]]
[1] "rs10199799"

[[77]]
[1] "rs58519838"

[[78]]
[1] "2:167085142_T/A"

[[79]]
[1] "rs61701909"

[[80]]
[1] "rs722204"

[[81]]
[1] "rs6746030"

[[82]]
[1] "rs6747673"

[[83]]
[1] "rs4429487"

[[84]]
[1] "rs9646771"

[[85]]
[1] "rs9646772"

[[86]]
[1] "rs58349687"

[[87]]
[1] "rs4443015"

[[88]]
[1] "rs4447616"

[[89]]
[1] "2:167163736_T/TC"

[[90]]
[1] "2:167167886_A/T"

[[91]]
[1] "rs6432899"

[[92]]
[1] "rs6432900"

[[93]]
[1] "rs6432901"

[[94]]
[1] "rs4131162"

[[95]]
[1] "rs4131161"

[[96]]
[1] "rs17766561"

[[97]]
[1] "rs2385539"

[[98]]
[1] "2:220370245_C/G"

[[99]]
[1] "rs1046474"

[[100]]
[1] "rs7427447"

[[101]]
[1] "rs6599213"

[[102]]
[1] "rs41312403"

[[103]]
[1] "rs190482467"

[[104]]
[1] "rs7430407"

[[105]]
[1] "rs41260344"

[[106]]
[1] "rs7432766"

[[107]]
[1] "rs7645173"

[[108]]
[1] "rs7432532"

[[109]]
[1] "rs1805124"

[[110]]
[1] "rs34535972"

[[111]]
[1] "rs13084981"

[[112]]
[1] "rs6770569"

[[113]]
[1] "rs6599222"

[[114]]
[1] "rs9873213"

[[115]]
[1] "rs41312441"

[[116]]
[1] "rs9851962"

[[117]]
[1] "rs6781731"

[[118]]
[1] "rs7431305"

[[119]]
[1] "rs41312963"

[[120]]
[1] "rs41261744"

[[121]]
[1] "rs6599230"

[[122]]
[1] "rs9311195"

[[123]]
[1] "rs6599241"

[[124]]
[1] "rs6599242"

[[125]]
[1] "rs6790627"

[[126]]
[1] "rs6771157"

[[127]]
[1] "rs12632942"

[[128]]
[1] "rs6795970"

[[129]]
[1] "rs59468016"

[[130]]
[1] "rs57326399"

[[131]]
[1] "rs7630989"

[[132]]
[1] "rs7617919"

[[133]]
[1] "rs62244070"

[[134]]
[1] "rs4234134"

[[135]]
[1] "rs4234133"

[[136]]
[1] "rs62244134"

[[137]]
[1] "rs35547754"

[[138]]
[1] "rs6599266"

[[139]]
[1] "rs13093915"

[[140]]
[1] "rs13099628"

[[141]]
[1] "rs33985936"

[[142]]
[1] "rs5848481"

[[143]]
[1] "rs4484137"

[[144]]
[1] "rs4359754"

[[145]]
[1] "rs4289298"

[[146]]
[1] "rs4073113"

[[147]]
[1] "rs4553926"

[[148]]
[1] "rs4514993"

[[149]]
[1] "rs13076816"

[[150]]
[1] "rs141158768"

[[151]]
[1] "3:38948839_ATC/A"

[[152]]
[1] "3:38948848_TACACACAC/TACACACACAC"

[[153]]
[1] "rs76050581"

[[154]]
[1] "rs6599267"

[[155]]
[1] "rs6599268"

[[156]]
[1] "rs7429404"

[[157]]
[1] "rs6779297"

[[158]]
[1] "rs6599269"

[[1

[[1]]
[1] "rs568376"

[[2]]
[1] "rs577306"

[[3]]
[1] "rs140274879"

[[4]]
[1] "rs72132828"

[[5]]
[1] "rs490317"

[[6]]
[1] "2:166897989_G/A"

[[7]]
[1] "2:166900130_T/G"

[[8]]
[1] "rs115015575"

[[9]]
[1] "rs538921"

[[10]]
[1] "rs78391141"

[[11]]
[1] "rs34747425"

[[12]]
[1] "2:166929347_C/T"

[[13]]
[1] "rs566839"

[[14]]
[1] "rs353133"

[[15]]
[1] "rs353134"

[[16]]
[1] "rs353135"

[[17]]
[1] "rs353126"

[[18]]
[1] "rs2075704"

[[19]]
[1] "rs7593568"

[[20]]
[1] "rs112475855"

[[21]]
[1] "rs1965757"

[[22]]
[1] "rs2116658"

[[23]]
[1] "rs7578237"

[[24]]
[1] "rs10178662"

[[25]]
[1] "rs2060198"

[[26]]
[1] "rs11885001"

[[27]]
[1] "rs1946892"

[[28]]
[1] "rs16850131"

[[29]]
[1] "2:166012356_G/T"

[[30]]
[1] "rs13011371"

[[31]]
[1] "rs13010846"

[[32]]
[1] "rs6710092"

[[33]]
[1] "rs56124029"

[[34]]
[1] "rs59172781"

[[35]]
[1] "rs6756990"

[[36]]
[1] "rs72264475"

[[37]]
[1] "rs11887309"

[[38]]
[1] "rs11887343"

[[39]]
[1] "rs9333576"

[[40]]
[1] "rs2304710"

[[41]]
[1] "rs7556825"

[[42]]
[1] "rs7579000"

[[43]]
[1] "rs116682517"

[[44]]
[1] "rs11903711"

[[45]]
[1] "rs11903799"

[[46]]
[1] "rs4667794"

[[47]]
[1] "rs4667795"

[[48]]
[1] "rs4667796"

[[49]]
[1] "rs11891394"

[[50]]
[1] "rs3754962"

[[51]]
[1] "rs3754961"

[[52]]
[1] "rs3754960"

[[53]]
[1] "rs16822785"

[[54]]
[1] "rs34971284"

[[55]]
[1] "2:167273403_T/C"

[[56]]
[1] "rs6738031"

[[57]]
[1] "rs113442768"

[[58]]
[1] "rs138641247"

[[59]]
[1] "rs71428927"

[[60]]
[1] "rs56951383"

[[61]]
[1] "rs7593207"

[[62]]
[1] "rs13417885"

[[63]]
[1] "rs2293567"

[[64]]
[1] "rs150350335"

[[65]]
[1] "rs7565062"

[[66]]
[1] "rs56164822"

[[67]]
[1] "rs10169523"

[[68]]
[1] "rs10180721"

[[69]]
[1] "2:167060900_C/T"

[[70]]
[1] "rs77944059"

[[71]]
[1] "rs34141377"

[[72]]
[1] "rs7595255"

[[73]]
[1] "rs3793936"

[[74]]
[1] "rs77684130"

[[75]]
[1] "2:167085081_T/G"

[[76]]
[1] "rs10199799"

[[77]]
[1] "rs58519838"

[[78]]
[1] "2:167085142_T/A"

[[79]]
[1] "rs61701909"

[[80]]
[1] "rs722204"

[[81]]
[1] "rs6746030"

[[82]]
[1] "rs6747673"

[[83]]
[1] "rs4429487"

[[84]]
[1] "rs9646771"

[[85]]
[1] "rs9646772"

[[86]]
[1] "rs58349687"

[[87]]
[1] "rs4443015"

[[88]]
[1] "rs4447616"

[[89]]
[1] "2:167163736_T/TC"

[[90]]
[1] "2:167167886_A/T"

[[91]]
[1] "rs6432899"

[[92]]
[1] "rs6432900"

[[93]]
[1] "rs6432901"

[[94]]
[1] "rs4131162"

[[95]]
[1] "rs4131161"

[[96]]
[1] "rs17766561"

[[97]]
[1] "rs2385539"

[[98]]
[1] "2:220370245_C/G"

[[99]]
[1] "rs1046474"

[[100]]
[1] "rs7427447"

[[101]]
[1] "rs6599213"

[[102]]
[1] "rs41312403"

[[103]]
[1] "rs190482467"

[[104]]
[1] "rs7430407"

[[105]]
[1] "rs41260344"

[[106]]
[1] "rs7432766"

[[107]]
[1] "rs7645173"

[[108]]
[1] "rs7432532"

[[109]]
[1] "rs1805124"

[[110]]
[1] "rs34535972"

[[111]]
[1] "rs13084981"

[[112]]
[1] "rs6770569"

[[113]]
[1] "rs6599222"

[[114]]
[1] "rs9873213"

[[115]]
[1] "rs41312441"

[[116]]
[1] "rs9851962"

[[117]]
[1] "rs6781731"

[[118]]
[1] "rs7431305"

[[119]]
[1] "rs41312963"

[[120]]
[1] "rs41261744"

[[121]]
[1] "rs6599230"

[[122]]
[1] "rs9311195"

[[123]]
[1] "rs6599241"

[[124]]
[1] "rs6599242"

[[125]]
[1] "rs6790627"

[[126]]
[1] "rs6771157"

[[127]]
[1] "rs12632942"

[[128]]
[1] "rs6795970"

[[129]]
[1] "rs59468016"

[[130]]
[1] "rs57326399"

[[131]]
[1] "rs7630989"

[[132]]
[1] "rs7617919"

[[133]]
[1] "rs62244070"

[[134]]
[1] "rs4234134"

[[135]]
[1] "rs4234133"

[[136]]
[1] "rs62244134"

[[137]]
[1] "rs35547754"

[[138]]
[1] "rs6599266"

[[139]]
[1] "rs13093915"

[[140]]
[1] "rs13099628"

[[141]]
[1] "rs33985936"

[[142]]
[1] "rs5848481"

[[143]]
[1] "rs4484137"

[[144]]
[1] "rs4359754"

[[145]]
[1] "rs4289298"

[[146]]
[1] "rs4073113"

[[147]]
[1] "rs4553926"

[[148]]
[1] "rs4514993"

[[149]]
[1] "rs13076816"

[[150]]
[1] "rs141158768"

[[151]]
[1] "3:38948839_ATC/A"

[[152]]
[1] "3:38948848_TACACACAC/TACACACACAC"

[[153]]
[1] "rs76050581"

[[154]]
[1] "rs6599267"

[[155]]
[1] "rs6599268"

[[156]]
[1] "rs7429404"

[[157]]
[1] "rs6779297"

[[158]]
[1] "rs6599269"

[[1

In order to compare the variants between the child, mother and father, we also perform the same variant identification with the mother and father. 

#### 2.1 Mother's Variants

In [105]:
row.names(vcf_m)
length(row.names(vcf_m))

[1] "rs34871628"         "rs568376"           "rs577306"          
  [4] "rs116647099"        "rs72132828"         "rs490317"          
  [7] "2:166900130_T/G"    "rs538921"           "rs78391141"        
 [10] "2:166929347_C/T"    "rs566839"           "rs353133"          
 [13] "rs353134"           "rs353135"           "rs353126"          
 [16] "rs3769951"          "rs150568699"        "rs6432821"         
 [19] "rs10178662"         "rs2060198"          "rs11885001"        
 [22] "rs73969176"         "rs77002577"         "rs1439993"         
 [25] "rs73019878"         "rs11393050"         "rs62174900"        
 [28] "rs78280506"         "rs1946892"          "rs2289400"         
 [31] "rs10048748"         "rs2289401"          "rs1982212"         
 [34] "rs2023225"          "rs16850131"         "rs58539679"        
 [37] "rs16850134"         "rs7574983"          "rs7561140"         
 [40] "rs3731761"          "rs3731760"          "rs13011371"        
 [43] "rs13010846"         "rs12464762"         "rs6710092"         
 [46] "rs56124029"         "rs59172781"         "rs6756990"         
 [49] "rs72264475"         "rs11887309"         "rs11887343"        
 [52] "rs9333576"          "rs2304710"          "rs7556825"         
 [55] "rs7579000"          "rs11903711"         "rs11903799"        
 [58] "rs4667795"          "rs4667796"          "rs11891394"        
 [61] "rs3754962"          "rs3754961"          "rs3754960"         
 [64] "rs16822785"         "rs34971284"         "2:167273403_T/C"   
 [67] "rs6738031"          "rs4550683"          "rs138641247"       
 [70] "rs7593207"          "rs13417885"         "rs2293567"         
 [73] "rs7565062"          "rs56164822"         "rs10169523"        
 [76] "rs10180721"         "2:167060900_C/T"    "rs77944059"        
 [79] "rs7595255"          "rs3793936"          "rs77684130"        
 [82] "2:167085081_T/G"    "rs10199799"         "rs58519838"        
 [85] "rs61701909"         "rs722204"           "rs6746030"         
 [88] "rs10803807"         "rs13395140"         "rs71428907"        
 [91] "rs35664204"         "rs10930215"         "rs3924001"         
 [94] "rs6432893"          "rs13007020"         "rs71395215"        
 [97] "rs7607896"          "rs7608201"          "rs9917231"         
[100] "rs4278943"          "rs4284843"          "rs9917396"         
[103] "rs113914877"        "rs9917401"          "rs9917250"         
[106] "rs10584581"         "rs7596100"          "rs6747673"         
[109] "rs13402180"         "rs13414203"         "rs13402540"        
[112] "rs7588632"          "2:167149825_G/T"    "rs13017637"        
[115] "rs12994338"         "rs4429487"          "rs9646771"         
[118] "rs9646772"          "rs58349687"         "rs4443015"         
[121] "rs4447616"          "2:167163736_T/TC"   "2:167167886_A/T"   
[124] "rs6432899"          "rs6432900"          "rs6432901"         
[127] "rs4131162"          "rs4131161"          "rs2385539"         
[130] "rs75911518"         "rs9646863"          "rs13428617"        
[133] "rs10932808"         "rs6599213"          "rs41312403"        
[136] "rs7430407"          "rs41260344"         "rs141932685"       
[139] "rs7645173"          "rs11711097"         "rs7432532"         
[142] "rs1805124"          "rs55899887"         "rs6770569"         
[145] "rs6599222"          "rs9873213"          "rs9851962"         
[148] "rs6781731"          "rs7431305"          "rs41312963"        
[151] "rs41261744"         "rs6599230"          "rs6599241"         
[154] "rs6599242"          "rs6599248"          "rs6781740"         
[157] "rs62242430"         "rs6805187"          "rs6771157"         
[160] "rs61487238"         "rs4076737"          "rs12632942"        
[163] "rs62242438"         "rs6795970"          "rs59858965"        
[166] "rs59468016"         "rs57326399"         "rs34786326"        
[169] "rs7433306"          "rs7428167"          "rs6599250"         
[172] "rs7617919"          "rs188541589"        "rs62244070"        


[1] 285

[1] "rs34871628"         "rs568376"           "rs577306"          
  [4] "rs116647099"        "rs72132828"         "rs490317"          
  [7] "2:166900130_T/G"    "rs538921"           "rs78391141"        
 [10] "2:166929347_C/T"    "rs566839"           "rs353133"          
 [13] "rs353134"           "rs353135"           "rs353126"          
 [16] "rs3769951"          "rs150568699"        "rs6432821"         
 [19] "rs10178662"         "rs2060198"          "rs11885001"        
 [22] "rs73969176"         "rs77002577"         "rs1439993"         
 [25] "rs73019878"         "rs11393050"         "rs62174900"        
 [28] "rs78280506"         "rs1946892"          "rs2289400"         
 [31] "rs10048748"         "rs2289401"          "rs1982212"         
 [34] "rs2023225"          "rs16850131"         "rs58539679"        
 [37] "rs16850134"         "rs7574983"          "rs7561140"         
 [40] "rs3731761"          "rs3731760"          "rs13011371"        
 [43] "rs13010846"         "rs12464762"         "rs6710092"         
 [46] "rs56124029"         "rs59172781"         "rs6756990"         
 [49] "rs72264475"         "rs11887309"         "rs11887343"        
 [52] "rs9333576"          "rs2304710"          "rs7556825"         
 [55] "rs7579000"          "rs11903711"         "rs11903799"        
 [58] "rs4667795"          "rs4667796"          "rs11891394"        
 [61] "rs3754962"          "rs3754961"          "rs3754960"         
 [64] "rs16822785"         "rs34971284"         "2:167273403_T/C"   
 [67] "rs6738031"          "rs4550683"          "rs138641247"       
 [70] "rs7593207"          "rs13417885"         "rs2293567"         
 [73] "rs7565062"          "rs56164822"         "rs10169523"        
 [76] "rs10180721"         "2:167060900_C/T"    "rs77944059"        
 [79] "rs7595255"          "rs3793936"          "rs77684130"        
 [82] "2:167085081_T/G"    "rs10199799"         "rs58519838"        
 [85] "rs61701909"         "rs722204"           "rs6746030"         
 [88] "rs10803807"         "rs13395140"         "rs71428907"        
 [91] "rs35664204"         "rs10930215"         "rs3924001"         
 [94] "rs6432893"          "rs13007020"         "rs71395215"        
 [97] "rs7607896"          "rs7608201"          "rs9917231"         
[100] "rs4278943"          "rs4284843"          "rs9917396"         
[103] "rs113914877"        "rs9917401"          "rs9917250"         
[106] "rs10584581"         "rs7596100"          "rs6747673"         
[109] "rs13402180"         "rs13414203"         "rs13402540"        
[112] "rs7588632"          "2:167149825_G/T"    "rs13017637"        
[115] "rs12994338"         "rs4429487"          "rs9646771"         
[118] "rs9646772"          "rs58349687"         "rs4443015"         
[121] "rs4447616"          "2:167163736_T/TC"   "2:167167886_A/T"   
[124] "rs6432899"          "rs6432900"          "rs6432901"         
[127] "rs4131162"          "rs4131161"          "rs2385539"         
[130] "rs75911518"         "rs9646863"          "rs13428617"        
[133] "rs10932808"         "rs6599213"          "rs41312403"        
[136] "rs7430407"          "rs41260344"         "rs141932685"       
[139] "rs7645173"          "rs11711097"         "rs7432532"         
[142] "rs1805124"          "rs55899887"         "rs6770569"         
[145] "rs6599222"          "rs9873213"          "rs9851962"         
[148] "rs6781731"          "rs7431305"          "rs41312963"        
[151] "rs41261744"         "rs6599230"          "rs6599241"         
[154] "rs6599242"          "rs6599248"          "rs6781740"         
[157] "rs62242430"         "rs6805187"          "rs6771157"         
[160] "rs61487238"         "rs4076737"          "rs12632942"        
[163] "rs62242438"         "rs6795970"          "rs59858965"        
[166] "rs59468016"         "rs57326399"         "rs34786326"        
[169] "rs7433306"          "rs7428167"          "rs6599250"         
[172] "rs7617919"          "rs188541589"        "rs62244070"        


[1] 285

In [106]:
# Extract the variant IDs into a list
vcf_m_l = lapply(vcf_m, rownames)
vcf_m_l

$rs34871628
[1] "rs34871628"

$rs568376
[1] "rs568376"

$rs577306
[1] "rs577306"

$rs116647099
[1] "rs116647099"

$rs72132828
[1] "rs72132828"

$rs490317
[1] "rs490317"

$`2:166900130_T/G`
[1] "2:166900130_T/G"

$rs538921
[1] "rs538921"

$rs78391141
[1] "rs78391141"

$`2:166929347_C/T`
[1] "2:166929347_C/T"

$rs566839
[1] "rs566839"

$rs353133
[1] "rs353133"

$rs353134
[1] "rs353134"

$rs353135
[1] "rs353135"

$rs353126
[1] "rs353126"

$rs3769951
[1] "rs3769951"

$rs150568699
[1] "rs150568699"

$rs6432821
[1] "rs6432821"

$rs10178662
[1] "rs10178662"

$rs2060198
[1] "rs2060198"

$rs11885001
[1] "rs11885001"

$rs73969176
[1] "rs73969176"

$rs77002577
[1] "rs77002577"

$rs1439993
[1] "rs1439993"

$rs73019878
[1] "rs73019878"

$rs11393050
[1] "rs11393050"

$rs62174900
[1] "rs62174900"

$rs78280506
[1] "rs78280506"

$rs1946892
[1] "rs1946892"

$rs2289400
[1] "rs2289400"

$rs10048748
[1] "rs10048748"

$rs2289401
[1] "rs2289401"

$rs1982212
[1] "rs1982212"

$rs2023225
[1] "rs2023225"

$rs16850131
[1] "rs16850131"

$rs58539679
[1] "rs58539679"

$rs16850134
[1] "rs16850134"

$rs7574983
[1] "rs7574983"

$rs7561140
[1] "rs7561140"

$rs3731761
[1] "rs3731761"

$rs3731760
[1] "rs3731760"

$rs13011371
[1] "rs13011371"

$rs13010846
[1] "rs13010846"

$rs12464762
[1] "rs12464762"

$rs6710092
[1] "rs6710092"

$rs56124029
[1] "rs56124029"

$rs59172781
[1] "rs59172781"

$rs6756990
[1] "rs6756990"

$rs72264475
[1] "rs72264475"

$rs11887309
[1] "rs11887309"

$rs11887343
[1] "rs11887343"

$rs9333576
[1] "rs9333576"

$rs2304710
[1] "rs2304710"

$rs7556825
[1] "rs7556825"

$rs7579000
[1] "rs7579000"

$rs11903711
[1] "rs11903711"

$rs11903799
[1] "rs11903799"

$rs4667795
[1] "rs4667795"

$rs4667796
[1] "rs4667796"

$rs11891394
[1] "rs11891394"

$rs3754962
[1] "rs3754962"

$rs3754961
[1] "rs3754961"

$rs3754960
[1] "rs3754960"

$rs16822785
[1] "rs16822785"

$rs34971284
[1] "rs34971284"

$`2:167273403_T/C`
[1] "2:167273403_T/C"

$rs6738031
[1] "rs6738031"

$rs4550683
[1] "rs4550683"

$rs138641247
[1] "rs138641247"

$rs7593207
[1] "rs7593207"

$rs13417885
[1] "rs13417885"

$rs2293567
[1] "rs2293567"

$rs7565062
[1] "rs7565062"

$rs56164822
[1] "rs56164822"

$rs10169523
[1] "rs10169523"

$rs10180721
[1] "rs10180721"

$`2:167060900_C/T`
[1] "2:167060900_C/T"

$rs77944059
[1] "rs77944059"

$rs7595255
[1] "rs7595255"

$rs3793936
[1] "rs3793936"

$rs77684130
[1] "rs77684130"

$`2:167085081_T/G`
[1] "2:167085081_T/G"

$rs10199799
[1] "rs10199799"

$rs58519838
[1] "rs58519838"

$rs61701909
[1] "rs61701909"

$rs722204
[1] "rs722204"

$rs6746030
[1] "rs6746030"

$rs10803807
[1] "rs10803807"

$rs13395140
[1] "rs13395140"

$rs71428907
[1] "rs71428907"

$rs35664204
[1] "rs35664204"

$rs10930215
[1] "rs10930215"

$rs3924001
[1] "rs3924001"

$rs6432893
[1] "rs6432893"

$rs13007020
[1] "rs13007020"

$rs71395215
[1] "rs71395215"

$rs7607896
[1] "rs7607896"

$rs7608201
[1] "rs7608201"

$rs9917231
[1] "rs9917231"

$rs4278943
[1] "rs4278943"

$rs4284843
[1] "rs4284843"

$rs9917396
[1] "rs9917396"

$rs113914877
[1] "rs113914877"

$rs9917401
[1] "rs9917401"

$rs9917250
[1] "rs9917250"

$rs10584581
[1] "rs10584581"

$rs7596100
[1] "rs7596100"

$rs6747673
[1] "rs6747673"

$rs13402180
[1] "rs13402180"

$rs13414203
[1] "rs13414203"

$rs13402540
[1] "rs13402540"

$rs7588632
[1] "rs7588632"

$`2:167149825_G/T`
[1] "2:167149825_G/T"

$rs13017637
[1] "rs13017637"

$rs12994338
[1] "rs12994338"

$rs4429487
[1] "rs4429487"

$rs9646771
[1] "rs9646771"

$rs9646772
[1] "rs9646772"

$rs58349687
[1] "rs58349687"

$rs4443015
[1] "rs4443015"

$rs4447616
[1] "rs4447616"

$`2:167163736_T/TC`
[1] "2:167163736_T/TC"

$`2:167167886_A/T`
[1] "2:167167886_A/T"

$rs6432899
[1] "rs6432899"

$rs6432900
[1] "rs6432900"

$rs6432901
[1] "rs6432901"

$rs4131162
[1] "rs4131162"

$rs4131161
[1] "rs4131161"

$rs2385539
[1] "rs2385539"

$rs75911518
[1] "rs75911518"

$rs9646863
[1] "rs9646863"

$rs13428617
[1] "rs13428617"

$rs10932808
[1] "rs10932808"

$rs6599213
[1] "rs6599213"

$rs41312403
[1] "rs41312403"



$rs34871628
[1] "rs34871628"

$rs568376
[1] "rs568376"

$rs577306
[1] "rs577306"

$rs116647099
[1] "rs116647099"

$rs72132828
[1] "rs72132828"

$rs490317
[1] "rs490317"

$`2:166900130_T/G`
[1] "2:166900130_T/G"

$rs538921
[1] "rs538921"

$rs78391141
[1] "rs78391141"

$`2:166929347_C/T`
[1] "2:166929347_C/T"

$rs566839
[1] "rs566839"

$rs353133
[1] "rs353133"

$rs353134
[1] "rs353134"

$rs353135
[1] "rs353135"

$rs353126
[1] "rs353126"

$rs3769951
[1] "rs3769951"

$rs150568699
[1] "rs150568699"

$rs6432821
[1] "rs6432821"

$rs10178662
[1] "rs10178662"

$rs2060198
[1] "rs2060198"

$rs11885001
[1] "rs11885001"

$rs73969176
[1] "rs73969176"

$rs77002577
[1] "rs77002577"

$rs1439993
[1] "rs1439993"

$rs73019878
[1] "rs73019878"

$rs11393050
[1] "rs11393050"

$rs62174900
[1] "rs62174900"

$rs78280506
[1] "rs78280506"

$rs1946892
[1] "rs1946892"

$rs2289400
[1] "rs2289400"

$rs10048748
[1] "rs10048748"

$rs2289401
[1] "rs2289401"

$rs1982212
[1] "rs1982212"

$rs2023225
[1] "rs2023225"

$rs16850131
[1] "rs16850131"

$rs58539679
[1] "rs58539679"

$rs16850134
[1] "rs16850134"

$rs7574983
[1] "rs7574983"

$rs7561140
[1] "rs7561140"

$rs3731761
[1] "rs3731761"

$rs3731760
[1] "rs3731760"

$rs13011371
[1] "rs13011371"

$rs13010846
[1] "rs13010846"

$rs12464762
[1] "rs12464762"

$rs6710092
[1] "rs6710092"

$rs56124029
[1] "rs56124029"

$rs59172781
[1] "rs59172781"

$rs6756990
[1] "rs6756990"

$rs72264475
[1] "rs72264475"

$rs11887309
[1] "rs11887309"

$rs11887343
[1] "rs11887343"

$rs9333576
[1] "rs9333576"

$rs2304710
[1] "rs2304710"

$rs7556825
[1] "rs7556825"

$rs7579000
[1] "rs7579000"

$rs11903711
[1] "rs11903711"

$rs11903799
[1] "rs11903799"

$rs4667795
[1] "rs4667795"

$rs4667796
[1] "rs4667796"

$rs11891394
[1] "rs11891394"

$rs3754962
[1] "rs3754962"

$rs3754961
[1] "rs3754961"

$rs3754960
[1] "rs3754960"

$rs16822785
[1] "rs16822785"

$rs34971284
[1] "rs34971284"

$`2:167273403_T/C`
[1] "2:167273403_T/C"

$rs6738031
[1] "rs6738031"

$rs4550683
[1] "rs4550683"

$rs138641247
[1] "rs138641247"

$rs7593207
[1] "rs7593207"

$rs13417885
[1] "rs13417885"

$rs2293567
[1] "rs2293567"

$rs7565062
[1] "rs7565062"

$rs56164822
[1] "rs56164822"

$rs10169523
[1] "rs10169523"

$rs10180721
[1] "rs10180721"

$`2:167060900_C/T`
[1] "2:167060900_C/T"

$rs77944059
[1] "rs77944059"

$rs7595255
[1] "rs7595255"

$rs3793936
[1] "rs3793936"

$rs77684130
[1] "rs77684130"

$`2:167085081_T/G`
[1] "2:167085081_T/G"

$rs10199799
[1] "rs10199799"

$rs58519838
[1] "rs58519838"

$rs61701909
[1] "rs61701909"

$rs722204
[1] "rs722204"

$rs6746030
[1] "rs6746030"

$rs10803807
[1] "rs10803807"

$rs13395140
[1] "rs13395140"

$rs71428907
[1] "rs71428907"

$rs35664204
[1] "rs35664204"

$rs10930215
[1] "rs10930215"

$rs3924001
[1] "rs3924001"

$rs6432893
[1] "rs6432893"

$rs13007020
[1] "rs13007020"

$rs71395215
[1] "rs71395215"

$rs7607896
[1] "rs7607896"

$rs7608201
[1] "rs7608201"

$rs9917231
[1] "rs9917231"

$rs4278943
[1] "rs4278943"

$rs4284843
[1] "rs4284843"

$rs9917396
[1] "rs9917396"

$rs113914877
[1] "rs113914877"

$rs9917401
[1] "rs9917401"

$rs9917250
[1] "rs9917250"

$rs10584581
[1] "rs10584581"

$rs7596100
[1] "rs7596100"

$rs6747673
[1] "rs6747673"

$rs13402180
[1] "rs13402180"

$rs13414203
[1] "rs13414203"

$rs13402540
[1] "rs13402540"

$rs7588632
[1] "rs7588632"

$`2:167149825_G/T`
[1] "2:167149825_G/T"

$rs13017637
[1] "rs13017637"

$rs12994338
[1] "rs12994338"

$rs4429487
[1] "rs4429487"

$rs9646771
[1] "rs9646771"

$rs9646772
[1] "rs9646772"

$rs58349687
[1] "rs58349687"

$rs4443015
[1] "rs4443015"

$rs4447616
[1] "rs4447616"

$`2:167163736_T/TC`
[1] "2:167163736_T/TC"

$`2:167167886_A/T`
[1] "2:167167886_A/T"

$rs6432899
[1] "rs6432899"

$rs6432900
[1] "rs6432900"

$rs6432901
[1] "rs6432901"

$rs4131162
[1] "rs4131162"

$rs4131161
[1] "rs4131161"

$rs2385539
[1] "rs2385539"

$rs75911518
[1] "rs75911518"

$rs9646863
[1] "rs9646863"

$rs13428617
[1] "rs13428617"

$rs10932808
[1] "rs10932808"

$rs6599213
[1] "rs6599213"

$rs41312403
[1] "rs41312403"



There are **285** total variants in the mother's VCF file.

##### locateVariants method

`locateVariants()` captures variants with genes' intron, cds, splice site, intergenic, promoter, and utr regions.

In [107]:
all_variants_m <- locateVariants( query   = vcf_m, 
                       subject = txdbgene, 
                       region  = AllVariants() )

Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 343 out-of-bound ranges located on sequences
  9542, 12208, 12209, 12210, 12211, 12212, 12213, 12225, 12226, 12227,
  10254, 10255, 15589, 15590, 15591, 15592, 15593, 15594, 15595, 15596,
  15597, 15598, 15599, 15600, 15601, 15602, 15603, 15606, 15607, 15507,
  46258, 46259, 46260, 46256, and 46257. Note that ranges located on a
  sequence whose length is unknown (NA) or on a circular sequence are not
  considered out-of-bound (use seqlengths() and isCircular() to get the
  lengths and circularity flags of the underlying sequences). You can use
  trim() to trim these ranges. See ?`trim,GenomicRanges-method` for more
  information.”
Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 343 out-of-bound ranges located on sequences
  9542, 12208, 12209, 12210, 12211, 12212, 12213, 12225, 12226, 12227,
  10254, 10255, 15589, 15590, 15591, 15592, 15593,

In [108]:
class(all_variants_m)
head(all_variants_m)

[1] "GRanges"
attr(,"package")
[1] "GenomicRanges"

[1] "GRanges"
attr(,"package")
[1] "GenomicRanges"

GRanges object with 6 ranges and 9 metadata columns:
                  seqnames              ranges strand | LOCATION  LOCSTART
                     <Rle>           <IRanges>  <Rle> | <factor> <integer>
         rs568376        2           166870600      - |   intron     56066
         rs577306        2           166871905      - |   intron     54761
      rs116647099        2           166894209      - |   intron     33061
       rs72132828        2 166895913-166895914      - |   intron     31713
         rs490317        2           166896178      - |   intron     31623
  2:166900130_T/G        2           166900130      - |   intron     28043
                     LOCEND   QUERYID        TXID         CDSID      GENEID
                  <integer> <integer> <character> <IntegerList> <character>
         rs568376     56066         2       11728                    653489
         rs577306     54761         3       11728                    653489
      rs116647099     33061         4      

GRanges object with 6 ranges and 9 metadata columns:
                  seqnames              ranges strand | LOCATION  LOCSTART
                     <Rle>           <IRanges>  <Rle> | <factor> <integer>
         rs568376        2           166870600      - |   intron     56066
         rs577306        2           166871905      - |   intron     54761
      rs116647099        2           166894209      - |   intron     33061
       rs72132828        2 166895913-166895914      - |   intron     31713
         rs490317        2           166896178      - |   intron     31623
  2:166900130_T/G        2           166900130      - |   intron     28043
                     LOCEND   QUERYID        TXID         CDSID      GENEID
                  <integer> <integer> <character> <IntegerList> <character>
         rs568376     56066         2       11728                    653489
         rs577306     54761         3       11728                    653489
      rs116647099     33061         4      

In [109]:
unique(mcols(all_variants_m)$GENEID)

[1] "653489" "6323"   "84620"  "2688"   "55041"  "974"    "6326"   "445582"
 [9] "6328"   "2274"   "6332"   "10913"  "6335"   "6329"   "3384"   "729121"
[17] NA       "659"    "150864" "29926"  "4311"   "6331"   "8833"   "7881"  
[25] "25976"  "6336"   "11280"  "389170" "55768"  "23350"  "9435"   "1181"  
[33] "6334"   "7066"   "4598"   "8086"   "79058"

[1] "653489" "6323"   "84620"  "2688"   "55041"  "974"    "6326"   "445582"
 [9] "6328"   "2274"   "6332"   "10913"  "6335"   "6329"   "3384"   "729121"
[17] NA       "659"    "150864" "29926"  "4311"   "6331"   "8833"   "7881"  
[25] "25976"  "6336"   "11280"  "389170" "55768"  "23350"  "9435"   "1181"  
[33] "6334"   "7066"   "4598"   "8086"   "79058"

In [110]:
table(mcols(all_variants_m)$LOCATION)


spliceSite     intron    fiveUTR   threeUTR     coding intergenic   promoter 
         0        508          1          4        132          0         51 


spliceSite     intron    fiveUTR   threeUTR     coding intergenic   promoter 
         0        508          1          4        132          0         51 

In [111]:
# There are repeats in this dataset. Count how many unique variants there are.
names(all_variants_m)
length(unique(names(all_variants_m)))

[1] "rs568376"           "rs577306"           "rs116647099"       
  [4] "rs72132828"         "rs490317"           "2:166900130_T/G"   
  [7] "rs538921"           "rs78391141"         "rs78391141"        
 [10] "rs78391141"         "2:166929347_C/T"    "2:166929347_C/T"   
 [13] "2:166929347_C/T"    "rs566839"           "rs566839"          
 [16] "rs566839"           "rs353133"           "rs353133"          
 [19] "rs353133"           "rs353134"           "rs353134"          
 [22] "rs353134"           "rs353135"           "rs353135"          
 [25] "rs353135"           "rs353126"           "rs353126"          
 [28] "rs353126"           "rs353126"           "rs353126"          
 [31] "rs353126"           "rs10178662"         "rs10178662"        
 [34] "rs2060198"          "rs2060198"          "rs2060198"         
 [37] "rs2060198"          "rs11885001"         "rs11885001"        
 [40] "rs11885001"         "rs11885001"         "rs73969176"        
 [43] "rs77002577"         "rs62174900"         "rs62174900"        
 [46] "rs62174900"         "rs62174900"         "rs62174900"        
 [49] "rs1946892"          "rs1946892"          "rs1946892"         
 [52] "rs1946892"          "rs1946892"          "rs16850131"        
 [55] "rs16850131"         "rs16850131"         "rs16850131"        
 [58] "rs16850131"         "rs13011371"         "rs13010846"        
 [61] "rs12464762"         "rs6710092"          "rs56124029"        
 [64] "rs59172781"         "rs6756990"          "rs72264475"        
 [67] "rs11887309"         "rs11887343"         "rs9333576"         
 [70] "rs2304710"          "rs7556825"          "rs7556825"         
 [73] "rs7579000"          "rs7579000"          "rs11903711"        
 [76] "rs11903711"         "rs11903711"         "rs11903711"        
 [79] "rs11903799"         "rs11903799"         "rs11903799"        
 [82] "rs11903799"         "rs4667795"          "rs4667795"         
 [85] "rs4667795"          "rs4667795"          "rs4667795"         
 [88] "rs4667795"          "rs4667796"          "rs4667796"         
 [91] "rs4667796"          "rs4667796"          "rs4667796"         
 [94] "rs11891394"         "rs11891394"         "rs11891394"        
 [97] "rs11891394"         "rs11891394"         "rs3754962"         
[100] "rs3754962"          "rs3754962"          "rs3754962"         
[103] "rs3754962"          "rs3754961"          "rs3754961"         
[106] "rs3754961"          "rs3754961"          "rs3754961"         
[109] "rs3754960"          "rs3754960"          "rs3754960"         
[112] "rs3754960"          "rs3754960"          "rs16822785"        
[115] "rs16822785"         "rs16822785"         "rs34971284"        
[118] "2:167273403_T/C"    "rs6738031"          "rs138641247"       
[121] "rs138641247"        "rs7593207"          "rs7593207"         
[124] "rs13417885"         "rs13417885"         "rs13417885"        
[127] "rs2293567"          "rs2293567"          "rs2293567"         
[130] "rs7565062"          "rs7565062"          "rs56164822"        
[133] "rs56164822"         "rs56164822"         "rs56164822"        
[136] "rs10169523"         "rs10169523"         "rs10180721"        
[139] "rs10180721"         "2:167060900_C/T"    "2:167060900_C/T"   
[142] "2:167060900_C/T"    "rs77944059"         "rs77944059"        
[145] "rs7595255"          "rs7595255"          "rs3793936"         
[148] "rs3793936"          "rs77684130"         "rs77684130"        
[151] "2:167085081_T/G"    "2:167085081_T/G"    "rs10199799"        
[154] "rs10199799"         "rs58519838"         "rs58519838"        
[157] "rs61701909"         "rs61701909"         "rs722204"          
[160] "rs722204"           "rs6746030"          "rs6746030"         
[163] "rs6746030"          "rs6747673"          "rs6747673"         
[166] "rs6747673"          "rs6747673"          "rs13402180"        
[169] "rs13402180"         "rs13402180"         "rs13402180"        
[172] "rs13414203"         "rs13414203"         "rs13414203"        


[1] 212

[1] "rs568376"           "rs577306"           "rs116647099"       
  [4] "rs72132828"         "rs490317"           "2:166900130_T/G"   
  [7] "rs538921"           "rs78391141"         "rs78391141"        
 [10] "rs78391141"         "2:166929347_C/T"    "2:166929347_C/T"   
 [13] "2:166929347_C/T"    "rs566839"           "rs566839"          
 [16] "rs566839"           "rs353133"           "rs353133"          
 [19] "rs353133"           "rs353134"           "rs353134"          
 [22] "rs353134"           "rs353135"           "rs353135"          
 [25] "rs353135"           "rs353126"           "rs353126"          
 [28] "rs353126"           "rs353126"           "rs353126"          
 [31] "rs353126"           "rs10178662"         "rs10178662"        
 [34] "rs2060198"          "rs2060198"          "rs2060198"         
 [37] "rs2060198"          "rs11885001"         "rs11885001"        
 [40] "rs11885001"         "rs11885001"         "rs73969176"        
 [43] "rs77002577"         "rs62174900"         "rs62174900"        
 [46] "rs62174900"         "rs62174900"         "rs62174900"        
 [49] "rs1946892"          "rs1946892"          "rs1946892"         
 [52] "rs1946892"          "rs1946892"          "rs16850131"        
 [55] "rs16850131"         "rs16850131"         "rs16850131"        
 [58] "rs16850131"         "rs13011371"         "rs13010846"        
 [61] "rs12464762"         "rs6710092"          "rs56124029"        
 [64] "rs59172781"         "rs6756990"          "rs72264475"        
 [67] "rs11887309"         "rs11887343"         "rs9333576"         
 [70] "rs2304710"          "rs7556825"          "rs7556825"         
 [73] "rs7579000"          "rs7579000"          "rs11903711"        
 [76] "rs11903711"         "rs11903711"         "rs11903711"        
 [79] "rs11903799"         "rs11903799"         "rs11903799"        
 [82] "rs11903799"         "rs4667795"          "rs4667795"         
 [85] "rs4667795"          "rs4667795"          "rs4667795"         
 [88] "rs4667795"          "rs4667796"          "rs4667796"         
 [91] "rs4667796"          "rs4667796"          "rs4667796"         
 [94] "rs11891394"         "rs11891394"         "rs11891394"        
 [97] "rs11891394"         "rs11891394"         "rs3754962"         
[100] "rs3754962"          "rs3754962"          "rs3754962"         
[103] "rs3754962"          "rs3754961"          "rs3754961"         
[106] "rs3754961"          "rs3754961"          "rs3754961"         
[109] "rs3754960"          "rs3754960"          "rs3754960"         
[112] "rs3754960"          "rs3754960"          "rs16822785"        
[115] "rs16822785"         "rs16822785"         "rs34971284"        
[118] "2:167273403_T/C"    "rs6738031"          "rs138641247"       
[121] "rs138641247"        "rs7593207"          "rs7593207"         
[124] "rs13417885"         "rs13417885"         "rs13417885"        
[127] "rs2293567"          "rs2293567"          "rs2293567"         
[130] "rs7565062"          "rs7565062"          "rs56164822"        
[133] "rs56164822"         "rs56164822"         "rs56164822"        
[136] "rs10169523"         "rs10169523"         "rs10180721"        
[139] "rs10180721"         "2:167060900_C/T"    "2:167060900_C/T"   
[142] "2:167060900_C/T"    "rs77944059"         "rs77944059"        
[145] "rs7595255"          "rs7595255"          "rs3793936"         
[148] "rs3793936"          "rs77684130"         "rs77684130"        
[151] "2:167085081_T/G"    "2:167085081_T/G"    "rs10199799"        
[154] "rs10199799"         "rs58519838"         "rs58519838"        
[157] "rs61701909"         "rs61701909"         "rs722204"          
[160] "rs722204"           "rs6746030"          "rs6746030"         
[163] "rs6746030"          "rs6747673"          "rs6747673"         
[166] "rs6747673"          "rs6747673"          "rs13402180"        
[169] "rs13402180"         "rs13402180"         "rs13402180"        
[172] "rs13414203"         "rs13414203"         "rs13414203"        


[1] 212

In [112]:
all_var_m = as.list(unique(names(all_variants_m)))
all_var_m

[[1]]
[1] "rs568376"

[[2]]
[1] "rs577306"

[[3]]
[1] "rs116647099"

[[4]]
[1] "rs72132828"

[[5]]
[1] "rs490317"

[[6]]
[1] "2:166900130_T/G"

[[7]]
[1] "rs538921"

[[8]]
[1] "rs78391141"

[[9]]
[1] "2:166929347_C/T"

[[10]]
[1] "rs566839"

[[11]]
[1] "rs353133"

[[12]]
[1] "rs353134"

[[13]]
[1] "rs353135"

[[14]]
[1] "rs353126"

[[15]]
[1] "rs10178662"

[[16]]
[1] "rs2060198"

[[17]]
[1] "rs11885001"

[[18]]
[1] "rs73969176"

[[19]]
[1] "rs77002577"

[[20]]
[1] "rs62174900"

[[21]]
[1] "rs1946892"

[[22]]
[1] "rs16850131"

[[23]]
[1] "rs13011371"

[[24]]
[1] "rs13010846"

[[25]]
[1] "rs12464762"

[[26]]
[1] "rs6710092"

[[27]]
[1] "rs56124029"

[[28]]
[1] "rs59172781"

[[29]]
[1] "rs6756990"

[[30]]
[1] "rs72264475"

[[31]]
[1] "rs11887309"

[[32]]
[1] "rs11887343"

[[33]]
[1] "rs9333576"

[[34]]
[1] "rs2304710"

[[35]]
[1] "rs7556825"

[[36]]
[1] "rs7579000"

[[37]]
[1] "rs11903711"

[[38]]
[1] "rs11903799"

[[39]]
[1] "rs4667795"

[[40]]
[1] "rs4667796"

[[41]]
[1] "rs11891394"

[[42]]
[1] "rs3754962"

[[43]]
[1] "rs3754961"

[[44]]
[1] "rs3754960"

[[45]]
[1] "rs16822785"

[[46]]
[1] "rs34971284"

[[47]]
[1] "2:167273403_T/C"

[[48]]
[1] "rs6738031"

[[49]]
[1] "rs138641247"

[[50]]
[1] "rs7593207"

[[51]]
[1] "rs13417885"

[[52]]
[1] "rs2293567"

[[53]]
[1] "rs7565062"

[[54]]
[1] "rs56164822"

[[55]]
[1] "rs10169523"

[[56]]
[1] "rs10180721"

[[57]]
[1] "2:167060900_C/T"

[[58]]
[1] "rs77944059"

[[59]]
[1] "rs7595255"

[[60]]
[1] "rs3793936"

[[61]]
[1] "rs77684130"

[[62]]
[1] "2:167085081_T/G"

[[63]]
[1] "rs10199799"

[[64]]
[1] "rs58519838"

[[65]]
[1] "rs61701909"

[[66]]
[1] "rs722204"

[[67]]
[1] "rs6746030"

[[68]]
[1] "rs6747673"

[[69]]
[1] "rs13402180"

[[70]]
[1] "rs13414203"

[[71]]
[1] "2:167149825_G/T"

[[72]]
[1] "rs4429487"

[[73]]
[1] "rs9646771"

[[74]]
[1] "rs9646772"

[[75]]
[1] "rs58349687"

[[76]]
[1] "rs4443015"

[[77]]
[1] "rs4447616"

[[78]]
[1] "2:167163736_T/TC"

[[79]]
[1] "2:167167886_A/T"

[[80]]
[1] "rs6432899"

[[81]]
[1] "rs6432900"

[[82]]
[1] "rs6432901"

[[83]]
[1] "rs4131162"

[[84]]
[1] "rs4131161"

[[85]]
[1] "rs2385539"

[[86]]
[1] "rs75911518"

[[87]]
[1] "rs9646863"

[[88]]
[1] "rs13428617"

[[89]]
[1] "rs6599213"

[[90]]
[1] "rs41312403"

[[91]]
[1] "rs7430407"

[[92]]
[1] "rs41260344"

[[93]]
[1] "rs141932685"

[[94]]
[1] "rs7645173"

[[95]]
[1] "rs11711097"

[[96]]
[1] "rs7432532"

[[97]]
[1] "rs1805124"

[[98]]
[1] "rs55899887"

[[99]]
[1] "rs6770569"

[[100]]
[1] "rs6599222"

[[101]]
[1] "rs9873213"

[[102]]
[1] "rs9851962"

[[103]]
[1] "rs6781731"

[[104]]
[1] "rs7431305"

[[105]]
[1] "rs41312963"

[[106]]
[1] "rs41261744"

[[107]]
[1] "rs6599230"

[[108]]
[1] "rs6599241"

[[109]]
[1] "rs6599242"

[[110]]
[1] "rs6771157"

[[111]]
[1] "rs12632942"

[[112]]
[1] "rs6795970"

[[113]]
[1] "rs59468016"

[[114]]
[1] "rs57326399"

[[115]]
[1] "rs7617919"

[[116]]
[1] "rs62244070"

[[117]]
[1] "rs4234134"

[[118]]
[1] "rs4234133"

[[119]]
[1] "rs62244134"

[[120]]
[1] "rs6599266"

[[121]]
[1] "rs13099628"

[[122]]
[1] "rs9818630"

[[123]]
[1] "rs33985936"

[[124]]
[1] "rs5848481"

[[125]]
[1] "rs4484137"

[[126]]
[1] "rs4359754"

[[127]]
[1] "rs4289298"

[[128]]
[1] "rs4073113"

[[129]]
[1] "rs4553926"

[[130]]
[1] "rs4514993"

[[131]]
[1] "rs13076816"

[[132]]
[1] "rs141158768"

[[133]]
[1] "3:38948831_G/A"

[[134]]
[1] "3:38948833_G/A"

[[135]]
[1] "3:38948839_ATC/A"

[[136]]
[1] "rs62242293"

[[137]]
[1] "rs76050581"

[[138]]
[1] "rs6599267"

[[139]]
[1] "rs6599268"

[[140]]
[1] "rs7429404"

[[141]]
[1] "rs6779297"

[[142]]
[1] "rs6599269"

[[143]]
[1] "rs4585134"

[[144]]
[1] "rs4676474"

[[145]]
[1] "rs4676583"

[[146]]
[1] "rs10556067"

[[147]]
[1] "rs78812474"

[[148]]
[1] "rs73068589"

[[149]]
[1] "3:25761024_TG/T"

[[150]]
[1] "rs5847370"

[[151]]
[1] "rs2932016"

[[152]]
[1] "rs10780014"

[[153]]
[1] "rs1488991"

[[154]]
[1] "rs1488992"

[[155]]
[1] "rs6550987"

[[156]]
[1] "3:25777564_G/T"

[[157]]
[1] "rs57163579"

[[158]]
[1] "rs34986608"

[[159]]
[1] "rs3080324"

[

[[1]]
[1] "rs568376"

[[2]]
[1] "rs577306"

[[3]]
[1] "rs116647099"

[[4]]
[1] "rs72132828"

[[5]]
[1] "rs490317"

[[6]]
[1] "2:166900130_T/G"

[[7]]
[1] "rs538921"

[[8]]
[1] "rs78391141"

[[9]]
[1] "2:166929347_C/T"

[[10]]
[1] "rs566839"

[[11]]
[1] "rs353133"

[[12]]
[1] "rs353134"

[[13]]
[1] "rs353135"

[[14]]
[1] "rs353126"

[[15]]
[1] "rs10178662"

[[16]]
[1] "rs2060198"

[[17]]
[1] "rs11885001"

[[18]]
[1] "rs73969176"

[[19]]
[1] "rs77002577"

[[20]]
[1] "rs62174900"

[[21]]
[1] "rs1946892"

[[22]]
[1] "rs16850131"

[[23]]
[1] "rs13011371"

[[24]]
[1] "rs13010846"

[[25]]
[1] "rs12464762"

[[26]]
[1] "rs6710092"

[[27]]
[1] "rs56124029"

[[28]]
[1] "rs59172781"

[[29]]
[1] "rs6756990"

[[30]]
[1] "rs72264475"

[[31]]
[1] "rs11887309"

[[32]]
[1] "rs11887343"

[[33]]
[1] "rs9333576"

[[34]]
[1] "rs2304710"

[[35]]
[1] "rs7556825"

[[36]]
[1] "rs7579000"

[[37]]
[1] "rs11903711"

[[38]]
[1] "rs11903799"

[[39]]
[1] "rs4667795"

[[40]]
[1] "rs4667796"

[[41]]
[1] "rs11891394"

[[42]]
[1] "rs3754962"

[[43]]
[1] "rs3754961"

[[44]]
[1] "rs3754960"

[[45]]
[1] "rs16822785"

[[46]]
[1] "rs34971284"

[[47]]
[1] "2:167273403_T/C"

[[48]]
[1] "rs6738031"

[[49]]
[1] "rs138641247"

[[50]]
[1] "rs7593207"

[[51]]
[1] "rs13417885"

[[52]]
[1] "rs2293567"

[[53]]
[1] "rs7565062"

[[54]]
[1] "rs56164822"

[[55]]
[1] "rs10169523"

[[56]]
[1] "rs10180721"

[[57]]
[1] "2:167060900_C/T"

[[58]]
[1] "rs77944059"

[[59]]
[1] "rs7595255"

[[60]]
[1] "rs3793936"

[[61]]
[1] "rs77684130"

[[62]]
[1] "2:167085081_T/G"

[[63]]
[1] "rs10199799"

[[64]]
[1] "rs58519838"

[[65]]
[1] "rs61701909"

[[66]]
[1] "rs722204"

[[67]]
[1] "rs6746030"

[[68]]
[1] "rs6747673"

[[69]]
[1] "rs13402180"

[[70]]
[1] "rs13414203"

[[71]]
[1] "2:167149825_G/T"

[[72]]
[1] "rs4429487"

[[73]]
[1] "rs9646771"

[[74]]
[1] "rs9646772"

[[75]]
[1] "rs58349687"

[[76]]
[1] "rs4443015"

[[77]]
[1] "rs4447616"

[[78]]
[1] "2:167163736_T/TC"

[[79]]
[1] "2:167167886_A/T"

[[80]]
[1] "rs6432899"

[[81]]
[1] "rs6432900"

[[82]]
[1] "rs6432901"

[[83]]
[1] "rs4131162"

[[84]]
[1] "rs4131161"

[[85]]
[1] "rs2385539"

[[86]]
[1] "rs75911518"

[[87]]
[1] "rs9646863"

[[88]]
[1] "rs13428617"

[[89]]
[1] "rs6599213"

[[90]]
[1] "rs41312403"

[[91]]
[1] "rs7430407"

[[92]]
[1] "rs41260344"

[[93]]
[1] "rs141932685"

[[94]]
[1] "rs7645173"

[[95]]
[1] "rs11711097"

[[96]]
[1] "rs7432532"

[[97]]
[1] "rs1805124"

[[98]]
[1] "rs55899887"

[[99]]
[1] "rs6770569"

[[100]]
[1] "rs6599222"

[[101]]
[1] "rs9873213"

[[102]]
[1] "rs9851962"

[[103]]
[1] "rs6781731"

[[104]]
[1] "rs7431305"

[[105]]
[1] "rs41312963"

[[106]]
[1] "rs41261744"

[[107]]
[1] "rs6599230"

[[108]]
[1] "rs6599241"

[[109]]
[1] "rs6599242"

[[110]]
[1] "rs6771157"

[[111]]
[1] "rs12632942"

[[112]]
[1] "rs6795970"

[[113]]
[1] "rs59468016"

[[114]]
[1] "rs57326399"

[[115]]
[1] "rs7617919"

[[116]]
[1] "rs62244070"

[[117]]
[1] "rs4234134"

[[118]]
[1] "rs4234133"

[[119]]
[1] "rs62244134"

[[120]]
[1] "rs6599266"

[[121]]
[1] "rs13099628"

[[122]]
[1] "rs9818630"

[[123]]
[1] "rs33985936"

[[124]]
[1] "rs5848481"

[[125]]
[1] "rs4484137"

[[126]]
[1] "rs4359754"

[[127]]
[1] "rs4289298"

[[128]]
[1] "rs4073113"

[[129]]
[1] "rs4553926"

[[130]]
[1] "rs4514993"

[[131]]
[1] "rs13076816"

[[132]]
[1] "rs141158768"

[[133]]
[1] "3:38948831_G/A"

[[134]]
[1] "3:38948833_G/A"

[[135]]
[1] "3:38948839_ATC/A"

[[136]]
[1] "rs62242293"

[[137]]
[1] "rs76050581"

[[138]]
[1] "rs6599267"

[[139]]
[1] "rs6599268"

[[140]]
[1] "rs7429404"

[[141]]
[1] "rs6779297"

[[142]]
[1] "rs6599269"

[[143]]
[1] "rs4585134"

[[144]]
[1] "rs4676474"

[[145]]
[1] "rs4676583"

[[146]]
[1] "rs10556067"

[[147]]
[1] "rs78812474"

[[148]]
[1] "rs73068589"

[[149]]
[1] "3:25761024_TG/T"

[[150]]
[1] "rs5847370"

[[151]]
[1] "rs2932016"

[[152]]
[1] "rs10780014"

[[153]]
[1] "rs1488991"

[[154]]
[1] "rs1488992"

[[155]]
[1] "rs6550987"

[[156]]
[1] "3:25777564_G/T"

[[157]]
[1] "rs57163579"

[[158]]
[1] "rs34986608"

[[159]]
[1] "rs3080324"

[

#### 2.2 Father's Variants

In [113]:
row.names(vcf_f)
length(row.names(vcf_m))

[1] "rs34871628"              "rs67967509"             
  [3] "rs568376"                "rs577306"               
  [5] "rs1834839"               "rs140274879"            
  [7] "rs114707732"             "rs2298771"              
  [9] "rs7601520"               "rs66463257"             
 [11] "rs2020318"               "rs6719077"              
 [13] "rs6147014"               "rs1019724"              
 [15] "rs1019723"               "rs72132828"             
 [17] "rs2126152"               "rs490317"               
 [19] "rs6432859"               "rs1019722"              
 [21] "rs6432860"               "2:166897989_G/A"        
 [23] "rs35257764"              "rs10167228"             
 [25] "rs6753355"               "rs5836074"              
 [27] "rs992894"                "rs538921"               
 [29] "rs7580482"               "rs13421166"             
 [31] "rs78391141"              "rs1461193"              
 [33] "rs994399"                "rs6706163"              
 [35] "rs6750294"               "rs34747425"             
 [37] "rs3812719"               "rs2195144"              
 [39] "rs2859817"               "rs8191987"              
 [41] "rs10930201"              "2:166929347_C/T"        
 [43] "rs566839"                "rs503872"               
 [45] "rs4667801"               "rs353127"               
 [47] "rs353133"                "rs353134"               
 [49] "rs353135"                "rs353126"               
 [51] "rs2075704"               "rs12468669"             
 [53] "rs139906774"             "rs7593568"              
 [55] "rs112475855"             "rs1965757"              
 [57] "rs2116658"               "rs7560348"              
 [59] "rs13025009"              "rs2121371"              
 [61] "rs1838846"               "rs1816918"              
 [63] "rs1867864"               "rs12993173"             
 [65] "rs149798712"             "rs7596027"              
 [67] "rs62174667"              "2:166210421_ATATTTATC/A"
 [69] "2:166210435_GA/G"        "rs10191771"             
 [71] "rs67198220"              "rs36121220"             
 [73] "2:166221949_G/T"         "rs3835935"              
 [75] "rs2304010"               "rs59934051"             
 [77] "rs2304012"               "rs2304013"              
 [79] "rs1469649"               "rs12692768"             
 [81] "rs6432821"               "rs1368238"              
 [83] "rs1864885"               "rs67443324"             
 [85] "rs7578237"               "rs10178662"             
 [87] "rs3213904"               "rs1946892"              
 [89] "rs2289400"               "rs79777698"             
 [91] "rs10048748"              "rs138549123"            
 [93] "rs78008317"              "rs7574983"              
 [95] "rs7561140"               "rs3731761"              
 [97] "rs3731760"               "2:166012356_G/T"        
 [99] "rs13011371"              "rs13010846"             
[101] "rs7572701"               "rs7586796"              
[103] "rs6710092"               "rs56124029"             
[105] "rs59172781"              "rs6756990"              
[107] "rs72264475"              "rs11887309"             
[109] "rs11887343"              "rs9333576"              
[111] "rs2304710"               "rs7556825"              
[113] "rs7579000"               "rs116682517"            
[115] "rs11903711"              "rs11903799"             
[117] "rs4667794"               "rs4667795"              
[119] "rs4667796"               "rs72532011"             
[121] "rs11891394"              "rs3754962"              
[123] "rs3754961"               "rs3754960"              
[125] "rs16822785"              "rs1406276"              
[127] "rs1406274"               "rs17819028"             
[129] "rs10930221"              "rs7561953"              
[131] "rs72623155"              "rs6738031"              
[133] "rs4550683"               "rs3791253"              
[135] "rs7570201"               "2:167300960_G/C"        
[137] "rs113442768"             "rs138641247"            


[1] 285

[1] "rs34871628"              "rs67967509"             
  [3] "rs568376"                "rs577306"               
  [5] "rs1834839"               "rs140274879"            
  [7] "rs114707732"             "rs2298771"              
  [9] "rs7601520"               "rs66463257"             
 [11] "rs2020318"               "rs6719077"              
 [13] "rs6147014"               "rs1019724"              
 [15] "rs1019723"               "rs72132828"             
 [17] "rs2126152"               "rs490317"               
 [19] "rs6432859"               "rs1019722"              
 [21] "rs6432860"               "2:166897989_G/A"        
 [23] "rs35257764"              "rs10167228"             
 [25] "rs6753355"               "rs5836074"              
 [27] "rs992894"                "rs538921"               
 [29] "rs7580482"               "rs13421166"             
 [31] "rs78391141"              "rs1461193"              
 [33] "rs994399"                "rs6706163"              
 [35] "rs6750294"               "rs34747425"             
 [37] "rs3812719"               "rs2195144"              
 [39] "rs2859817"               "rs8191987"              
 [41] "rs10930201"              "2:166929347_C/T"        
 [43] "rs566839"                "rs503872"               
 [45] "rs4667801"               "rs353127"               
 [47] "rs353133"                "rs353134"               
 [49] "rs353135"                "rs353126"               
 [51] "rs2075704"               "rs12468669"             
 [53] "rs139906774"             "rs7593568"              
 [55] "rs112475855"             "rs1965757"              
 [57] "rs2116658"               "rs7560348"              
 [59] "rs13025009"              "rs2121371"              
 [61] "rs1838846"               "rs1816918"              
 [63] "rs1867864"               "rs12993173"             
 [65] "rs149798712"             "rs7596027"              
 [67] "rs62174667"              "2:166210421_ATATTTATC/A"
 [69] "2:166210435_GA/G"        "rs10191771"             
 [71] "rs67198220"              "rs36121220"             
 [73] "2:166221949_G/T"         "rs3835935"              
 [75] "rs2304010"               "rs59934051"             
 [77] "rs2304012"               "rs2304013"              
 [79] "rs1469649"               "rs12692768"             
 [81] "rs6432821"               "rs1368238"              
 [83] "rs1864885"               "rs67443324"             
 [85] "rs7578237"               "rs10178662"             
 [87] "rs3213904"               "rs1946892"              
 [89] "rs2289400"               "rs79777698"             
 [91] "rs10048748"              "rs138549123"            
 [93] "rs78008317"              "rs7574983"              
 [95] "rs7561140"               "rs3731761"              
 [97] "rs3731760"               "2:166012356_G/T"        
 [99] "rs13011371"              "rs13010846"             
[101] "rs7572701"               "rs7586796"              
[103] "rs6710092"               "rs56124029"             
[105] "rs59172781"              "rs6756990"              
[107] "rs72264475"              "rs11887309"             
[109] "rs11887343"              "rs9333576"              
[111] "rs2304710"               "rs7556825"              
[113] "rs7579000"               "rs116682517"            
[115] "rs11903711"              "rs11903799"             
[117] "rs4667794"               "rs4667795"              
[119] "rs4667796"               "rs72532011"             
[121] "rs11891394"              "rs3754962"              
[123] "rs3754961"               "rs3754960"              
[125] "rs16822785"              "rs1406276"              
[127] "rs1406274"               "rs17819028"             
[129] "rs10930221"              "rs7561953"              
[131] "rs72623155"              "rs6738031"              
[133] "rs4550683"               "rs3791253"              
[135] "rs7570201"               "2:167300960_G/C"        
[137] "rs113442768"             "rs138641247"            


[1] 285

In [114]:
class(vcf_f)

[1] "CollapsedVCF"
attr(,"package")
[1] "VariantAnnotation"

[1] "CollapsedVCF"
attr(,"package")
[1] "VariantAnnotation"

In [115]:
# Extract the variant IDs into a list
vcf_f_l = lapply(vcf_f, rownames)
vcf_f_l

$rs34871628
[1] "rs34871628"

$rs67967509
[1] "rs67967509"

$rs568376
[1] "rs568376"

$rs577306
[1] "rs577306"

$rs1834839
[1] "rs1834839"

$rs140274879
[1] "rs140274879"

$rs114707732
[1] "rs114707732"

$rs2298771
[1] "rs2298771"

$rs7601520
[1] "rs7601520"

$rs66463257
[1] "rs66463257"

$rs2020318
[1] "rs2020318"

$rs6719077
[1] "rs6719077"

$rs6147014
[1] "rs6147014"

$rs1019724
[1] "rs1019724"

$rs1019723
[1] "rs1019723"

$rs72132828
[1] "rs72132828"

$rs2126152
[1] "rs2126152"

$rs490317
[1] "rs490317"

$rs6432859
[1] "rs6432859"

$rs1019722
[1] "rs1019722"

$rs6432860
[1] "rs6432860"

$`2:166897989_G/A`
[1] "2:166897989_G/A"

$rs35257764
[1] "rs35257764"

$rs10167228
[1] "rs10167228"

$rs6753355
[1] "rs6753355"

$rs5836074
[1] "rs5836074"

$rs992894
[1] "rs992894"

$rs538921
[1] "rs538921"

$rs7580482
[1] "rs7580482"

$rs13421166
[1] "rs13421166"

$rs78391141
[1] "rs78391141"

$rs1461193
[1] "rs1461193"

$rs994399
[1] "rs994399"

$rs6706163
[1] "rs6706163"

$rs6750294
[1] "rs6750294"

$rs34747425
[1] "rs34747425"

$rs3812719
[1] "rs3812719"

$rs2195144
[1] "rs2195144"

$rs2859817
[1] "rs2859817"

$rs8191987
[1] "rs8191987"

$rs10930201
[1] "rs10930201"

$`2:166929347_C/T`
[1] "2:166929347_C/T"

$rs566839
[1] "rs566839"

$rs503872
[1] "rs503872"

$rs4667801
[1] "rs4667801"

$rs353127
[1] "rs353127"

$rs353133
[1] "rs353133"

$rs353134
[1] "rs353134"

$rs353135
[1] "rs353135"

$rs353126
[1] "rs353126"

$rs2075704
[1] "rs2075704"

$rs12468669
[1] "rs12468669"

$rs139906774
[1] "rs139906774"

$rs7593568
[1] "rs7593568"

$rs112475855
[1] "rs112475855"

$rs1965757
[1] "rs1965757"

$rs2116658
[1] "rs2116658"

$rs7560348
[1] "rs7560348"

$rs13025009
[1] "rs13025009"

$rs2121371
[1] "rs2121371"

$rs1838846
[1] "rs1838846"

$rs1816918
[1] "rs1816918"

$rs1867864
[1] "rs1867864"

$rs12993173
[1] "rs12993173"

$rs149798712
[1] "rs149798712"

$rs7596027
[1] "rs7596027"

$rs62174667
[1] "rs62174667"

$`2:166210421_ATATTTATC/A`
[1] "2:166210421_ATATTTATC/A"

$`2:166210435_GA/G`
[1] "2:166210435_GA/G"

$rs10191771
[1] "rs10191771"

$rs67198220
[1] "rs67198220"

$rs36121220
[1] "rs36121220"

$`2:166221949_G/T`
[1] "2:166221949_G/T"

$rs3835935
[1] "rs3835935"

$rs2304010
[1] "rs2304010"

$rs59934051
[1] "rs59934051"

$rs2304012
[1] "rs2304012"

$rs2304013
[1] "rs2304013"

$rs1469649
[1] "rs1469649"

$rs12692768
[1] "rs12692768"

$rs6432821
[1] "rs6432821"

$rs1368238
[1] "rs1368238"

$rs1864885
[1] "rs1864885"

$rs67443324
[1] "rs67443324"

$rs7578237
[1] "rs7578237"

$rs10178662
[1] "rs10178662"

$rs3213904
[1] "rs3213904"

$rs1946892
[1] "rs1946892"

$rs2289400
[1] "rs2289400"

$rs79777698
[1] "rs79777698"

$rs10048748
[1] "rs10048748"

$rs138549123
[1] "rs138549123"

$rs78008317
[1] "rs78008317"

$rs7574983
[1] "rs7574983"

$rs7561140
[1] "rs7561140"

$rs3731761
[1] "rs3731761"

$rs3731760
[1] "rs3731760"

$`2:166012356_G/T`
[1] "2:166012356_G/T"

$rs13011371
[1] "rs13011371"

$rs13010846
[1] "rs13010846"

$rs7572701
[1] "rs7572701"

$rs7586796
[1] "rs7586796"

$rs6710092
[1] "rs6710092"

$rs56124029
[1] "rs56124029"

$rs59172781
[1] "rs59172781"

$rs6756990
[1] "rs6756990"

$rs72264475
[1] "rs72264475"

$rs11887309
[1] "rs11887309"

$rs11887343
[1] "rs11887343"

$rs9333576
[1] "rs9333576"

$rs2304710
[1] "rs2304710"

$rs7556825
[1] "rs7556825"

$rs7579000
[1] "rs7579000"

$rs116682517
[1] "rs116682517"

$rs11903711
[1] "rs11903711"

$rs11903799
[1] "rs11903799"

$rs4667794
[1] "rs4667794"

$rs4667795
[1] "rs4667795"

$rs4667796
[1] "rs4667796"

$rs72532011
[1] "rs72532011"

$rs11891394
[1] "rs11891394"

$rs3754962
[1] "rs3754962"

$rs3754961
[1] "rs3754961"

$rs3754960
[1] "rs3754960"

$rs16822785
[1] "rs16822785"

$rs1406276
[1] "rs1406276"

$rs1406274
[1] "rs1406274"

$rs17819028
[1] "rs17819028"

$rs10930221
[1] "rs10930221"

$rs7561953
[1] "rs7561953"

$rs72623155
[1] "rs72623155"

$rs6738031
[1] "rs6738031"

$rs4550683
[1] "rs4550683"

$rs3791253
[1] "rs3791253"

$rs7570201
[1] "rs7570201"

$`2:167300960_G/C`
[1] "2:167300960_G/

$rs34871628
[1] "rs34871628"

$rs67967509
[1] "rs67967509"

$rs568376
[1] "rs568376"

$rs577306
[1] "rs577306"

$rs1834839
[1] "rs1834839"

$rs140274879
[1] "rs140274879"

$rs114707732
[1] "rs114707732"

$rs2298771
[1] "rs2298771"

$rs7601520
[1] "rs7601520"

$rs66463257
[1] "rs66463257"

$rs2020318
[1] "rs2020318"

$rs6719077
[1] "rs6719077"

$rs6147014
[1] "rs6147014"

$rs1019724
[1] "rs1019724"

$rs1019723
[1] "rs1019723"

$rs72132828
[1] "rs72132828"

$rs2126152
[1] "rs2126152"

$rs490317
[1] "rs490317"

$rs6432859
[1] "rs6432859"

$rs1019722
[1] "rs1019722"

$rs6432860
[1] "rs6432860"

$`2:166897989_G/A`
[1] "2:166897989_G/A"

$rs35257764
[1] "rs35257764"

$rs10167228
[1] "rs10167228"

$rs6753355
[1] "rs6753355"

$rs5836074
[1] "rs5836074"

$rs992894
[1] "rs992894"

$rs538921
[1] "rs538921"

$rs7580482
[1] "rs7580482"

$rs13421166
[1] "rs13421166"

$rs78391141
[1] "rs78391141"

$rs1461193
[1] "rs1461193"

$rs994399
[1] "rs994399"

$rs6706163
[1] "rs6706163"

$rs6750294
[1] "rs6750294"

$rs34747425
[1] "rs34747425"

$rs3812719
[1] "rs3812719"

$rs2195144
[1] "rs2195144"

$rs2859817
[1] "rs2859817"

$rs8191987
[1] "rs8191987"

$rs10930201
[1] "rs10930201"

$`2:166929347_C/T`
[1] "2:166929347_C/T"

$rs566839
[1] "rs566839"

$rs503872
[1] "rs503872"

$rs4667801
[1] "rs4667801"

$rs353127
[1] "rs353127"

$rs353133
[1] "rs353133"

$rs353134
[1] "rs353134"

$rs353135
[1] "rs353135"

$rs353126
[1] "rs353126"

$rs2075704
[1] "rs2075704"

$rs12468669
[1] "rs12468669"

$rs139906774
[1] "rs139906774"

$rs7593568
[1] "rs7593568"

$rs112475855
[1] "rs112475855"

$rs1965757
[1] "rs1965757"

$rs2116658
[1] "rs2116658"

$rs7560348
[1] "rs7560348"

$rs13025009
[1] "rs13025009"

$rs2121371
[1] "rs2121371"

$rs1838846
[1] "rs1838846"

$rs1816918
[1] "rs1816918"

$rs1867864
[1] "rs1867864"

$rs12993173
[1] "rs12993173"

$rs149798712
[1] "rs149798712"

$rs7596027
[1] "rs7596027"

$rs62174667
[1] "rs62174667"

$`2:166210421_ATATTTATC/A`
[1] "2:166210421_ATATTTATC/A"

$`2:166210435_GA/G`
[1] "2:166210435_GA/G"

$rs10191771
[1] "rs10191771"

$rs67198220
[1] "rs67198220"

$rs36121220
[1] "rs36121220"

$`2:166221949_G/T`
[1] "2:166221949_G/T"

$rs3835935
[1] "rs3835935"

$rs2304010
[1] "rs2304010"

$rs59934051
[1] "rs59934051"

$rs2304012
[1] "rs2304012"

$rs2304013
[1] "rs2304013"

$rs1469649
[1] "rs1469649"

$rs12692768
[1] "rs12692768"

$rs6432821
[1] "rs6432821"

$rs1368238
[1] "rs1368238"

$rs1864885
[1] "rs1864885"

$rs67443324
[1] "rs67443324"

$rs7578237
[1] "rs7578237"

$rs10178662
[1] "rs10178662"

$rs3213904
[1] "rs3213904"

$rs1946892
[1] "rs1946892"

$rs2289400
[1] "rs2289400"

$rs79777698
[1] "rs79777698"

$rs10048748
[1] "rs10048748"

$rs138549123
[1] "rs138549123"

$rs78008317
[1] "rs78008317"

$rs7574983
[1] "rs7574983"

$rs7561140
[1] "rs7561140"

$rs3731761
[1] "rs3731761"

$rs3731760
[1] "rs3731760"

$`2:166012356_G/T`
[1] "2:166012356_G/T"

$rs13011371
[1] "rs13011371"

$rs13010846
[1] "rs13010846"

$rs7572701
[1] "rs7572701"

$rs7586796
[1] "rs7586796"

$rs6710092
[1] "rs6710092"

$rs56124029
[1] "rs56124029"

$rs59172781
[1] "rs59172781"

$rs6756990
[1] "rs6756990"

$rs72264475
[1] "rs72264475"

$rs11887309
[1] "rs11887309"

$rs11887343
[1] "rs11887343"

$rs9333576
[1] "rs9333576"

$rs2304710
[1] "rs2304710"

$rs7556825
[1] "rs7556825"

$rs7579000
[1] "rs7579000"

$rs116682517
[1] "rs116682517"

$rs11903711
[1] "rs11903711"

$rs11903799
[1] "rs11903799"

$rs4667794
[1] "rs4667794"

$rs4667795
[1] "rs4667795"

$rs4667796
[1] "rs4667796"

$rs72532011
[1] "rs72532011"

$rs11891394
[1] "rs11891394"

$rs3754962
[1] "rs3754962"

$rs3754961
[1] "rs3754961"

$rs3754960
[1] "rs3754960"

$rs16822785
[1] "rs16822785"

$rs1406276
[1] "rs1406276"

$rs1406274
[1] "rs1406274"

$rs17819028
[1] "rs17819028"

$rs10930221
[1] "rs10930221"

$rs7561953
[1] "rs7561953"

$rs72623155
[1] "rs72623155"

$rs6738031
[1] "rs6738031"

$rs4550683
[1] "rs4550683"

$rs3791253
[1] "rs3791253"

$rs7570201
[1] "rs7570201"

$`2:167300960_G/C`
[1] "2:167300960_G/

There are **285** total variants in the father's VCF file.

`locateVariants()` captures variants with genes' intron, cds, splice site, intergenic, promoter, and utr regions.

In [116]:
all_variants_f <- locateVariants( query   = vcf_f, 
                       subject = txdbgene, 
                       region  = AllVariants() )

Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 323 out-of-bound ranges located on sequences
  10254, 10255, 15589, 15590, 15591, 15592, 15593, 15594, 15595, 15596,
  15597, 15598, 15599, 15600, 15601, 15602, 15603, 15606, 15607, 15507,
  46258, 46259, 46260, 46256, 46257, and 64075. Note that ranges located
  on a sequence whose length is unknown (NA) or on a circular sequence
  are not considered out-of-bound (use seqlengths() and isCircular() to
  get the lengths and circularity flags of the underlying sequences). You
  can use trim() to trim these ranges. See ?`trim,GenomicRanges-method`
  for more information.”
Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 323 out-of-bound ranges located on sequences
  10254, 10255, 15589, 15590, 15591, 15592, 15593, 15594, 15595, 15596,
  15597, 15598, 15599, 15600, 15601, 15602, 15603, 15606, 15607, 15507,
  46258, 46259, 46260, 46256, 46257, and 

In [117]:
class(all_variants_f)
head(all_variants_f)

[1] "GRanges"
attr(,"package")
[1] "GenomicRanges"

[1] "GRanges"
attr(,"package")
[1] "GenomicRanges"

GRanges object with 6 ranges and 9 metadata columns:
              seqnames              ranges strand | LOCATION  LOCSTART
                 <Rle>           <IRanges>  <Rle> | <factor> <integer>
     rs568376        2           166870600      - |   intron     56066
     rs577306        2           166871905      - |   intron     54761
    rs1834839        2           166885888      - |   intron     40899
  rs140274879        2 166892444-166892447      - |   intron     34340
  rs114707732        2           166892448      - |   intron     34339
    rs2298771        2           166892788      - |   coding      3115
                 LOCEND   QUERYID        TXID         CDSID      GENEID
              <integer> <integer> <character> <IntegerList> <character>
     rs568376     56066         3       11728                    653489
     rs577306     54761         4       11728                    653489
    rs1834839     40899         5       11728                    653489
  rs140274879     3

GRanges object with 6 ranges and 9 metadata columns:
              seqnames              ranges strand | LOCATION  LOCSTART
                 <Rle>           <IRanges>  <Rle> | <factor> <integer>
     rs568376        2           166870600      - |   intron     56066
     rs577306        2           166871905      - |   intron     54761
    rs1834839        2           166885888      - |   intron     40899
  rs140274879        2 166892444-166892447      - |   intron     34340
  rs114707732        2           166892448      - |   intron     34339
    rs2298771        2           166892788      - |   coding      3115
                 LOCEND   QUERYID        TXID         CDSID      GENEID
              <integer> <integer> <character> <IntegerList> <character>
     rs568376     56066         3       11728                    653489
     rs577306     54761         4       11728                    653489
    rs1834839     40899         5       11728                    653489
  rs140274879     3

In [118]:
unique(mcols(all_variants_f)$GENEID)

[1] "653489" "6323"   "84620"  "6329"   "3384"   "445582" "2688"   "55041" 
 [9] "974"    "6326"   "6328"   "2274"   "6332"   "10913"  "6335"   "729121"
[17] NA       "659"    "150864" "29926"  "6331"   "4311"   "8833"   "25976" 
[25] "7881"   "6336"   "11280"  "389170" "23350"  "55768"  "9435"   "1181"  
[33] "6334"   "7066"   "4598"   "8086"   "79058"

[1] "653489" "6323"   "84620"  "6329"   "3384"   "445582" "2688"   "55041" 
 [9] "974"    "6326"   "6328"   "2274"   "6332"   "10913"  "6335"   "729121"
[17] NA       "659"    "150864" "29926"  "6331"   "4311"   "8833"   "25976" 
[25] "7881"   "6336"   "11280"  "389170" "23350"  "55768"  "9435"   "1181"  
[33] "6334"   "7066"   "4598"   "8086"   "79058"

In [119]:
table(mcols(all_variants_f)$LOCATION)


spliceSite     intron    fiveUTR   threeUTR     coding intergenic   promoter 
         0        608          1         23        103          0         49 


spliceSite     intron    fiveUTR   threeUTR     coding intergenic   promoter 
         0        608          1         23        103          0         49 

In [120]:
# There are repeats in this dataset. Count how many unique variants there are.
names(all_variants_f)
length(unique(names(all_variants_f)))

[1] "rs568376"           "rs577306"           "rs1834839"         
  [4] "rs140274879"        "rs114707732"        "rs2298771"         
  [7] "rs2298771"          "rs2298771"          "rs7601520"         
 [10] "rs66463257"         "rs2020318"          "rs6719077"         
 [13] "rs6147014"          "rs1019724"          "rs1019723"         
 [16] "rs72132828"         "rs2126152"          "rs490317"          
 [19] "rs6432859"          "rs1019722"          "rs6432860"         
 [22] "rs6432860"          "rs6432860"          "2:166897989_G/A"   
 [25] "rs35257764"         "rs10167228"         "rs6753355"         
 [28] "rs5836074"          "rs992894"           "rs538921"          
 [31] "rs7580482"          "rs7580482"          "rs7580482"         
 [34] "rs13421166"         "rs78391141"         "rs78391141"        
 [37] "rs78391141"         "rs1461193"          "rs994399"          
 [40] "rs6706163"          "rs6750294"          "rs34747425"        
 [43] "rs34747425"         "rs3812719"          "rs3812719"         
 [46] "rs2195144"          "rs2195144"          "rs2859817"         
 [49] "rs2859817"          "rs8191987"          "rs8191987"         
 [52] "rs8191987"          "rs10930201"         "rs10930201"        
 [55] "rs10930201"         "2:166929347_C/T"    "2:166929347_C/T"   
 [58] "2:166929347_C/T"    "rs566839"           "rs566839"          
 [61] "rs566839"           "rs503872"           "rs503872"          
 [64] "rs503872"           "rs503872"           "rs4667801"         
 [67] "rs4667801"          "rs4667801"          "rs353127"          
 [70] "rs353127"           "rs353127"           "rs353133"          
 [73] "rs353133"           "rs353133"           "rs353134"          
 [76] "rs353134"           "rs353134"           "rs353135"          
 [79] "rs353135"           "rs353135"           "rs353126"          
 [82] "rs353126"           "rs353126"           "rs353126"          
 [85] "rs353126"           "rs353126"           "rs2075704"         
 [88] "rs2075704"          "rs2075704"          "rs2075704"         
 [91] "rs2075704"          "rs2075704"          "rs12468669"        
 [94] "rs12468669"         "rs12468669"         "rs12468669"        
 [97] "rs12468669"         "rs12468669"         "rs12468669"        
[100] "rs139906774"        "rs139906774"        "rs139906774"       
[103] "rs139906774"        "rs139906774"        "rs139906774"       
[106] "rs139906774"        "rs7593568"          "rs7593568"         
[109] "rs7593568"          "rs7593568"          "rs7593568"         
[112] "rs7593568"          "rs7593568"          "rs112475855"       
[115] "rs112475855"        "rs112475855"        "rs112475855"       
[118] "rs112475855"        "rs112475855"        "rs112475855"       
[121] "rs1965757"          "rs1965757"          "rs1965757"         
[124] "rs1965757"          "rs1965757"          "rs1965757"         
[127] "rs2116658"          "rs2116658"          "rs2116658"         
[130] "rs2116658"          "rs7560348"          "rs67443324"        
[133] "rs7578237"          "rs7578237"          "rs10178662"        
[136] "rs10178662"         "rs1946892"          "rs1946892"         
[139] "rs1946892"          "rs1946892"          "rs1946892"         
[142] "2:166012356_G/T"    "2:166012356_G/T"    "2:166012356_G/T"   
[145] "2:166012356_G/T"    "2:166012356_G/T"    "rs13011371"        
[148] "rs13010846"         "rs7572701"          "rs7586796"         
[151] "rs6710092"          "rs56124029"         "rs59172781"        
[154] "rs6756990"          "rs72264475"         "rs11887309"        
[157] "rs11887343"         "rs9333576"          "rs2304710"         
[160] "rs7556825"          "rs7556825"          "rs7579000"         
[163] "rs7579000"          "rs116682517"        "rs116682517"       
[166] "rs116682517"        "rs116682517"        "rs116682517"       
[169] "rs11903711"         "rs11903711"         "rs11903711"        
[172] "rs11903711"         "rs11903799"         "rs11903799"        


[1] 237

[1] "rs568376"           "rs577306"           "rs1834839"         
  [4] "rs140274879"        "rs114707732"        "rs2298771"         
  [7] "rs2298771"          "rs2298771"          "rs7601520"         
 [10] "rs66463257"         "rs2020318"          "rs6719077"         
 [13] "rs6147014"          "rs1019724"          "rs1019723"         
 [16] "rs72132828"         "rs2126152"          "rs490317"          
 [19] "rs6432859"          "rs1019722"          "rs6432860"         
 [22] "rs6432860"          "rs6432860"          "2:166897989_G/A"   
 [25] "rs35257764"         "rs10167228"         "rs6753355"         
 [28] "rs5836074"          "rs992894"           "rs538921"          
 [31] "rs7580482"          "rs7580482"          "rs7580482"         
 [34] "rs13421166"         "rs78391141"         "rs78391141"        
 [37] "rs78391141"         "rs1461193"          "rs994399"          
 [40] "rs6706163"          "rs6750294"          "rs34747425"        
 [43] "rs34747425"         "rs3812719"          "rs3812719"         
 [46] "rs2195144"          "rs2195144"          "rs2859817"         
 [49] "rs2859817"          "rs8191987"          "rs8191987"         
 [52] "rs8191987"          "rs10930201"         "rs10930201"        
 [55] "rs10930201"         "2:166929347_C/T"    "2:166929347_C/T"   
 [58] "2:166929347_C/T"    "rs566839"           "rs566839"          
 [61] "rs566839"           "rs503872"           "rs503872"          
 [64] "rs503872"           "rs503872"           "rs4667801"         
 [67] "rs4667801"          "rs4667801"          "rs353127"          
 [70] "rs353127"           "rs353127"           "rs353133"          
 [73] "rs353133"           "rs353133"           "rs353134"          
 [76] "rs353134"           "rs353134"           "rs353135"          
 [79] "rs353135"           "rs353135"           "rs353126"          
 [82] "rs353126"           "rs353126"           "rs353126"          
 [85] "rs353126"           "rs353126"           "rs2075704"         
 [88] "rs2075704"          "rs2075704"          "rs2075704"         
 [91] "rs2075704"          "rs2075704"          "rs12468669"        
 [94] "rs12468669"         "rs12468669"         "rs12468669"        
 [97] "rs12468669"         "rs12468669"         "rs12468669"        
[100] "rs139906774"        "rs139906774"        "rs139906774"       
[103] "rs139906774"        "rs139906774"        "rs139906774"       
[106] "rs139906774"        "rs7593568"          "rs7593568"         
[109] "rs7593568"          "rs7593568"          "rs7593568"         
[112] "rs7593568"          "rs7593568"          "rs112475855"       
[115] "rs112475855"        "rs112475855"        "rs112475855"       
[118] "rs112475855"        "rs112475855"        "rs112475855"       
[121] "rs1965757"          "rs1965757"          "rs1965757"         
[124] "rs1965757"          "rs1965757"          "rs1965757"         
[127] "rs2116658"          "rs2116658"          "rs2116658"         
[130] "rs2116658"          "rs7560348"          "rs67443324"        
[133] "rs7578237"          "rs7578237"          "rs10178662"        
[136] "rs10178662"         "rs1946892"          "rs1946892"         
[139] "rs1946892"          "rs1946892"          "rs1946892"         
[142] "2:166012356_G/T"    "2:166012356_G/T"    "2:166012356_G/T"   
[145] "2:166012356_G/T"    "2:166012356_G/T"    "rs13011371"        
[148] "rs13010846"         "rs7572701"          "rs7586796"         
[151] "rs6710092"          "rs56124029"         "rs59172781"        
[154] "rs6756990"          "rs72264475"         "rs11887309"        
[157] "rs11887343"         "rs9333576"          "rs2304710"         
[160] "rs7556825"          "rs7556825"          "rs7579000"         
[163] "rs7579000"          "rs116682517"        "rs116682517"       
[166] "rs116682517"        "rs116682517"        "rs116682517"       
[169] "rs11903711"         "rs11903711"         "rs11903711"        
[172] "rs11903711"         "rs11903799"         "rs11903799"        


[1] 237

In [121]:
# List of unique variants
all_var_f = as.list(unique(names(all_variants_f)))
all_var_f

[[1]]
[1] "rs568376"

[[2]]
[1] "rs577306"

[[3]]
[1] "rs1834839"

[[4]]
[1] "rs140274879"

[[5]]
[1] "rs114707732"

[[6]]
[1] "rs2298771"

[[7]]
[1] "rs7601520"

[[8]]
[1] "rs66463257"

[[9]]
[1] "rs2020318"

[[10]]
[1] "rs6719077"

[[11]]
[1] "rs6147014"

[[12]]
[1] "rs1019724"

[[13]]
[1] "rs1019723"

[[14]]
[1] "rs72132828"

[[15]]
[1] "rs2126152"

[[16]]
[1] "rs490317"

[[17]]
[1] "rs6432859"

[[18]]
[1] "rs1019722"

[[19]]
[1] "rs6432860"

[[20]]
[1] "2:166897989_G/A"

[[21]]
[1] "rs35257764"

[[22]]
[1] "rs10167228"

[[23]]
[1] "rs6753355"

[[24]]
[1] "rs5836074"

[[25]]
[1] "rs992894"

[[26]]
[1] "rs538921"

[[27]]
[1] "rs7580482"

[[28]]
[1] "rs13421166"

[[29]]
[1] "rs78391141"

[[30]]
[1] "rs1461193"

[[31]]
[1] "rs994399"

[[32]]
[1] "rs6706163"

[[33]]
[1] "rs6750294"

[[34]]
[1] "rs34747425"

[[35]]
[1] "rs3812719"

[[36]]
[1] "rs2195144"

[[37]]
[1] "rs2859817"

[[38]]
[1] "rs8191987"

[[39]]
[1] "rs10930201"

[[40]]
[1] "2:166929347_C/T"

[[41]]
[1] "rs566839"

[[42]]
[1] "rs503872"

[[43]]
[1] "rs4667801"

[[44]]
[1] "rs353127"

[[45]]
[1] "rs353133"

[[46]]
[1] "rs353134"

[[47]]
[1] "rs353135"

[[48]]
[1] "rs353126"

[[49]]
[1] "rs2075704"

[[50]]
[1] "rs12468669"

[[51]]
[1] "rs139906774"

[[52]]
[1] "rs7593568"

[[53]]
[1] "rs112475855"

[[54]]
[1] "rs1965757"

[[55]]
[1] "rs2116658"

[[56]]
[1] "rs7560348"

[[57]]
[1] "rs67443324"

[[58]]
[1] "rs7578237"

[[59]]
[1] "rs10178662"

[[60]]
[1] "rs1946892"

[[61]]
[1] "2:166012356_G/T"

[[62]]
[1] "rs13011371"

[[63]]
[1] "rs13010846"

[[64]]
[1] "rs7572701"

[[65]]
[1] "rs7586796"

[[66]]
[1] "rs6710092"

[[67]]
[1] "rs56124029"

[[68]]
[1] "rs59172781"

[[69]]
[1] "rs6756990"

[[70]]
[1] "rs72264475"

[[71]]
[1] "rs11887309"

[[72]]
[1] "rs11887343"

[[73]]
[1] "rs9333576"

[[74]]
[1] "rs2304710"

[[75]]
[1] "rs7556825"

[[76]]
[1] "rs7579000"

[[77]]
[1] "rs116682517"

[[78]]
[1] "rs11903711"

[[79]]
[1] "rs11903799"

[[80]]
[1] "rs4667794"

[[81]]
[1] "rs4667795"

[[82]]
[1] "rs4667796"

[[83]]
[1] "rs72532011"

[[84]]
[1] "rs11891394"

[[85]]
[1] "rs3754962"

[[86]]
[1] "rs3754961"

[[87]]
[1] "rs3754960"

[[88]]
[1] "rs16822785"

[[89]]
[1] "rs6738031"

[[90]]
[1] "rs113442768"

[[91]]
[1] "rs138641247"

[[92]]
[1] "rs7593207"

[[93]]
[1] "rs2293567"

[[94]]
[1] "rs150350335"

[[95]]
[1] "rs7565062"

[[96]]
[1] "rs56164822"

[[97]]
[1] "rs56821066"

[[98]]
[1] "rs10169523"

[[99]]
[1] "rs10180721"

[[100]]
[1] "2:167060900_C/T"

[[101]]
[1] "rs77944059"

[[102]]
[1] "rs34141377"

[[103]]
[1] "rs2212662"

[[104]]
[1] "rs7595255"

[[105]]
[1] "rs3793936"

[[106]]
[1] "rs77684130"

[[107]]
[1] "2:167085081_T/G"

[[108]]
[1] "rs10199799"

[[109]]
[1] "rs58519838"

[[110]]
[1] "2:167085142_T/A"

[[111]]
[1] "rs61701909"

[[112]]
[1] "rs722204"

[[113]]
[1] "rs6758728"

[[114]]
[1] "rs6746030"

[[115]]
[1] "rs4429487"

[[116]]
[1] "rs9646771"

[[117]]
[1] "rs9646772"

[[118]]
[1] "rs58349687"

[[119]]
[1] "rs4443015"

[[120]]
[1] "rs4447616"

[[121]]
[1] "rs6432899"

[[122]]
[1] "rs6432900"

[[123]]
[1] "rs6432901"

[[124]]
[1] "rs4131162"

[[125]]
[1] "rs4131161"

[[126]]
[1] "rs4386335"

[[127]]
[1] "rs2385539"

[[128]]
[1] "rs9646863"

[[129]]
[1] "rs1046474"

[[130]]
[1] "rs7429945"

[[131]]
[1] "rs1805126"

[[132]]
[1] "rs41315501"

[[133]]
[1] "rs7645178"

[[134]]
[1] "rs41315507"

[[135]]
[1] "rs45505695"

[[136]]
[1] "rs6804918"

[[137]]
[1] "rs6793245"

[[138]]
[1] "rs6799868"

[[139]]
[1] "rs7374138"

[[140]]
[1] "rs6599213"

[[141]]
[1] "rs7430407"

[[142]]
[1] "rs7432766"

[[143]]
[1] "rs7645173"

[[144]]
[1] "rs7645358"

[[145]]
[1] "rs7432532"

[[146]]
[1] "rs7433889"

[[147]]
[1] "rs6599214"

[[148]]
[1] "rs34535972"

[[149]]
[1] "rs13084981"

[[150]]
[1] "rs6599222"

[[151]]
[1] "rs9873213"

[[152]]
[1] "rs41312437"

[[153]]
[1] "rs9851962"

[[154]]
[1] "rs6797133"

[[155]]
[1] "rs7431305"

[[156]]
[1] "rs41261744"

[[157]]
[1] "rs6599230"

[[158]]
[1] "rs41314851"

[[159]]
[1] "rs6599241"

[[160]]
[1] "rs6599242"

[[161]]
[1] "rs6790627"

[[1

[[1]]
[1] "rs568376"

[[2]]
[1] "rs577306"

[[3]]
[1] "rs1834839"

[[4]]
[1] "rs140274879"

[[5]]
[1] "rs114707732"

[[6]]
[1] "rs2298771"

[[7]]
[1] "rs7601520"

[[8]]
[1] "rs66463257"

[[9]]
[1] "rs2020318"

[[10]]
[1] "rs6719077"

[[11]]
[1] "rs6147014"

[[12]]
[1] "rs1019724"

[[13]]
[1] "rs1019723"

[[14]]
[1] "rs72132828"

[[15]]
[1] "rs2126152"

[[16]]
[1] "rs490317"

[[17]]
[1] "rs6432859"

[[18]]
[1] "rs1019722"

[[19]]
[1] "rs6432860"

[[20]]
[1] "2:166897989_G/A"

[[21]]
[1] "rs35257764"

[[22]]
[1] "rs10167228"

[[23]]
[1] "rs6753355"

[[24]]
[1] "rs5836074"

[[25]]
[1] "rs992894"

[[26]]
[1] "rs538921"

[[27]]
[1] "rs7580482"

[[28]]
[1] "rs13421166"

[[29]]
[1] "rs78391141"

[[30]]
[1] "rs1461193"

[[31]]
[1] "rs994399"

[[32]]
[1] "rs6706163"

[[33]]
[1] "rs6750294"

[[34]]
[1] "rs34747425"

[[35]]
[1] "rs3812719"

[[36]]
[1] "rs2195144"

[[37]]
[1] "rs2859817"

[[38]]
[1] "rs8191987"

[[39]]
[1] "rs10930201"

[[40]]
[1] "2:166929347_C/T"

[[41]]
[1] "rs566839"

[[42]]
[1] "rs503872"

[[43]]
[1] "rs4667801"

[[44]]
[1] "rs353127"

[[45]]
[1] "rs353133"

[[46]]
[1] "rs353134"

[[47]]
[1] "rs353135"

[[48]]
[1] "rs353126"

[[49]]
[1] "rs2075704"

[[50]]
[1] "rs12468669"

[[51]]
[1] "rs139906774"

[[52]]
[1] "rs7593568"

[[53]]
[1] "rs112475855"

[[54]]
[1] "rs1965757"

[[55]]
[1] "rs2116658"

[[56]]
[1] "rs7560348"

[[57]]
[1] "rs67443324"

[[58]]
[1] "rs7578237"

[[59]]
[1] "rs10178662"

[[60]]
[1] "rs1946892"

[[61]]
[1] "2:166012356_G/T"

[[62]]
[1] "rs13011371"

[[63]]
[1] "rs13010846"

[[64]]
[1] "rs7572701"

[[65]]
[1] "rs7586796"

[[66]]
[1] "rs6710092"

[[67]]
[1] "rs56124029"

[[68]]
[1] "rs59172781"

[[69]]
[1] "rs6756990"

[[70]]
[1] "rs72264475"

[[71]]
[1] "rs11887309"

[[72]]
[1] "rs11887343"

[[73]]
[1] "rs9333576"

[[74]]
[1] "rs2304710"

[[75]]
[1] "rs7556825"

[[76]]
[1] "rs7579000"

[[77]]
[1] "rs116682517"

[[78]]
[1] "rs11903711"

[[79]]
[1] "rs11903799"

[[80]]
[1] "rs4667794"

[[81]]
[1] "rs4667795"

[[82]]
[1] "rs4667796"

[[83]]
[1] "rs72532011"

[[84]]
[1] "rs11891394"

[[85]]
[1] "rs3754962"

[[86]]
[1] "rs3754961"

[[87]]
[1] "rs3754960"

[[88]]
[1] "rs16822785"

[[89]]
[1] "rs6738031"

[[90]]
[1] "rs113442768"

[[91]]
[1] "rs138641247"

[[92]]
[1] "rs7593207"

[[93]]
[1] "rs2293567"

[[94]]
[1] "rs150350335"

[[95]]
[1] "rs7565062"

[[96]]
[1] "rs56164822"

[[97]]
[1] "rs56821066"

[[98]]
[1] "rs10169523"

[[99]]
[1] "rs10180721"

[[100]]
[1] "2:167060900_C/T"

[[101]]
[1] "rs77944059"

[[102]]
[1] "rs34141377"

[[103]]
[1] "rs2212662"

[[104]]
[1] "rs7595255"

[[105]]
[1] "rs3793936"

[[106]]
[1] "rs77684130"

[[107]]
[1] "2:167085081_T/G"

[[108]]
[1] "rs10199799"

[[109]]
[1] "rs58519838"

[[110]]
[1] "2:167085142_T/A"

[[111]]
[1] "rs61701909"

[[112]]
[1] "rs722204"

[[113]]
[1] "rs6758728"

[[114]]
[1] "rs6746030"

[[115]]
[1] "rs4429487"

[[116]]
[1] "rs9646771"

[[117]]
[1] "rs9646772"

[[118]]
[1] "rs58349687"

[[119]]
[1] "rs4443015"

[[120]]
[1] "rs4447616"

[[121]]
[1] "rs6432899"

[[122]]
[1] "rs6432900"

[[123]]
[1] "rs6432901"

[[124]]
[1] "rs4131162"

[[125]]
[1] "rs4131161"

[[126]]
[1] "rs4386335"

[[127]]
[1] "rs2385539"

[[128]]
[1] "rs9646863"

[[129]]
[1] "rs1046474"

[[130]]
[1] "rs7429945"

[[131]]
[1] "rs1805126"

[[132]]
[1] "rs41315501"

[[133]]
[1] "rs7645178"

[[134]]
[1] "rs41315507"

[[135]]
[1] "rs45505695"

[[136]]
[1] "rs6804918"

[[137]]
[1] "rs6793245"

[[138]]
[1] "rs6799868"

[[139]]
[1] "rs7374138"

[[140]]
[1] "rs6599213"

[[141]]
[1] "rs7430407"

[[142]]
[1] "rs7432766"

[[143]]
[1] "rs7645173"

[[144]]
[1] "rs7645358"

[[145]]
[1] "rs7432532"

[[146]]
[1] "rs7433889"

[[147]]
[1] "rs6599214"

[[148]]
[1] "rs34535972"

[[149]]
[1] "rs13084981"

[[150]]
[1] "rs6599222"

[[151]]
[1] "rs9873213"

[[152]]
[1] "rs41312437"

[[153]]
[1] "rs9851962"

[[154]]
[1] "rs6797133"

[[155]]
[1] "rs7431305"

[[156]]
[1] "rs41261744"

[[157]]
[1] "rs6599230"

[[158]]
[1] "rs41314851"

[[159]]
[1] "rs6599241"

[[160]]
[1] "rs6599242"

[[161]]
[1] "rs6790627"

[[1

#### 2.3 Variant Analysis

For the subsequent analyses, we will include the variants identifed by the `locateVariants()` function.

*De novo* variants are variants listed in the child's genome but not in the mother or father

#### List 1. De novo variants

We look for variants in the child's VCF file that are not found in either the father or the mother.

In [122]:
# List of de novo variants
de_novo <- Reduce(setdiff, list(all_var_c, all_var_f, all_var_m))
de_novo

[[1]]
[1] "rs115015575"

[[2]]
[1] "rs71428927"

[[3]]
[1] "rs56951383"

[[4]]
[1] "rs17766561"

[[5]]
[1] "2:220370245_C/G"

[[6]]
[1] "rs7427447"

[[7]]
[1] "rs190482467"

[[8]]
[1] "rs41312441"

[[9]]
[1] "rs9311195"

[[10]]
[1] "rs35547754"

[[11]]
[1] "3:38948848_TACACACAC/TACACACACAC"

[[12]]
[1] "3:38991874_C/A"

[[13]]
[1] "rs4681032"

[[14]]
[1] "rs78133713"

[[15]]
[1] "12:52115363_T/C"

[[16]]
[1] "rs414355"

[[17]]
[1] "rs303779"

[[18]]
[1] "rs303806"

[[19]]
[1] "rs7133959"

[[20]]
[1] "12:52174388_TGG/T"

[[21]]
[1] "12:52174393_A/T"

[[22]]
[1] "rs74091666"

[[23]]
[1] "rs2302237"

[[1]]
[1] "rs115015575"

[[2]]
[1] "rs71428927"

[[3]]
[1] "rs56951383"

[[4]]
[1] "rs17766561"

[[5]]
[1] "2:220370245_C/G"

[[6]]
[1] "rs7427447"

[[7]]
[1] "rs190482467"

[[8]]
[1] "rs41312441"

[[9]]
[1] "rs9311195"

[[10]]
[1] "rs35547754"

[[11]]
[1] "3:38948848_TACACACAC/TACACACACAC"

[[12]]
[1] "3:38991874_C/A"

[[13]]
[1] "rs4681032"

[[14]]
[1] "rs78133713"

[[15]]
[1] "12:52115363_T/C"

[[16]]
[1] "rs414355"

[[17]]
[1] "rs303779"

[[18]]
[1] "rs303806"

[[19]]
[1] "rs7133959"

[[20]]
[1] "12:52174388_TGG/T"

[[21]]
[1] "12:52174393_A/T"

[[22]]
[1] "rs74091666"

[[23]]
[1] "rs2302237"

There are **23 variants** that are *de novo* in the child.

#### List 2. Variants inherited from parents

We look for variants in the child that are either from the mother or the father.

In [123]:
# First we create a list of the combined variants beteween the mother and father
parents <- Reduce(union, list(all_var_f, all_var_m))

In [124]:
# Next we generate the list of the intersection between the child's variants and that of the parents'
both_par <- Reduce(intersect, list(all_var_c, parents))
both_par

[[1]]
[1] "rs568376"

[[2]]
[1] "rs577306"

[[3]]
[1] "rs140274879"

[[4]]
[1] "rs72132828"

[[5]]
[1] "rs490317"

[[6]]
[1] "2:166897989_G/A"

[[7]]
[1] "2:166900130_T/G"

[[8]]
[1] "rs538921"

[[9]]
[1] "rs78391141"

[[10]]
[1] "rs34747425"

[[11]]
[1] "2:166929347_C/T"

[[12]]
[1] "rs566839"

[[13]]
[1] "rs353133"

[[14]]
[1] "rs353134"

[[15]]
[1] "rs353135"

[[16]]
[1] "rs353126"

[[17]]
[1] "rs2075704"

[[18]]
[1] "rs7593568"

[[19]]
[1] "rs112475855"

[[20]]
[1] "rs1965757"

[[21]]
[1] "rs2116658"

[[22]]
[1] "rs7578237"

[[23]]
[1] "rs10178662"

[[24]]
[1] "rs2060198"

[[25]]
[1] "rs11885001"

[[26]]
[1] "rs1946892"

[[27]]
[1] "rs16850131"

[[28]]
[1] "2:166012356_G/T"

[[29]]
[1] "rs13011371"

[[30]]
[1] "rs13010846"

[[31]]
[1] "rs6710092"

[[32]]
[1] "rs56124029"

[[33]]
[1] "rs59172781"

[[34]]
[1] "rs6756990"

[[35]]
[1] "rs72264475"

[[36]]
[1] "rs11887309"

[[37]]
[1] "rs11887343"

[[38]]
[1] "rs9333576"

[[39]]
[1] "rs2304710"

[[40]]
[1] "rs7556825"

[[41]]
[1] "rs7579000"

[[42]]
[1] "rs116682517"

[[43]]
[1] "rs11903711"

[[44]]
[1] "rs11903799"

[[45]]
[1] "rs4667794"

[[46]]
[1] "rs4667795"

[[47]]
[1] "rs4667796"

[[48]]
[1] "rs11891394"

[[49]]
[1] "rs3754962"

[[50]]
[1] "rs3754961"

[[51]]
[1] "rs3754960"

[[52]]
[1] "rs16822785"

[[53]]
[1] "rs34971284"

[[54]]
[1] "2:167273403_T/C"

[[55]]
[1] "rs6738031"

[[56]]
[1] "rs113442768"

[[57]]
[1] "rs138641247"

[[58]]
[1] "rs7593207"

[[59]]
[1] "rs13417885"

[[60]]
[1] "rs2293567"

[[61]]
[1] "rs150350335"

[[62]]
[1] "rs7565062"

[[63]]
[1] "rs56164822"

[[64]]
[1] "rs10169523"

[[65]]
[1] "rs10180721"

[[66]]
[1] "2:167060900_C/T"

[[67]]
[1] "rs77944059"

[[68]]
[1] "rs34141377"

[[69]]
[1] "rs7595255"

[[70]]
[1] "rs3793936"

[[71]]
[1] "rs77684130"

[[72]]
[1] "2:167085081_T/G"

[[73]]
[1] "rs10199799"

[[74]]
[1] "rs58519838"

[[75]]
[1] "2:167085142_T/A"

[[76]]
[1] "rs61701909"

[[77]]
[1] "rs722204"

[[78]]
[1] "rs6746030"

[[79]]
[1] "rs6747673"

[[80]]
[1] "rs4429487"

[[81]]
[1] "rs9646771"

[[82]]
[1] "rs9646772"

[[83]]
[1] "rs58349687"

[[84]]
[1] "rs4443015"

[[85]]
[1] "rs4447616"

[[86]]
[1] "2:167163736_T/TC"

[[87]]
[1] "2:167167886_A/T"

[[88]]
[1] "rs6432899"

[[89]]
[1] "rs6432900"

[[90]]
[1] "rs6432901"

[[91]]
[1] "rs4131162"

[[92]]
[1] "rs4131161"

[[93]]
[1] "rs2385539"

[[94]]
[1] "rs1046474"

[[95]]
[1] "rs6599213"

[[96]]
[1] "rs41312403"

[[97]]
[1] "rs7430407"

[[98]]
[1] "rs41260344"

[[99]]
[1] "rs7432766"

[[100]]
[1] "rs7645173"

[[101]]
[1] "rs7432532"

[[102]]
[1] "rs1805124"

[[103]]
[1] "rs34535972"

[[104]]
[1] "rs13084981"

[[105]]
[1] "rs6770569"

[[106]]
[1] "rs6599222"

[[107]]
[1] "rs9873213"

[[108]]
[1] "rs9851962"

[[109]]
[1] "rs6781731"

[[110]]
[1] "rs7431305"

[[111]]
[1] "rs41312963"

[[112]]
[1] "rs41261744"

[[113]]
[1] "rs6599230"

[[114]]
[1] "rs6599241"

[[115]]
[1] "rs6599242"

[[116]]
[1] "rs6790627"

[[117]]
[1] "rs6771157"

[[118]]
[1] "rs12632942"

[[119]]
[1] "rs6795970"

[[120]]
[1] "rs59468016"

[[121]]
[1] "rs57326399"

[[122]]
[1] "rs7630989"

[[123]]
[1] "rs7617919"

[[124]]
[1] "rs62244070"

[[125]]
[1] "rs4234134"

[[126]]
[1] "rs4234133"

[[127]]
[1] "rs62244134"

[[128]]
[1] "rs6599266"

[[129]]
[1] "rs13093915"

[[130]]
[1] "rs13099628"

[[131]]
[1] "rs33985936"

[[132]]
[1] "rs5848481"

[[133]]
[1] "rs4484137"

[[134]]
[1] "rs4359754"

[[135]]
[1] "rs4289298"

[[136]]
[1] "rs4073113"

[[137]]
[1] "rs4553926"

[[138]]
[1] "rs4514993"

[[139]]
[1] "rs13076816"

[[140]]
[1] "rs141158768"

[[141]]
[1] "3:38948839_ATC/A"

[[142]]
[1] "rs76050581"

[[143]]
[1] "rs6599267"

[[144]]
[1] "rs6599268"

[[145]]
[1] "rs7429404"

[[146]]
[1] "rs6779297"

[[147]]
[1] "rs6599269"

[[148]]
[1] "rs4676474"

[[149]]
[1] "rs4676583"

[[150]]
[1] "3:25761024_TG/T"

[[151]]
[1] "rs5847370"

[[152]]
[1] "rs2932016"

[[153]]
[1] "rs10780014"

[[154]]
[1] "rs1488991"

[[155]]
[1] "rs1488992"

[[156]]
[1] "rs6550987"

[[157]]
[1] "3:25775422_T/A"

[[158]]
[1] "3:25777564_G/T"

[[159]]
[1] "3:25779035_

[[1]]
[1] "rs568376"

[[2]]
[1] "rs577306"

[[3]]
[1] "rs140274879"

[[4]]
[1] "rs72132828"

[[5]]
[1] "rs490317"

[[6]]
[1] "2:166897989_G/A"

[[7]]
[1] "2:166900130_T/G"

[[8]]
[1] "rs538921"

[[9]]
[1] "rs78391141"

[[10]]
[1] "rs34747425"

[[11]]
[1] "2:166929347_C/T"

[[12]]
[1] "rs566839"

[[13]]
[1] "rs353133"

[[14]]
[1] "rs353134"

[[15]]
[1] "rs353135"

[[16]]
[1] "rs353126"

[[17]]
[1] "rs2075704"

[[18]]
[1] "rs7593568"

[[19]]
[1] "rs112475855"

[[20]]
[1] "rs1965757"

[[21]]
[1] "rs2116658"

[[22]]
[1] "rs7578237"

[[23]]
[1] "rs10178662"

[[24]]
[1] "rs2060198"

[[25]]
[1] "rs11885001"

[[26]]
[1] "rs1946892"

[[27]]
[1] "rs16850131"

[[28]]
[1] "2:166012356_G/T"

[[29]]
[1] "rs13011371"

[[30]]
[1] "rs13010846"

[[31]]
[1] "rs6710092"

[[32]]
[1] "rs56124029"

[[33]]
[1] "rs59172781"

[[34]]
[1] "rs6756990"

[[35]]
[1] "rs72264475"

[[36]]
[1] "rs11887309"

[[37]]
[1] "rs11887343"

[[38]]
[1] "rs9333576"

[[39]]
[1] "rs2304710"

[[40]]
[1] "rs7556825"

[[41]]
[1] "rs7579000"

[[42]]
[1] "rs116682517"

[[43]]
[1] "rs11903711"

[[44]]
[1] "rs11903799"

[[45]]
[1] "rs4667794"

[[46]]
[1] "rs4667795"

[[47]]
[1] "rs4667796"

[[48]]
[1] "rs11891394"

[[49]]
[1] "rs3754962"

[[50]]
[1] "rs3754961"

[[51]]
[1] "rs3754960"

[[52]]
[1] "rs16822785"

[[53]]
[1] "rs34971284"

[[54]]
[1] "2:167273403_T/C"

[[55]]
[1] "rs6738031"

[[56]]
[1] "rs113442768"

[[57]]
[1] "rs138641247"

[[58]]
[1] "rs7593207"

[[59]]
[1] "rs13417885"

[[60]]
[1] "rs2293567"

[[61]]
[1] "rs150350335"

[[62]]
[1] "rs7565062"

[[63]]
[1] "rs56164822"

[[64]]
[1] "rs10169523"

[[65]]
[1] "rs10180721"

[[66]]
[1] "2:167060900_C/T"

[[67]]
[1] "rs77944059"

[[68]]
[1] "rs34141377"

[[69]]
[1] "rs7595255"

[[70]]
[1] "rs3793936"

[[71]]
[1] "rs77684130"

[[72]]
[1] "2:167085081_T/G"

[[73]]
[1] "rs10199799"

[[74]]
[1] "rs58519838"

[[75]]
[1] "2:167085142_T/A"

[[76]]
[1] "rs61701909"

[[77]]
[1] "rs722204"

[[78]]
[1] "rs6746030"

[[79]]
[1] "rs6747673"

[[80]]
[1] "rs4429487"

[[81]]
[1] "rs9646771"

[[82]]
[1] "rs9646772"

[[83]]
[1] "rs58349687"

[[84]]
[1] "rs4443015"

[[85]]
[1] "rs4447616"

[[86]]
[1] "2:167163736_T/TC"

[[87]]
[1] "2:167167886_A/T"

[[88]]
[1] "rs6432899"

[[89]]
[1] "rs6432900"

[[90]]
[1] "rs6432901"

[[91]]
[1] "rs4131162"

[[92]]
[1] "rs4131161"

[[93]]
[1] "rs2385539"

[[94]]
[1] "rs1046474"

[[95]]
[1] "rs6599213"

[[96]]
[1] "rs41312403"

[[97]]
[1] "rs7430407"

[[98]]
[1] "rs41260344"

[[99]]
[1] "rs7432766"

[[100]]
[1] "rs7645173"

[[101]]
[1] "rs7432532"

[[102]]
[1] "rs1805124"

[[103]]
[1] "rs34535972"

[[104]]
[1] "rs13084981"

[[105]]
[1] "rs6770569"

[[106]]
[1] "rs6599222"

[[107]]
[1] "rs9873213"

[[108]]
[1] "rs9851962"

[[109]]
[1] "rs6781731"

[[110]]
[1] "rs7431305"

[[111]]
[1] "rs41312963"

[[112]]
[1] "rs41261744"

[[113]]
[1] "rs6599230"

[[114]]
[1] "rs6599241"

[[115]]
[1] "rs6599242"

[[116]]
[1] "rs6790627"

[[117]]
[1] "rs6771157"

[[118]]
[1] "rs12632942"

[[119]]
[1] "rs6795970"

[[120]]
[1] "rs59468016"

[[121]]
[1] "rs57326399"

[[122]]
[1] "rs7630989"

[[123]]
[1] "rs7617919"

[[124]]
[1] "rs62244070"

[[125]]
[1] "rs4234134"

[[126]]
[1] "rs4234133"

[[127]]
[1] "rs62244134"

[[128]]
[1] "rs6599266"

[[129]]
[1] "rs13093915"

[[130]]
[1] "rs13099628"

[[131]]
[1] "rs33985936"

[[132]]
[1] "rs5848481"

[[133]]
[1] "rs4484137"

[[134]]
[1] "rs4359754"

[[135]]
[1] "rs4289298"

[[136]]
[1] "rs4073113"

[[137]]
[1] "rs4553926"

[[138]]
[1] "rs4514993"

[[139]]
[1] "rs13076816"

[[140]]
[1] "rs141158768"

[[141]]
[1] "3:38948839_ATC/A"

[[142]]
[1] "rs76050581"

[[143]]
[1] "rs6599267"

[[144]]
[1] "rs6599268"

[[145]]
[1] "rs7429404"

[[146]]
[1] "rs6779297"

[[147]]
[1] "rs6599269"

[[148]]
[1] "rs4676474"

[[149]]
[1] "rs4676583"

[[150]]
[1] "3:25761024_TG/T"

[[151]]
[1] "rs5847370"

[[152]]
[1] "rs2932016"

[[153]]
[1] "rs10780014"

[[154]]
[1] "rs1488991"

[[155]]
[1] "rs1488992"

[[156]]
[1] "rs6550987"

[[157]]
[1] "3:25775422_T/A"

[[158]]
[1] "3:25777564_G/T"

[[159]]
[1] "3:25779035_

There are **210 variants** inherited from the parents.

Of the variants inherted from the parents, we can identify the variants' maternal or paternal origins.

* Note, since we are using `setdiff()` these lists' counts do not include overlaps. 

In [125]:
# Just the father's variants
allvars_justpaternal <- Reduce(setdiff, list(all_var_f, all_var_m)) # Only variants found in the father
allvars_paternal <- Reduce(intersect, list(all_var_c, allvars_justpaternal))
allvars_paternal

[[1]]
[1] "rs140274879"

[[2]]
[1] "2:166897989_G/A"

[[3]]
[1] "rs34747425"

[[4]]
[1] "rs2075704"

[[5]]
[1] "rs7593568"

[[6]]
[1] "rs112475855"

[[7]]
[1] "rs1965757"

[[8]]
[1] "rs2116658"

[[9]]
[1] "rs7578237"

[[10]]
[1] "2:166012356_G/T"

[[11]]
[1] "rs116682517"

[[12]]
[1] "rs4667794"

[[13]]
[1] "rs113442768"

[[14]]
[1] "rs150350335"

[[15]]
[1] "rs34141377"

[[16]]
[1] "2:167085142_T/A"

[[17]]
[1] "rs1046474"

[[18]]
[1] "rs7432766"

[[19]]
[1] "rs34535972"

[[20]]
[1] "rs13084981"

[[21]]
[1] "rs6790627"

[[22]]
[1] "rs7630989"

[[23]]
[1] "rs13093915"

[[24]]
[1] "3:25775422_T/A"

[[25]]
[1] "3:25779035_AT/A"

[[26]]
[1] "rs303807"

[[27]]
[1] "rs117671652"

[[28]]
[1] "rs2070720"

[[29]]
[1] "rs2058194"

[[1]]
[1] "rs140274879"

[[2]]
[1] "2:166897989_G/A"

[[3]]
[1] "rs34747425"

[[4]]
[1] "rs2075704"

[[5]]
[1] "rs7593568"

[[6]]
[1] "rs112475855"

[[7]]
[1] "rs1965757"

[[8]]
[1] "rs2116658"

[[9]]
[1] "rs7578237"

[[10]]
[1] "2:166012356_G/T"

[[11]]
[1] "rs116682517"

[[12]]
[1] "rs4667794"

[[13]]
[1] "rs113442768"

[[14]]
[1] "rs150350335"

[[15]]
[1] "rs34141377"

[[16]]
[1] "2:167085142_T/A"

[[17]]
[1] "rs1046474"

[[18]]
[1] "rs7432766"

[[19]]
[1] "rs34535972"

[[20]]
[1] "rs13084981"

[[21]]
[1] "rs6790627"

[[22]]
[1] "rs7630989"

[[23]]
[1] "rs13093915"

[[24]]
[1] "3:25775422_T/A"

[[25]]
[1] "3:25779035_AT/A"

[[26]]
[1] "rs303807"

[[27]]
[1] "rs117671652"

[[28]]
[1] "rs2070720"

[[29]]
[1] "rs2058194"

In [126]:
# Maternal variants
allvars_justmaternal <- Reduce(setdiff, list(all_var_m, all_var_f)) # Only variants found in the mother
allvars_maternal <- Reduce(intersect, list(all_var_c, allvars_justmaternal)) 
allvars_maternal

[[1]]
[1] "2:166900130_T/G"

[[2]]
[1] "rs2060198"

[[3]]
[1] "rs11885001"

[[4]]
[1] "rs16850131"

[[5]]
[1] "rs34971284"

[[6]]
[1] "2:167273403_T/C"

[[7]]
[1] "rs13417885"

[[8]]
[1] "rs6747673"

[[9]]
[1] "2:167163736_T/TC"

[[10]]
[1] "2:167167886_A/T"

[[11]]
[1] "rs41312403"

[[12]]
[1] "rs41260344"

[[13]]
[1] "rs1805124"

[[14]]
[1] "rs6770569"

[[15]]
[1] "rs6781731"

[[16]]
[1] "rs41312963"

[[17]]
[1] "rs6771157"

[[18]]
[1] "rs12632942"

[[19]]
[1] "rs59468016"

[[20]]
[1] "rs57326399"

[[21]]
[1] "rs7617919"

[[22]]
[1] "rs62244070"

[[23]]
[1] "rs62244134"

[[24]]
[1] "rs4289298"

[[25]]
[1] "rs4073113"

[[26]]
[1] "rs4553926"

[[27]]
[1] "rs4514993"

[[28]]
[1] "rs13076816"

[[29]]
[1] "rs141158768"

[[30]]
[1] "3:38948839_ATC/A"

[[31]]
[1] "rs76050581"

[[32]]
[1] "rs6599267"

[[33]]
[1] "rs6599268"

[[34]]
[1] "rs7429404"

[[35]]
[1] "rs6779297"

[[36]]
[1] "rs6599269"

[[37]]
[1] "rs4676474"

[[38]]
[1] "3:25761024_TG/T"

[[39]]
[1] "rs2304698"

[[40]]
[1] "rs1439791"

[[41]]
[1] "rs9943714"

[[42]]
[1] "rs9943809"

[[43]]
[1] "rs2331408"

[[44]]
[1] "17:62042109_T/C"

[[45]]
[1] "rs2302236"

[[1]]
[1] "2:166900130_T/G"

[[2]]
[1] "rs2060198"

[[3]]
[1] "rs11885001"

[[4]]
[1] "rs16850131"

[[5]]
[1] "rs34971284"

[[6]]
[1] "2:167273403_T/C"

[[7]]
[1] "rs13417885"

[[8]]
[1] "rs6747673"

[[9]]
[1] "2:167163736_T/TC"

[[10]]
[1] "2:167167886_A/T"

[[11]]
[1] "rs41312403"

[[12]]
[1] "rs41260344"

[[13]]
[1] "rs1805124"

[[14]]
[1] "rs6770569"

[[15]]
[1] "rs6781731"

[[16]]
[1] "rs41312963"

[[17]]
[1] "rs6771157"

[[18]]
[1] "rs12632942"

[[19]]
[1] "rs59468016"

[[20]]
[1] "rs57326399"

[[21]]
[1] "rs7617919"

[[22]]
[1] "rs62244070"

[[23]]
[1] "rs62244134"

[[24]]
[1] "rs4289298"

[[25]]
[1] "rs4073113"

[[26]]
[1] "rs4553926"

[[27]]
[1] "rs4514993"

[[28]]
[1] "rs13076816"

[[29]]
[1] "rs141158768"

[[30]]
[1] "3:38948839_ATC/A"

[[31]]
[1] "rs76050581"

[[32]]
[1] "rs6599267"

[[33]]
[1] "rs6599268"

[[34]]
[1] "rs7429404"

[[35]]
[1] "rs6779297"

[[36]]
[1] "rs6599269"

[[37]]
[1] "rs4676474"

[[38]]
[1] "3:25761024_TG/T"

[[39]]
[1] "rs2304698"

[[40]]
[1] "rs1439791"

[[41]]
[1] "rs9943714"

[[42]]
[1] "rs9943809"

[[43]]
[1] "rs2331408"

[[44]]
[1] "17:62042109_T/C"

[[45]]
[1] "rs2302236"

References:
* Efficent intersections of lists: https://stackoverflow.com/questions/6630792/intersection-of-lists-in-r
* How to use setdiff() https://www.oreilly.com/library/view/the-r-book/9780470510247/ch002-sec073.html

### 3. Disease-causing variants


Prompt: _Find all the candidate disease-causing mutations (e.g., amino acid coding changes) in the genes (SCN1A, SCN2A, SCN3A, SCN4A, SCN5A, SCN7A, SCN8A, SCN9A, SCN10A, SCN11A, NGLY1, AAAS, GMPPA). These mutations can either be de novo mutations or inherited from the father or the mother.
Please be aware that we identified all mutations in class, not only those resulting in amino acid changes. You might find the function predictCoding() helpful.t_

First, we reset our annotation set.

In [127]:
txdb <- restoreSeqlevels(txdb)
txdb <- renameSeqlevels(txdb, gsub("chr", "", seqlevels(txdb)))
# subset to chromosomes 2, 3, 12, 17 as the genes are located on these chromosomes (data from NCBI, gene database)
txdbgene <- keepSeqlevels(txdb, c("2","3","12","17"))

The homosapiens annotation set has to be coerced into the same format as txdb.

In [128]:
Hsapiens <- renameSeqlevels(Hsapiens, gsub("chr", "", seqlevels(Hsapiens)))

In [129]:
# Verify Hsapiens was fixed
si3 <- seqinfo(Hsapiens)
si3

Seqinfo object with 298 sequences (2 circular) from hg19 genome:
  seqnames        seqlengths isCircular genome
  1                249250621      FALSE   hg19
  2                243199373      FALSE   hg19
  3                198022430      FALSE   hg19
  4                191154276      FALSE   hg19
  5                180915260      FALSE   hg19
  ...                    ...        ...    ...
  21_gl383580_alt      74652      FALSE   hg19
  21_gl383581_alt     116690      FALSE   hg19
  22_gl383582_alt     162811      FALSE   hg19
  22_gl383583_alt      96924      FALSE   hg19
  22_kb663609_alt      74013      FALSE   hg19

Seqinfo object with 298 sequences (2 circular) from hg19 genome:
  seqnames        seqlengths isCircular genome
  1                249250621      FALSE   hg19
  2                243199373      FALSE   hg19
  3                198022430      FALSE   hg19
  4                191154276      FALSE   hg19
  5                180915260      FALSE   hg19
  ...                    ...        ...    ...
  21_gl383580_alt      74652      FALSE   hg19
  21_gl383581_alt     116690      FALSE   hg19
  22_gl383582_alt     162811      FALSE   hg19
  22_gl383583_alt      96924      FALSE   hg19
  22_kb663609_alt      74013      FALSE   hg19

In [130]:
# Sanity check to insure that the child's VCF file has the same chromosome subset as the txdbgene
intersect(seqlevels(vcf_c), seqlevels(txdbgene))

[1] "12" "17" "2"  "3"

[1] "12" "17" "2"  "3"

In [131]:
# Isolate the candidate disease-causing mutations/variants
aa <- predictCoding(vcf_c, txdbgene, Hsapiens)
head(aa, 3)

Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 330 out-of-bound ranges located on sequences
  9542, 12208, 12209, 12210, 12211, 12225, 12226, 12227, 10254, 10255,
  15589, 15590, 15591, 15592, 15593, 15594, 15595, 15596, 15597, 15598,
  15599, 15600, 15601, 15602, 15603, 15606, 15607, 15507, 46258, 46259,
  46260, 46256, 46257, and 64075. Note that ranges located on a sequence
  whose length is unknown (NA) or on a circular sequence are not
  considered out-of-bound (use seqlengths() and isCircular() to get the
  lengths and circularity flags of the underlying sequences). You can use
  trim() to trim these ranges. See ?`trim,GenomicRanges-method` for more
  information.”
Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 330 out-of-bound ranges located on sequences
  9542, 12208, 12209, 12210, 12211, 12225, 12226, 12227, 10254, 10255,
  15589, 15590, 15591, 15592, 15593, 15594, 15595, 15596,

GRanges object with 3 ranges and 17 metadata columns:
             seqnames    ranges strand | paramRangeID            REF
                <Rle> <IRanges>  <Rle> |     <factor> <DNAStringSet>
  rs78391141        2 166904221      - |        SCN1A              G
  rs78391141        2 166904221      - |        SCN1A              G
  rs78391141        2 166904221      - |        SCN1A              G
                            ALT      QUAL                 FILTER      varAllele
             <DNAStringSetList> <numeric>            <character> <DNAStringSet>
  rs78391141                  T    451.95 TruthSensitivityTran..              A
  rs78391141                  T    451.95 TruthSensitivityTran..              A
  rs78391141                  T    451.95 TruthSensitivityTran..              A
                CDSLOC    PROTEINLOC   QUERYID        TXID         CDSID
             <IRanges> <IntegerList> <integer> <character> <IntegerList>
  rs78391141      1086           362        12       12

GRanges object with 3 ranges and 17 metadata columns:
             seqnames    ranges strand | paramRangeID            REF
                <Rle> <IRanges>  <Rle> |     <factor> <DNAStringSet>
  rs78391141        2 166904221      - |        SCN1A              G
  rs78391141        2 166904221      - |        SCN1A              G
  rs78391141        2 166904221      - |        SCN1A              G
                            ALT      QUAL                 FILTER      varAllele
             <DNAStringSetList> <numeric>            <character> <DNAStringSet>
  rs78391141                  T    451.95 TruthSensitivityTran..              A
  rs78391141                  T    451.95 TruthSensitivityTran..              A
  rs78391141                  T    451.95 TruthSensitivityTran..              A
                CDSLOC    PROTEINLOC   QUERYID        TXID         CDSID
             <IRanges> <IntegerList> <integer> <character> <IntegerList>
  rs78391141      1086           362        12       12

In [132]:
# Curate the candidates into a list
aa_c = as.list(names(aa))
aa_c

[[1]]
[1] "rs78391141"

[[2]]
[1] "rs78391141"

[[3]]
[1] "rs78391141"

[[4]]
[1] "rs2060198"

[[5]]
[1] "rs2060198"

[[6]]
[1] "rs2060198"

[[7]]
[1] "rs2060198"

[[8]]
[1] "rs11885001"

[[9]]
[1] "rs11885001"

[[10]]
[1] "rs11885001"

[[11]]
[1] "rs11885001"

[[12]]
[1] "rs1946892"

[[13]]
[1] "rs1946892"

[[14]]
[1] "rs1946892"

[[15]]
[1] "rs1946892"

[[16]]
[1] "rs1946892"

[[17]]
[1] "rs16850131"

[[18]]
[1] "rs16850131"

[[19]]
[1] "rs16850131"

[[20]]
[1] "rs16850131"

[[21]]
[1] "rs16850131"

[[22]]
[1] "2:166012356_G/T"

[[23]]
[1] "2:166012356_G/T"

[[24]]
[1] "2:166012356_G/T"

[[25]]
[1] "2:166012356_G/T"

[[26]]
[1] "2:166012356_G/T"

[[27]]
[1] "rs116682517"

[[28]]
[1] "rs116682517"

[[29]]
[1] "rs116682517"

[[30]]
[1] "rs116682517"

[[31]]
[1] "rs116682517"

[[32]]
[1] "rs34971284"

[[33]]
[1] "2:167273403_T/C"

[[34]]
[1] "rs6738031"

[[35]]
[1] "rs7565062"

[[36]]
[1] "rs7565062"

[[37]]
[1] "2:167060900_C/T"

[[38]]
[1] "rs6746030"

[[39]]
[1] "rs6747673"

[[40]]
[1] "rs6747673"

[[41]]
[1] "rs6747673"

[[42]]
[1] "rs6747673"

[[43]]
[1] "rs9646771"

[[44]]
[1] "rs9646771"

[[45]]
[1] "rs9646771"

[[46]]
[1] "rs9646771"

[[47]]
[1] "rs6432901"

[[48]]
[1] "2:220370245_C/G"

[[49]]
[1] "2:220370245_C/G"

[[50]]
[1] "rs1046474"

[[51]]
[1] "rs1046474"

[[52]]
[1] "rs7430407"

[[53]]
[1] "rs7430407"

[[54]]
[1] "rs7430407"

[[55]]
[1] "rs7430407"

[[56]]
[1] "rs7430407"

[[57]]
[1] "rs7430407"

[[58]]
[1] "rs7430407"

[[59]]
[1] "rs7430407"

[[60]]
[1] "rs7430407"

[[61]]
[1] "rs7430407"

[[62]]
[1] "rs7430407"

[[63]]
[1] "rs7430407"

[[64]]
[1] "rs7430407"

[[65]]
[1] "rs7430407"

[[66]]
[1] "rs7430407"

[[67]]
[1] "rs1805124"

[[68]]
[1] "rs1805124"

[[69]]
[1] "rs1805124"

[[70]]
[1] "rs1805124"

[[71]]
[1] "rs1805124"

[[72]]
[1] "rs1805124"

[[73]]
[1] "rs1805124"

[[74]]
[1] "rs1805124"

[[75]]
[1] "rs1805124"

[[76]]
[1] "rs1805124"

[[77]]
[1] "rs1805124"

[[78]]
[1] "rs1805124"

[[79]]
[1] "rs1805124"

[[80]]
[1] "rs1805124"

[[81]]
[1] "rs1805124"

[[82]]
[1] "rs6599230"

[[83]]
[1] "rs6599230"

[[84]]
[1] "rs6599230"

[[85]]
[1] "rs6599230"

[[86]]
[1] "rs6599230"

[[87]]
[1] "rs6599230"

[[88]]
[1] "rs6599230"

[[89]]
[1] "rs6599230"

[[90]]
[1] "rs6599230"

[[91]]
[1] "rs6599230"

[[92]]
[1] "rs6599230"

[[93]]
[1] "rs6599230"

[[94]]
[1] "rs6599230"

[[95]]
[1] "rs6599241"

[[96]]
[1] "rs6599242"

[[97]]
[1] "rs6790627"

[[98]]
[1] "rs6771157"

[[99]]
[1] "rs12632942"

[[100]]
[1] "rs6795970"

[[101]]
[1] "rs59468016"

[[102]]
[1] "rs57326399"

[[103]]
[1] "rs7630989"

[[104]]
[1] "rs7617919"

[[105]]
[1] "rs62244070"

[[106]]
[1] "rs62244134"

[[107]]
[1] "rs33985936"

[[108]]
[1] "rs33985936"

[[109]]
[1] "rs4073113"

[[110]]
[1] "3:25761024_TG/T"

[[111]]
[1] "3:25761024_TG/T"

[[112]]
[1] "3:25761024_TG/T"

[[113]]
[1] "3:25775422_T/A"

[[114]]
[1] "3:25775422_T/A"

[[115]]
[1] "3:25775422_T/A"

[[116]]
[1] "3:25775422_T/A"

[[117]]
[1] "3:25777564_G/T"

[[118]]
[1] "3:25777564_G/T"

[[119]]
[1] "3:25777564_G/T"

[[120]]
[1] "rs4761829"

[[121]]
[1] "rs4761829"

[[122]]
[1] "rs4761829"

[[123]]
[1] "rs4761829"

[[124]]
[1] "rs4761829"

[[125]]
[1] "12:52115363_T/C"

[[126]]
[1] "12:52115363_T/C"

[[127]]
[1] "12:52115363_T/C"

[[128]]
[1] "12:52115363_T/C"

[[129]]
[1] "12:52115363_T/C"

[[130]]
[1] "rs303815"

[[131]]
[1] "rs303815"

[[132]]
[1] "rs60637"

[[133]]
[1] "rs60637"

[[134]]
[1] "rs1546808"

[[135]]
[1] "rs1546808"

[[136]]
[1] "rs2070720"

[[137]]
[1] "rs2058194"

[[138]]
[1] "rs6504191"

[[1]]
[1] "rs78391141"

[[2]]
[1] "rs78391141"

[[3]]
[1] "rs78391141"

[[4]]
[1] "rs2060198"

[[5]]
[1] "rs2060198"

[[6]]
[1] "rs2060198"

[[7]]
[1] "rs2060198"

[[8]]
[1] "rs11885001"

[[9]]
[1] "rs11885001"

[[10]]
[1] "rs11885001"

[[11]]
[1] "rs11885001"

[[12]]
[1] "rs1946892"

[[13]]
[1] "rs1946892"

[[14]]
[1] "rs1946892"

[[15]]
[1] "rs1946892"

[[16]]
[1] "rs1946892"

[[17]]
[1] "rs16850131"

[[18]]
[1] "rs16850131"

[[19]]
[1] "rs16850131"

[[20]]
[1] "rs16850131"

[[21]]
[1] "rs16850131"

[[22]]
[1] "2:166012356_G/T"

[[23]]
[1] "2:166012356_G/T"

[[24]]
[1] "2:166012356_G/T"

[[25]]
[1] "2:166012356_G/T"

[[26]]
[1] "2:166012356_G/T"

[[27]]
[1] "rs116682517"

[[28]]
[1] "rs116682517"

[[29]]
[1] "rs116682517"

[[30]]
[1] "rs116682517"

[[31]]
[1] "rs116682517"

[[32]]
[1] "rs34971284"

[[33]]
[1] "2:167273403_T/C"

[[34]]
[1] "rs6738031"

[[35]]
[1] "rs7565062"

[[36]]
[1] "rs7565062"

[[37]]
[1] "2:167060900_C/T"

[[38]]
[1] "rs6746030"

[[39]]
[1] "rs6747673"

[[40]]
[1] "rs6747673"

[[41]]
[1] "rs6747673"

[[42]]
[1] "rs6747673"

[[43]]
[1] "rs9646771"

[[44]]
[1] "rs9646771"

[[45]]
[1] "rs9646771"

[[46]]
[1] "rs9646771"

[[47]]
[1] "rs6432901"

[[48]]
[1] "2:220370245_C/G"

[[49]]
[1] "2:220370245_C/G"

[[50]]
[1] "rs1046474"

[[51]]
[1] "rs1046474"

[[52]]
[1] "rs7430407"

[[53]]
[1] "rs7430407"

[[54]]
[1] "rs7430407"

[[55]]
[1] "rs7430407"

[[56]]
[1] "rs7430407"

[[57]]
[1] "rs7430407"

[[58]]
[1] "rs7430407"

[[59]]
[1] "rs7430407"

[[60]]
[1] "rs7430407"

[[61]]
[1] "rs7430407"

[[62]]
[1] "rs7430407"

[[63]]
[1] "rs7430407"

[[64]]
[1] "rs7430407"

[[65]]
[1] "rs7430407"

[[66]]
[1] "rs7430407"

[[67]]
[1] "rs1805124"

[[68]]
[1] "rs1805124"

[[69]]
[1] "rs1805124"

[[70]]
[1] "rs1805124"

[[71]]
[1] "rs1805124"

[[72]]
[1] "rs1805124"

[[73]]
[1] "rs1805124"

[[74]]
[1] "rs1805124"

[[75]]
[1] "rs1805124"

[[76]]
[1] "rs1805124"

[[77]]
[1] "rs1805124"

[[78]]
[1] "rs1805124"

[[79]]
[1] "rs1805124"

[[80]]
[1] "rs1805124"

[[81]]
[1] "rs1805124"

[[82]]
[1] "rs6599230"

[[83]]
[1] "rs6599230"

[[84]]
[1] "rs6599230"

[[85]]
[1] "rs6599230"

[[86]]
[1] "rs6599230"

[[87]]
[1] "rs6599230"

[[88]]
[1] "rs6599230"

[[89]]
[1] "rs6599230"

[[90]]
[1] "rs6599230"

[[91]]
[1] "rs6599230"

[[92]]
[1] "rs6599230"

[[93]]
[1] "rs6599230"

[[94]]
[1] "rs6599230"

[[95]]
[1] "rs6599241"

[[96]]
[1] "rs6599242"

[[97]]
[1] "rs6790627"

[[98]]
[1] "rs6771157"

[[99]]
[1] "rs12632942"

[[100]]
[1] "rs6795970"

[[101]]
[1] "rs59468016"

[[102]]
[1] "rs57326399"

[[103]]
[1] "rs7630989"

[[104]]
[1] "rs7617919"

[[105]]
[1] "rs62244070"

[[106]]
[1] "rs62244134"

[[107]]
[1] "rs33985936"

[[108]]
[1] "rs33985936"

[[109]]
[1] "rs4073113"

[[110]]
[1] "3:25761024_TG/T"

[[111]]
[1] "3:25761024_TG/T"

[[112]]
[1] "3:25761024_TG/T"

[[113]]
[1] "3:25775422_T/A"

[[114]]
[1] "3:25775422_T/A"

[[115]]
[1] "3:25775422_T/A"

[[116]]
[1] "3:25775422_T/A"

[[117]]
[1] "3:25777564_G/T"

[[118]]
[1] "3:25777564_G/T"

[[119]]
[1] "3:25777564_G/T"

[[120]]
[1] "rs4761829"

[[121]]
[1] "rs4761829"

[[122]]
[1] "rs4761829"

[[123]]
[1] "rs4761829"

[[124]]
[1] "rs4761829"

[[125]]
[1] "12:52115363_T/C"

[[126]]
[1] "12:52115363_T/C"

[[127]]
[1] "12:52115363_T/C"

[[128]]
[1] "12:52115363_T/C"

[[129]]
[1] "12:52115363_T/C"

[[130]]
[1] "rs303815"

[[131]]
[1] "rs303815"

[[132]]
[1] "rs60637"

[[133]]
[1] "rs60637"

[[134]]
[1] "rs1546808"

[[135]]
[1] "rs1546808"

[[136]]
[1] "rs2070720"

[[137]]
[1] "rs2058194"

[[138]]
[1] "rs6504191"

Next, we create a list of the unique candidate disease-causing mutations.

In [133]:
# Isolate the unique variants
aa_c = as.list(unique(names(aa)))
aa_c

[[1]]
[1] "rs78391141"

[[2]]
[1] "rs2060198"

[[3]]
[1] "rs11885001"

[[4]]
[1] "rs1946892"

[[5]]
[1] "rs16850131"

[[6]]
[1] "2:166012356_G/T"

[[7]]
[1] "rs116682517"

[[8]]
[1] "rs34971284"

[[9]]
[1] "2:167273403_T/C"

[[10]]
[1] "rs6738031"

[[11]]
[1] "rs7565062"

[[12]]
[1] "2:167060900_C/T"

[[13]]
[1] "rs6746030"

[[14]]
[1] "rs6747673"

[[15]]
[1] "rs9646771"

[[16]]
[1] "rs6432901"

[[17]]
[1] "2:220370245_C/G"

[[18]]
[1] "rs1046474"

[[19]]
[1] "rs7430407"

[[20]]
[1] "rs1805124"

[[21]]
[1] "rs6599230"

[[22]]
[1] "rs6599241"

[[23]]
[1] "rs6599242"

[[24]]
[1] "rs6790627"

[[25]]
[1] "rs6771157"

[[26]]
[1] "rs12632942"

[[27]]
[1] "rs6795970"

[[28]]
[1] "rs59468016"

[[29]]
[1] "rs57326399"

[[30]]
[1] "rs7630989"

[[31]]
[1] "rs7617919"

[[32]]
[1] "rs62244070"

[[33]]
[1] "rs62244134"

[[34]]
[1] "rs33985936"

[[35]]
[1] "rs4073113"

[[36]]
[1] "3:25761024_TG/T"

[[37]]
[1] "3:25775422_T/A"

[[38]]
[1] "3:25777564_G/T"

[[39]]
[1] "rs4761829"

[[40]]
[1] "12:52115363_T/C"

[[41]]
[1] "rs303815"

[[42]]
[1] "rs60637"

[[43]]
[1] "rs1546808"

[[44]]
[1] "rs2070720"

[[45]]
[1] "rs2058194"

[[46]]
[1] "rs6504191"

[[1]]
[1] "rs78391141"

[[2]]
[1] "rs2060198"

[[3]]
[1] "rs11885001"

[[4]]
[1] "rs1946892"

[[5]]
[1] "rs16850131"

[[6]]
[1] "2:166012356_G/T"

[[7]]
[1] "rs116682517"

[[8]]
[1] "rs34971284"

[[9]]
[1] "2:167273403_T/C"

[[10]]
[1] "rs6738031"

[[11]]
[1] "rs7565062"

[[12]]
[1] "2:167060900_C/T"

[[13]]
[1] "rs6746030"

[[14]]
[1] "rs6747673"

[[15]]
[1] "rs9646771"

[[16]]
[1] "rs6432901"

[[17]]
[1] "2:220370245_C/G"

[[18]]
[1] "rs1046474"

[[19]]
[1] "rs7430407"

[[20]]
[1] "rs1805124"

[[21]]
[1] "rs6599230"

[[22]]
[1] "rs6599241"

[[23]]
[1] "rs6599242"

[[24]]
[1] "rs6790627"

[[25]]
[1] "rs6771157"

[[26]]
[1] "rs12632942"

[[27]]
[1] "rs6795970"

[[28]]
[1] "rs59468016"

[[29]]
[1] "rs57326399"

[[30]]
[1] "rs7630989"

[[31]]
[1] "rs7617919"

[[32]]
[1] "rs62244070"

[[33]]
[1] "rs62244134"

[[34]]
[1] "rs33985936"

[[35]]
[1] "rs4073113"

[[36]]
[1] "3:25761024_TG/T"

[[37]]
[1] "3:25775422_T/A"

[[38]]
[1] "3:25777564_G/T"

[[39]]
[1] "rs4761829"

[[40]]
[1] "12:52115363_T/C"

[[41]]
[1] "rs303815"

[[42]]
[1] "rs60637"

[[43]]
[1] "rs1546808"

[[44]]
[1] "rs2070720"

[[45]]
[1] "rs2058194"

[[46]]
[1] "rs6504191"

### 4.0 Variant Type

Definitions:
* (i) **homozygous variant**: when a subject inherits the same allele of a given gene from each parent. The result is a pair of matching genes.
* (ii) **heterozygous variant**: when a subject inherits a different allele of the gene from one of the parents. 
* (iii) **compound heterozygous variant**: when a subject inherts a different allele of the gene from each parent AND both alleles have different mutations.

Prompt: _Use the GT field of the VCF files from the child and identify which of the candidate disease-causing variants (identified in section 3) in the child are homozygous and heterozygous variants._

In [166]:
# Load tidyverse
library(tidyverse)

Warning message:
“Your system is mis-configured: ‘/etc/localtime’ is not a symlink”
Warning message:
“Your system is mis-configured: ‘/etc/localtime’ is not a symlink”
Warning message:
“It is strongly recommended to set envionment variable TZ to ‘Etc/UCT’ (or equivalent)”
Warning message:
“It is strongly recommended to set envionment variable TZ to ‘Etc/UCT’ (or equivalent)”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::collapse()   masks Biostrings::collapse(), IRanges::collapse()
✖ d

In [153]:
# Pull up the child's VCF files and GT data
GT <- geno(vcf_c)[["GT"]]
#GT[1:10,]
#dim(GT)

In [154]:
# Examine at the all the child's variants' genotypes
GT

,van2554
rs34871628,1/1
rs67967509,0/1
rs568376,1/1
rs577306,1/1
rs140274879,0/1
rs72132828,0/1
rs490317,1/1
2:166897989_G/A,0/1
2:166900130_T/G,0/1
rs115015575,1/1


,van2554
rs34871628,1/1
rs67967509,0/1
rs568376,1/1
rs577306,1/1
rs140274879,0/1
rs72132828,0/1
rs490317,1/1
2:166897989_G/A,0/1
2:166900130_T/G,0/1
rs115015575,1/1


In [160]:
# Subset the child's 46 disease causing variants 
#GT <- as.data.frame(GT)
dzvar <- aa_c
dzvar_subset <- subset(GT, rownames(GT) %in% dzvar)

In [161]:
dzvar_subset

,van2554
rs78391141,0/1
rs2060198,0/1
rs11885001,0/1
rs1946892,1/1
rs16850131,0/1
2:166012356_G/T,0/1
rs116682517,0/1
rs34971284,0/1
2:167273403_T/C,0/1
rs6738031,1/1


,van2554
rs78391141,0/1
rs2060198,0/1
rs11885001,0/1
rs1946892,1/1
rs16850131,0/1
2:166012356_G/T,0/1
rs116682517,0/1
rs34971284,0/1
2:167273403_T/C,0/1
rs6738031,1/1


In [169]:
# Convert to dataframe for ease of use
dzvar_subset2 <- as.data.frame(dzvar_subset)

In [172]:
# Isolate the homozygous variants (1/1)
dzvar_subset2 %>% filter(van2554 == "1/1")

,van2554
,<chr>
rs1946892,1/1
rs6738031,1/1
rs7565062,1/1
rs6746030,1/1
rs9646771,1/1
rs6432901,1/1
rs6599230,1/1
rs6599241,1/1
rs6599242,1/1


,van2554
,<chr>
rs1946892,1/1
rs6738031,1/1
rs7565062,1/1
rs6746030,1/1
rs9646771,1/1
rs6432901,1/1
rs6599230,1/1
rs6599241,1/1
rs6599242,1/1


There are **13** homozygous variants in the child's possible disease causing mutation variants.

In [174]:
# Isolate the heterozygous variants (0/1 or 1/0)

dzvar_subset2 %>% filter(van2554 == "0/1" | van2554 == "0/1")

,van2554
,<chr>
rs78391141,0/1
rs2060198,0/1
rs11885001,0/1
rs16850131,0/1
2:166012356_G/T,0/1
rs116682517,0/1
rs34971284,0/1
2:167273403_T/C,0/1
2:167060900_C/T,0/1


,van2554
,<chr>
rs78391141,0/1
rs2060198,0/1
rs11885001,0/1
rs16850131,0/1
2:166012356_G/T,0/1
rs116682517,0/1
rs34971284,0/1
2:167273403_T/C,0/1
2:167060900_C/T,0/1


There are **33** heterozygous variants in the child's possible disease causing mutation variants.